# 자동차 환경 빅데이터 3차
- 과제2 : 건설기계 배출가스 관리 체계 구축
- 과제4 : 배출가스 전문정비제도 관리체계 구축
- 과제5 : 자동차 환경 빅데이터 제작사 공동 활용체게 구축

# Library

In [1]:
import pandas as pd
import os, math
import numpy as np
from datetime import datetime
import pyexasol

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [3]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [4]:
pub_fold = 'D:/data/processing/big3/raw/public'
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
folds = {
    'pub_fold':pub_fold, 
    'raw_fold':raw_fold, 
    'raw2_fold':raw2_fold, 
    'raw4_fold':raw4_fold, 
    'raw5_fold':raw5_fold, 
    'an_fold':an_fold, 
    'an2_fold:':an2_fold, 
    'an4_fold:':an4_fold, 
    'an5_fold:':an5_fold, 
    'df_fold':df_fold, 
    'df2_fold':df2_fold, 
    'df4_fold':df4_fold, 
    'df5_fold':df5_fold, 
    }
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an_fold': 'D:/data/processing/big3/analysis',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df_fold': 'D:/data/processing/big3/df',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# server

In [5]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
wz = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSZ')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

## 분석 결과 출력 옵션

In [477]:
# y : 출력
# 그 외 : 미출력
export_opt = 'y'

# EDA
+ 과제2
    - 건설기계 인증자료
    - 건설기계 인증대장
+ 과제4
    - 자동차전문정비 : STD_TB_ERR_RES
    - 자동차전문정비상세 : STD_TB_ERR_RES_DET
+ 과제5
    - 구조변경검사 : STD_DLM_TB_ERP_GUBUN_BAECHUL

# 과제4

## 로드

### [❌] 등록정보

In [7]:
# # 5m 31.4s
# car = wd.export_to_pandas("SELECT VIN, MANP_MNG_NO, VHCL_ERSR_YN, FRST_REG_YMD, YRIDNW, VHCTY_CD, PURPS_CD2, BSPL_STDG_CD, EXHST_GAS_GRD_CD FROM STD_CEG_CAR_MIG;")
# car_ch_col = {
#     'VIN':'차대번호', 
#     'MANP_MNG_NO':'제원관리번호', 
#     'VHCL_ERSR_YN':'차량말소여부',
#     'FRST_REG_YMD':'최초등록일자',
#     'YRIDNW':'연식', 
#     'VHCTY_CD':'차종', 
#     'PURPS_CD2':'용도', 
#     'BSPL_STDG_CD':'법정동코드', 
#     'EXHST_GAS_GRD_CD':'배출가스등급', 

#     # 'EXHST_GAS_CERT_NO':'배출가스인증번호',
#     # 'VHCL_FBCTN_YMD':'제작일자', 
#     # 'VHCL_MNG_NO':'차량관리번호', 
#     # 'VHRNO':'자동차등록번호',
#     # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
#     # 'VHCL_OWNR_CL_CD':'소유자구분',  
# }
# carr = car.rename(columns=car_ch_col)

# print(car.shape)
# print(carr.columns)
# print('data load : STD_CEG_CAR_MIG')
# print()

# #### 등급, 차종, 용도 코드 변환
# print(carr['배출가스등급'].unique())
# ## 배출가스등급 코드 변환
# grd_dict = {
#     'A0501':'1', 
#     'A0502':'2', 
#     'A0503':'3', 
#     'A0504':'4', 
#     'A0505':'5', 
#     'A05T2':'2',
#     'A05T3':'3', 
#     'A05T4':'4', 
#     'A05T5':'5', 
#     'A05X':'X', 
# }
# carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
# print(carr['배출가스등급'].unique())

# print(carr['차종'].unique())
# # 4.0s
# ## 차종 코드 변환
# cd_dict = {
#     'A31M':'이륜', 
#     'A31P':'승용', 
#     'A31S':'특수', 
#     'A31T':'화물', 
#     'A31V':'승합'
# }
# carr['차종'] = carr['차종'].replace(cd_dict)
# print(carr['차종'].unique())

# print(carr['용도'].unique())
# ## 용도 코드 변환
# purps_dict = {
#     'A08P':'개인용', 
#     'A08B':'영업용', 
#     'A08O':'관용',
# }
# carr['용도'] = carr['용도'].replace(purps_dict)
# print(carr['용도'].unique())

### [❌] 제원정보

In [8]:
# # 4.1s
# src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, MNFCTR_NM, VHCTY_TY_CD2, VHCNM, VHCL_FRM, EGIN_TY, TOTL_WGHT, DSPLVL, EGIN_OTPT  FROM STD_CEG_CAR_SRC_MIG;")
# src_ch_col = {
#     'MANP_MNG_NO':'제원관리번호', 
#     'FUEL_CD':'연료', 
#     'MNFCTR_NM':'제작사명', 
#     'VHCTY_TY_CD2':'차종유형', 
#     'VHCNM':'차명', 
#     'VHCL_FRM':'자동차형식', 
#     'EGIN_TY':'엔진형식', 
#     'TOTL_WGHT':'총중량', 
#     'DSPLVL':'배기량', 
#     'EGIN_OTPT':'엔진출력', 

#     # 'VHCTY_CL_CD':'차종분류',
#     # 'CRYNG_WGHT':'적재중량',
# }
# srcr = src.rename(columns=src_ch_col)

# print(srcr.shape)
# print(srcr.columns)
# print('data load : STD_CEG_CAR_SRC_MIG')
# print()

# #### 연료, 차종유형 코드 변화
# print(srcr['연료'].unique())
# # 7.9s
# ## 연료 코드 변환
# fuel_dict = {
#     'A90GS':'휘발유', 
#     'A91DS':'경유',
#     'A92LP':'LPG(액화석유가스)', 
#     'A90GH':'휘발유 하이브리드', 
#     'A93EV':'전기', 
#     'A91DH':'경유 하이브리드', 
#     'A92CN':'CNG(압축천연가스)', 
#     'A93HD':'수소', 
#     'A92LH':'LPG 하이브리드', 
#     'A94OT':'기타연료', 
#     'A92CH':'CNG 하이브리드',
#     'A90AC':'알코올', 
#     'A93SH':'태양열', 
#     'A91KS':'등유', 
#     'A92LN':'LNG(액화천연가스)', 
#     'A90PH':'플러그인 하이브리드', 
# }
# srcr['연료'] = srcr['연료'].replace(fuel_dict)
# print(srcr['연료'].unique())

# print(srcr['차종유형'].unique())
# ## 차종유형 코드 변환
# ty_dict = {
#     'A30C':'경형', 
#     'A30L':'대형', 
#     'A30M':'중형', 
#     'A30S':'소형',
# }
# srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
# print(srcr['차종유형'].unique())

### 등록&제원(RH제공)

#### from DB

In [9]:
# 3m 46.9s
cs_raw = wz.export_to_pandas("SELECT CLS_YMD, VIN, MANP_MNG_NO, VHCL_ERSR_YN, FRST_REG_YMD, YRIDNW, VHCTY_CD, PURPS_CD2, BSPL_STDG_CD, EXHST_GAS_GRD_CD, FUEL_CD, MNFCTR_NM, VHCTY_TY_CD2, VHCNM, VHCL_FRM, EGIN_TY, TOTL_WGHT, DSPLVL, EGIN_OTPT, STDG_CTPV_NM, STDG_SGG_NM, VHCL_MNG_NO FROM STD_CEG_GRD_LST WHERE CLS_YMD = (SELECT MAX(CLS_YMD) FROM STD_CEG_GRD_LST);")
cs_raw_dict = {
    'CLS_YMD':'원천데이터생성일자',

    'VIN':'차대번호', 
    'MANP_MNG_NO':'제원관리번호', 
    'VHCL_ERSR_YN':'차량말소여부',
    'FRST_REG_YMD':'최초등록일자',
    'YRIDNW':'연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'FUEL_CD':'연료',
    'MNFCTR_NM':'제작사명', 
    'VHCTY_TY_CD2':'차종유형', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'TOTL_WGHT':'총중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',

    # 'EXHST_GAS_CERT_NO':'배출가스인증번호',
    # 'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    # 'VHRNO':'자동차등록번호',
    # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
    # 'VHCL_OWNR_CL_CD':'소유자구분',

    # 'VHCTY_CL_CD':'차종분류',
    # 'CRYNG_WGHT':'적재중량',

    'STDG_CTPV_NM':'시도명',
    'STDG_SGG_NM':'시군구명',
    # 'LEM_KND_CD':'저공해조치종류코드',
    # 'STRCT_CHG_YN':'구조변경여부',
    # 'FUEL_G_NM':'연료_G_명',
    # 'TKCAR_PSCP':'승차정원',
    # 'OVRLH':'전장',
    # 'OVRWD':'전폭',
    # 'OVRHG':'전고',
    # 'FBCTN_NTN_NM':'제작국명',
    # 'BOHUM_YN':'보훈여부',
}

cs = cs_raw.rename(columns=cs_raw_dict)

print(cs_raw['CLS_YMD'].unique()[0])
print()
cs = cs.drop('원천데이터생성일자', axis=1)
print(cs.columns)
print(cs.shape)
print('data load : STD_CEG_GRD_LST')

20231231

Index(['차대번호', '제원관리번호', '차량말소여부', '최초등록일자', '연식', '차종', '용도', '법정동코드',
       '배출가스등급', '연료', '제작사명', '차종유형', '차명', '자동차형식', '엔진형식', '총중량', '배기량',
       '엔진출력', '시도명', '시군구명', '차량관리번호'],
      dtype='object')
(31498871, 21)
data load : STD_CEG_GRD_LST


In [10]:
cs.drop_duplicates().shape

(31498871, 21)

In [11]:
# 31.8s
# 코드 변환
## 배출가스등급 코드 변환
print(cs['배출가스등급'].unique())
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
cs['배출가스등급'] = cs['배출가스등급'].replace(grd_dict)
print(cs['배출가스등급'].unique())
print()

## 차종 코드 변환
print(cs['차종'].unique())
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs['차종'] = cs['차종'].replace(cd_dict)
print(cs['차종'].unique())
print()

## 용도 코드 변환
print(cs['용도'].unique())
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs['용도'] = cs['용도'].replace(purps_dict)
cs['용도'].unique()
print()

# ## 소유자구분 코드 변환
# print(cs['소유자구분'].unique())
# ownr_dict = {
#     'A27B':'사업자', 
#     'A27F':'외국인', 
#     'A27L':'법인', 
#     'A27O':'기타', 
#     'A27R':'주민', 
# }
# cs['소유자구분'] = cs['소유자구분'].replace(ownr_dict)
# print(cs['소유자구분'].unique())
# print()

## 연료 코드 변환
print(cs['연료'].unique())
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs['연료'] = cs['연료'].replace(fuel_dict)
print(cs['연료'].unique())
print()

## 차종유형 코드 변환
print(cs['차종유형'].unique())
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
cs['차종유형'] = cs['차종유형'].replace(ty_dict)
print(cs['차종유형'].unique())

['A0505' 'A0504' 'A0501' 'A0503' 'A0502' '9999']
['5' '4' '1' '3' '2' '9999']

['A31T' 'A31P' 'A31V' 'A31S']
['화물' '승용' '승합' '특수']

['A08P' 'A08B' 'A08O']

['A91DS' 'A92LP' 'A90GS' 'A92CN' 'A90GH' 'A93EV' '9999' 'A92LH' 'A94OT'
 'A91DH' 'A93HD' 'A92CH' 'A90AC' 'A91KS' 'A93SH' 'A92LN']
['경유' 'LPG(액화석유가스)' '휘발유' 'CNG(압축천연가스)' '휘발유 하이브리드' '전기' '9999'
 'LPG 하이브리드' '기타연료' '경유 하이브리드' '수소' 'CNG 하이브리드' '알코올' '등유' '태양열'
 'LNG(액화천연가스)']

['A30S' 'A30L' 'A30M' 'A30C' '9999']
['소형' '대형' '중형' '경형' '9999']


### 정기검사

In [12]:
# 3m 34.9s
# 4m 50.5s(2024.01.04)
# jgt = wz.export_to_pandas("SELECT VIN, FDRM_INSP_YMD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_DRVNG_DSTNC FROM VSYSZ.STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wz.export_to_pandas("SELECT VIN, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_INSP_KND_CD, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_JT_YN1, FDRM_NLOD_SMO_JT_YN2, FDRM_NLOD_SMO_JT_YN3, FDRM_NLOD_SMO_JT_YN4, FDRM_NLOD_SMO_JT_YN5, FDRM_NLOD_SMO_JT_YN6, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_MSTVL4, FDRM_NLOD_SMO_MSTVL5, FDRM_NLOD_SMO_MSTVL6, FDRM_NLOD_SMO_PRMVL1, FDRM_NLOD_SMO_PRMVL2, FDRM_NLOD_SMO_PRMVL3, FDRM_NLOD_SMO_PRMVL4, FDRM_NLOD_SMO_PRMVL5, FDRM_NLOD_SMO_PRMVL6 FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_YMD':'검사일자', 
    'FDRM_DRVNG_DSTNC':'주행거리', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_JGMT':'검사판정', 

    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    'FDRM_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    'FDRM_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    'FDRM_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    'FDRM_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    'FDRM_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    'FDRM_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    'FDRM_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    'FDRM_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    'FDRM_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    'FDRM_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    'FDRM_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
}
jgtr = jgt.rename(columns=jgt_ch_col)

print(jgtr.shape)
print(jgtr.columns)
print('data load : STD_TB_JGT_HIS')

(67890732, 24)
Index(['차대번호', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연판정1', '무부하매연판정2',
       '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연측정치1',
       '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6',
       '무부하매연허용치1', '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5',
       '무부하매연허용치6'],
      dtype='object')
data load : STD_TB_JGT_HIS


In [13]:
# 2011.10.26 ~ 2023.11.21(2024.01.04)
jgtr['검사일자'].describe()

count                       67890732
unique                          3629
top       2017-10-10 00:00:00.000000
freq                           38867
Name: 검사일자, dtype: object

#### 검사종류, 검사방법 코드 변환

In [14]:
print(jgtr['검사종류'].unique())
## 검사종류 코드 변환
knd_dict = {
    'A1901':'재검사(1부재검사)', 
    'A1902':'재검사(2부재검사)', 
    'A1903':'정기(경과)검사', 
    'A1904':'정기검사', 
}
jgtr['검사종류'] = jgtr['검사종류'].replace(knd_dict)
print(jgtr['검사종류'].unique())

print(jgtr['검사방법'].unique())
## 검사방법 코드 변환
mthd_dict = {
    'A18A':'무부하검사(TSI)',
    'A18B':'무부하검사(급가속)',
    'A18C':'무부하검사(정지가동)', 
}
jgtr['검사방법'] = jgtr['검사방법'].replace(mthd_dict)
print(jgtr['검사방법'].unique())

['A1904' 'A1902' 'A1903' 'A1901']
['정기검사' '재검사(2부재검사)' '정기(경과)검사' '재검사(1부재검사)']
['A18A' 'A18C' 'A18B']
['무부하검사(TSI)' '무부하검사(정지가동)' '무부하검사(급가속)']


### 정밀검사

In [15]:
# 2m 51.0s
# 10m 32.4s(2024.01.04)
# eet = wb.export_to_pandas("SELECT VIN, PRCINSP_YMD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME_ME WHERE ROWNUM <= 10000;") # 테스트용
# eet = wd.export_to_pandas("SELECT VIN, PRCINSP_YMD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME;")
eet = wz.export_to_pandas("SELECT VIN, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PRCINSP_KND_CD, PRCINSP_MSRMT_MTHD_CD, PRCINSP_JGMT, PREC_NLOD_SMO_JT_YN1, PREC_NLOD_SMO_JT_YN2, PREC_NLOD_SMO_JT_YN3, PREC_NLOD_SMO_JT_YN4, PREC_NLOD_SMO_JT_YN5, PREC_NLOD_SMO_JT_YN6, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_MSTVL4, PREC_NLOD_SMO_MSTVL5, PREC_NLOD_SMO_MSTVL6, PREC_NLOD_SMO_PRMVL1, PREC_NLOD_SMO_PRMVL2, PREC_NLOD_SMO_PRMVL3, PREC_NLOD_SMO_PRMVL4, PREC_NLOD_SMO_PRMVL5, PREC_NLOD_SMO_PRMVL6 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_JGMT':'검사판정', 

    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PREC_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    'PREC_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    'PREC_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    'PREC_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    'PREC_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    'PREC_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    'PREC_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    'PREC_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    'PREC_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    'PREC_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    'PREC_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    'PREC_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
}
eetr = eet.rename(columns=eet_ch_col)

print(eetr.shape)
print(eetr.columns)
print('data load : STD_TB_EET_HIS_ME')

(105849354, 24)
Index(['차대번호', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연판정1', '무부하매연판정2',
       '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연측정치1',
       '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6',
       '무부하매연허용치1', '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5',
       '무부하매연허용치6'],
      dtype='object')
data load : STD_TB_EET_HIS_ME


In [16]:
# 2002.05.20 ~ 2023.07.31
# 2002.05.20 ~ 2023.11.20(2024.01.04)
eetr['검사일자'].describe()

count                      105849354
unique                          6449
top       2019-05-07 00:00:00.000000
freq                           74583
Name: 검사일자, dtype: object

#### 검사종류, 검사방법 코드 변환

In [17]:
print(eetr['검사종류'].unique())
## 검사종류 코드 변환
knd_dict = {
    'A21C01':'구조변경',
    'A21C02':'구조변경검사',
    'A21E01':'배출재검사',
    'A21E02':'배출재검사(구변)',
    'A21E03':'배출재검사(임시)',
    'A21O01':'관능재검사',
    'A21O02':'관능재검사(임시)',
    'A21P01':'정밀검사',
    'A21P02':'정밀검사(경과)',
    'A21R01':'1부재검사(종합)',
    'A21R02':'2부재검사(종합)',
    'A21R03':'재검사(1부재검사)',
    'A21R04':'재검사(2부재검사)',
    'A21R05':'재검사(관능)',
    'A21R06':'재검사(배출)',
    'A21S01':'특정검사',
    'A21S02':'특정검사(경과)',
    'A21S03':'특정재검사(관능)',
    'A21S04':'특정재검사(배출)',
    'A21T01':'종합(경과:정밀)',
    'A21T02':'종합(정밀)',
    'A21T03':'종합(정밀) 1부재검사',
    'A21T04':'종합(정밀) 2부재검사',
    'A21T05':'종합검사',
    'A21T06':'종합검사(경과)',
    'A21TMP':'임시검사',
    'A21001':'관능재검사',
    'A21002':'관능재검사(임시)',
}
eetr['검사종류'] = eetr['검사종류'].replace(knd_dict)
print(eetr['검사종류'].unique())

print(eetr['검사방법'].unique())
## 검사방법 코드 변환
mthd_dict = {
    'A2301':'무부하검사(급가속)',
    'A2302':'부하검사(ASM-Idling)',
    'A2303':'무부하검사(TSI)',
    'A2304':'부하검사(LUG DOWN)',
    'A2305':'부하검사(KD-147)',
    'A2306':'부하검사(LUG DOWN)',
    'A2307':'부하검사(ASM2525)',
}
eetr['검사방법'] = eetr['검사방법'].replace(mthd_dict)
print(eetr['검사방법'].unique())

['A21S04' 'A21T06' 'A21R02' 'A21T05' 'A21P01' 'A21S01' 'A21001' 'A21E01'
 'A21P02' 'A21S03' 'A21S02' 'A21R01' 'A21T02' 'A21T04' 'A21T01' 'A21T03'
 'A21C02' 'A21E02' '9999' 'A21R05' 'A21R03' 'A21R06' 'A21002' 'A21TMP'
 'A21R04']
['특정재검사(배출)' '종합검사(경과)' '2부재검사(종합)' '종합검사' '정밀검사' '특정검사' '관능재검사' '배출재검사'
 '정밀검사(경과)' '특정재검사(관능)' '특정검사(경과)' '1부재검사(종합)' '종합(정밀)' '종합(정밀) 2부재검사'
 '종합(경과:정밀)' '종합(정밀) 1부재검사' '구조변경검사' '배출재검사(구변)' '9999' '재검사(관능)'
 '재검사(1부재검사)' '재검사(배출)' '관능재검사(임시)' '임시검사' '재검사(2부재검사)']
['A2304' 'A2306' 'A2301' 'A2303' 'A2307' 'A2305' 'A2302' '9999']
['부하검사(LUG DOWN)' '무부하검사(급가속)' '무부하검사(TSI)' '부하검사(ASM2525)' '부하검사(KD-147)'
 '부하검사(ASM-Idling)' '9999']


### 법정동코드

In [18]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM, STDG_CTPV_CD, STDG_SGG_CD FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드',
    'STDG_SGG_CD':'시군구코드', 
}
coder = code.rename(columns=code_ch_col)

print(coder.shape)
print(coder.columns)
print('data load : STD_BJCD_INFO')

(47916, 5)
Index(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], dtype='object')
data load : STD_BJCD_INFO


### 1\. 전문정비(STD_TB_ERR_RES)

In [478]:
er_dict = {
    'DRVNG_DSTNC':'주행거리', 
    'VHRNO':'차량번호', 
    'VIN':'차대번호', 
    'LOD_COX_MEVLU':'부하일산화탄소측정치', 
    'LOD_HC_MEVLU':'부하탄화수소측정치', 
    'LOD_NOX_MEVLU':'부하질소산화물측정치', 
    'LOD_SMO_MEVLU':'부하매연측정치', 
    'LOD_TQ_MEVLU':'부하토크측정치', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
    'NOLOD_COX_MEVLU':'무부하일산화탄소측정치', 
    'NOLOD_HC_MEVLU':'무부하탄화수소측정치', 
    'NOLOD_AIR_EXCRT_MEVLU':'무부하공기과잉률측정치', 
    'NOLOD_SMO_MEVLU':'무부하매연측정치', 
}

In [479]:
# 0.3s
name = 'STD_TB_ERR_RES_20231011'
file_name = f'{name}.csv'
er = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
er.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54617 entries, 0 to 54616
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DRVNG_DSTNC            54617 non-null  float64
 1   VHRNO                  54617 non-null  object 
 2   VIN                    51051 non-null  object 
 3   LOD_COX_MEVLU          53884 non-null  object 
 4   LOD_HC_MEVLU           53886 non-null  object 
 5   LOD_NOX_MEVLU          53883 non-null  object 
 6   LOD_SMO_MEVLU          53941 non-null  object 
 7   LOD_TQ_MEVLU           53819 non-null  object 
 8   OVHUL_YMD              54617 non-null  int64  
 9   SPCY_OVHUL_BIZPLC_NM   54617 non-null  object 
 10  OVHUL_RCPT_NO          54617 non-null  int64  
 11  PRCINSP_RINSP_RCPT_NO  54617 non-null  object 
 12  NOLOD_COX_MEVLU        53870 non-null  object 
 13  NOLOD_HC_MEVLU         53871 non-null  object 
 14  NOLOD_AIR_EXCRT_MEVLU  28484 non-null  object 
 15  NO

In [480]:
err = er.rename(columns=er_dict)
err.columns

Index(['주행거리', '차량번호', '차대번호', '부하일산화탄소측정치', '부하탄화수소측정치', '부하질소산화물측정치',
       '부하매연측정치', '부하토크측정치', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호',
       '무부하일산화탄소측정치', '무부하탄화수소측정치', '무부하공기과잉률측정치', '무부하매연측정치'],
      dtype='object')

#### 데이터 확인

In [481]:
set(err['정비접수번호'].unique())

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}

In [482]:
# shape
# (54617, 16) (2024.01.04)
err.shape, len(err['차대번호'].unique())

((54617, 16), 43771)

##### [출력] 컬럼, 데이터 정보들

In [483]:
er_cols = pd.DataFrame({'컬럼명':er_dict.keys(), '컬럼내용':er_dict.values()})
er_cols

,컬럼명,컬럼내용
0,DRVNG_DSTNC,주행거리
1,VHRNO,차량번호
2,VIN,차대번호
3,LOD_COX_MEVLU,부하일산화탄소측정치
4,LOD_HC_MEVLU,부하탄화수소측정치
5,LOD_NOX_MEVLU,부하질소산화물측정치
6,LOD_SMO_MEVLU,부하매연측정치
7,LOD_TQ_MEVLU,부하토크측정치
8,OVHUL_YMD,정비일자
9,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명


In [484]:
temp = err.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = err.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(er_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,DRVNG_DSTNC,주행거리,0,54617
1,2,VHRNO,차량번호,0,54617
2,3,VIN,차대번호,3566,54617
3,4,LOD_COX_MEVLU,부하일산화탄소측정치,733,54617
4,5,LOD_HC_MEVLU,부하탄화수소측정치,731,54617
5,6,LOD_NOX_MEVLU,부하질소산화물측정치,734,54617
6,7,LOD_SMO_MEVLU,부하매연측정치,676,54617
7,8,LOD_TQ_MEVLU,부하토크측정치,798,54617
8,9,OVHUL_YMD,정비일자,0,54617
9,10,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,0,54617


In [485]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2024.01.19'

In [486]:
# ttemp.to_excel(os.path.join(an4_fold, f'전문정비_데이터정보({today_date}).xlsx'), index=False)

#### 데이터 수정

In [487]:
err['정비일자'].isnull().sum()

0

In [488]:
err['정비일자'].describe()

count      54617.00
mean    20083896.95
std        16435.80
min     20060727.00
25%     20070801.00
50%     20080507.00
75%     20091109.00
max     20121010.00
Name: 정비일자, dtype: float64

In [489]:
err['정비일자'] = pd.to_datetime(err['정비일자'], format='%Y%m%d', errors='coerce')
err['정비일자'].head()

0   2008-01-10
1   2008-01-23
2   2008-01-08
3   2008-01-18
4   2008-01-07
Name: 정비일자, dtype: datetime64[ns]

In [490]:
err['정비일자'].describe()

count                            54617
mean     2008-10-29 10:43:41.117966592
min                2006-07-27 00:00:00
25%                2007-08-01 00:00:00
50%                2008-05-07 00:00:00
75%                2009-11-09 00:00:00
max                2012-10-10 00:00:00
Name: 정비일자, dtype: object

### 2\. 전문정비상세(STD_TB_ERR_RES_DET)

In [491]:
erd_dict = {
    'VHRNO':'차량번호', 
    'VIN':'차대번호', 
    'OVHUL_CN':'정비부품명', # 내용상 정비부품명(테이블 정의서 : 정비내용)
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_CMPNT_NM':'정비내용', # 내용상 정비내용(테이블 정의서 : 정비부품명)
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
}

In [492]:
# 0.3s
name = 'STD_TB_ERR_RES_DET_20231011'
file_name = f'{name}.csv'
erd = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
erd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106819 entries, 0 to 106818
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VHRNO                  106818 non-null  object 
 1   VIN                    94670 non-null   object 
 2   OVHUL_CN               106817 non-null  object 
 3   OVHUL_YMD              106818 non-null  float64
 4   SPCY_OVHUL_BIZPLC_NM   106818 non-null  object 
 5   OVHUL_CMPNT_NM         106817 non-null  object 
 6   OVHUL_RCPT_NO          106818 non-null  float64
 7   PRCINSP_RINSP_RCPT_NO  0 non-null       float64
dtypes: float64(3), object(5)
memory usage: 6.5+ MB


In [493]:
erdr = erd.rename(columns=erd_dict)
erdr.columns

Index(['차량번호', '차대번호', '정비부품명', '정비일자', '전문정비사업소명', '정비내용', '정비접수번호',
       '정밀재검사접수번호'],
      dtype='object')

In [494]:
# shape
# (106819, 8) (2024.01.04)
erdr.shape, len(erdr['차대번호'].unique())

((106819, 8), 40927)

#### 정비내용 수정

In [495]:
erdr['정비내용'].unique()

array(['연료장치', '엔진계통', '흡기계통', '배출가스 재순환장치', '점화계통', '배출가스 전환장치',
       '연료증발가스 방지장치', '각종센서', '블로바이가스 환원장치', '배기계통', nan], dtype=object)

In [496]:
erdr['정비내용'] = erdr['정비내용'].str.replace(' ', '')
erdr['정비내용'].unique()

array(['연료장치', '엔진계통', '흡기계통', '배출가스재순환장치', '점화계통', '배출가스전환장치',
       '연료증발가스방지장치', '각종센서', '블로바이가스환원장치', '배기계통', nan], dtype=object)

#### 정비부품명 수정

In [497]:
erdr['정비부품명'].unique()

array(['연료분사펌프(조속기조정\x7f봉인)', '엔진오일', '공회전속도', '연료휠터', '노즐(디젤인젝터포함)',
       '공연비(혼합비)', '에어크리너', '연료압력조절기', '엔진정비', '냉각계통', 'EGR제어용 서머밸브',
       '과급기', '점화시기(분사시기) 조정', 'EGR밸브', '디젤산화촉매(DOC)',
       '정화조절밸브(Purge Control Valve)', '배전기(Distributor)', '냉각수온센서',
       '산소센서', '기화기(믹서포함)', '정화용촉매(Catalytic Converter)', '공기유량센서',
       '전자제어장치(ECU)', '블로바이가스 호스', '캐니스터 및 필터', '점화플러그', '점화배선', '배기관',
       '드로틀바디', '인젝터(휘발유)', 'PCV밸브', '소음기', '드로틀포지션센서', '매연여과장치(DPF)',
       nan], dtype=object)

In [498]:
repair_info_dict = {
    '연료분사펌프(조속기조정\x7f봉인)':'연료분사펌프(조속기조정,봉인)', 
    '점화시기(분사시기) 조정':'점화시기(분사시기)조정', 
}
erdr['정비부품명'] = erdr['정비부품명'].replace(repair_info_dict)
erdr['정비부품명'] = erdr['정비부품명'].str.replace(' ', '')
erdr['정비부품명'].unique()

array(['연료분사펌프(조속기조정,봉인)', '엔진오일', '공회전속도', '연료휠터', '노즐(디젤인젝터포함)',
       '공연비(혼합비)', '에어크리너', '연료압력조절기', '엔진정비', '냉각계통', 'EGR제어용서머밸브',
       '과급기', '점화시기(분사시기)조정', 'EGR밸브', '디젤산화촉매(DOC)',
       '정화조절밸브(PurgeControlValve)', '배전기(Distributor)', '냉각수온센서', '산소센서',
       '기화기(믹서포함)', '정화용촉매(CatalyticConverter)', '공기유량센서', '전자제어장치(ECU)',
       '블로바이가스호스', '캐니스터및필터', '점화플러그', '점화배선', '배기관', '드로틀바디', '인젝터(휘발유)',
       'PCV밸브', '소음기', '드로틀포지션센서', '매연여과장치(DPF)', nan], dtype=object)

#### 정비내용 수정

In [499]:
erdr['정비내용'].unique()

array(['연료장치', '엔진계통', '흡기계통', '배출가스재순환장치', '점화계통', '배출가스전환장치',
       '연료증발가스방지장치', '각종센서', '블로바이가스환원장치', '배기계통', nan], dtype=object)

In [500]:
# 정비내용 : 연료장치와 엔지계통 뒤바뀌어 있음
print(erdr.loc[erdr['정비내용'] == '연료장치', '정비부품명'].unique())
print()
print(erdr.loc[erdr['정비내용'] == '엔진계통', '정비부품명'].unique())


['연료분사펌프(조속기조정,봉인)' '연료휠터' '노즐(디젤인젝터포함)' '공연비(혼합비)' '연료압력조절기' '기화기(믹서포함)'
 '인젝터(휘발유)']

['엔진오일' '공회전속도' '엔진정비' '냉각계통' '전자제어장치(ECU)']


In [501]:
erdr['정비내용'] = erdr['정비내용'].replace({'연료장치':'엔진계통', '엔진계통':'연료장치'})

In [502]:
# 정비내용 : 연료장치와 엔지계통 뒤바뀌어 있음
print(erdr.loc[erdr['정비내용'] == '연료장치', '정비부품명'].unique())
print()
print(erdr.loc[erdr['정비내용'] == '엔진계통', '정비부품명'].unique())


['엔진오일' '공회전속도' '엔진정비' '냉각계통' '전자제어장치(ECU)']

['연료분사펌프(조속기조정,봉인)' '연료휠터' '노즐(디젤인젝터포함)' '공연비(혼합비)' '연료압력조절기' '기화기(믹서포함)'
 '인젝터(휘발유)']


#### 데이터 확인

In [503]:
erdr.shape

(106819, 8)

In [504]:
set(erdr['정비접수번호'].unique())

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, nan}

##### [출력] 컬럼, 데이터 정보들

In [505]:
erd_cols = pd.DataFrame({'컬럼명':erd_dict.keys(), '컬럼내용':erd_dict.values()})
erd_cols

,컬럼명,컬럼내용
0,VHRNO,차량번호
1,VIN,차대번호
2,OVHUL_CN,정비부품명
3,OVHUL_YMD,정비일자
4,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명
5,OVHUL_CMPNT_NM,정비내용
6,OVHUL_RCPT_NO,정비접수번호
7,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호


In [506]:
temp = erdr.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = erdr.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(erd_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,VHRNO,차량번호,1,106819
1,2,VIN,차대번호,12149,106819
2,3,OVHUL_CN,정비부품명,2,106819
3,4,OVHUL_YMD,정비일자,1,106819
4,5,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,1,106819
5,6,OVHUL_CMPNT_NM,정비내용,2,106819
6,7,OVHUL_RCPT_NO,정비접수번호,1,106819
7,8,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호,106819,106819


In [507]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2024.01.19'

In [508]:
# ttemp.to_excel(os.path.join(an4_fold, f'전문정비상세_데이터정보({today_date}).xlsx'), index=False)

#### 데이터 수정

In [509]:
erdr['정비일자'].isnull().sum()

1

In [510]:
erdr['정비일자'].describe()

count     106818.00
mean    20083911.85
std        15852.89
min     20060727.00
25%     20070829.00
50%     20080529.00
75%     20090921.00
max     20120928.00
Name: 정비일자, dtype: float64

In [511]:
erdr['정비일자'] = pd.to_datetime(erdr['정비일자'], format='%Y%m%d', errors='coerce')
erdr['정비일자'].head()

0   2007-02-08
1   2006-08-11
2   2006-08-11
3   2006-08-11
4   2006-12-11
Name: 정비일자, dtype: datetime64[ns]

In [512]:
erdr['정비일자'].describe()

count                           106818
mean     2008-10-30 13:16:57.738583296
min                2006-07-27 00:00:00
25%                2007-08-29 00:00:00
50%                2008-05-29 00:00:00
75%                2009-09-21 00:00:00
max                2012-09-28 00:00:00
Name: 정비일자, dtype: object

### 전문정비&상세 병합
- 기준 : 자동차등록정보, 정비접수번호, 전문정비사업소명

In [513]:
err.shape, erdr.shape

((54617, 16), (106819, 8))

In [514]:
len(err['차대번호'].unique()), len(erdr['차대번호'].unique())

(43771, 40927)

In [515]:
name = ['차대번호', '차량번호', '정비접수번호', '정밀재검사접수번호', '전문정비사업소명']
err[name].isnull().sum(), erdr[name].isnull().sum()

(차대번호         3566
 차량번호            0
 정비접수번호          0
 정밀재검사접수번호       0
 전문정비사업소명        0
 dtype: int64,
 차대번호          12149
 차량번호              1
 정비접수번호            1
 정밀재검사접수번호    106819
 전문정비사업소명          1
 dtype: int64)

In [516]:
err['정밀재검사접수번호'].dtype, erdr['정밀재검사접수번호'].dtype

(dtype('O'), dtype('float64'))

In [517]:
# 0.1s
# 기준 : '자동차등록번호', '정비접수번호', '전문정비사업소명'
# 방법 : inner
ered = err[['차량번호', '차대번호', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호']].merge(erdr, on=['차량번호', '정비접수번호', '전문정비사업소명'], how='inner')
ered.shape

(106804, 11)

In [518]:
ered.columns

Index(['차량번호', '차대번호_x', '정비일자_x', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호_x',
       '차대번호_y', '정비부품명', '정비일자_y', '정비내용', '정밀재검사접수번호_y'],
      dtype='object')

In [519]:
ered = ered.rename(columns={'차대번호_x':'차대번호', '차대번호_y':'차대번호_상세', '정비일자_x':'정비일자', '정비일자_y':'정비일자_상세', '정밀재검사접수번호_x':'정밀재검사접수번호', '정밀재검사접수번호_y':'정밀재검사접수번호_상세'})
ered.columns

Index(['차량번호', '차대번호', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호', '차대번호_상세',
       '정비부품명', '정비일자_상세', '정비내용', '정밀재검사접수번호_상세'],
      dtype='object')

In [520]:
ered[ered['정비일자'] != ered['정비일자_상세']].shape

(8, 11)

In [521]:
ered[['차량번호', '정비접수번호', '전문정비사업소명']].isnull().sum()

차량번호        0
정비접수번호      0
전문정비사업소명    0
dtype: int64

In [522]:
ered['정비일자'].dtype

dtype('<M8[ns]')

In [523]:
ered['정비일자'].head()

0   2008-01-10
1   2008-01-23
2   2008-01-08
3   2008-01-18
4   2008-01-18
Name: 정비일자, dtype: datetime64[ns]

In [524]:
ered['정비일자'] = ered['정비일자'].astype('str')
ered['정비일자'].head()

0    2008-01-10
1    2008-01-23
2    2008-01-08
3    2008-01-18
4    2008-01-18
Name: 정비일자, dtype: object

In [525]:
ered['정비일자'] = ered['정비일자'].str.replace('-', '')
ered['정비일자'].head()

0    20080110
1    20080123
2    20080108
3    20080118
4    20080118
Name: 정비일자, dtype: object

In [526]:
ered['정비일자'] = ered['정비일자'].astype('int')

In [527]:
ered['정비일자'].dtype

dtype('int32')

### 1\. 전문 + 등록

In [528]:
# print(carr.shape)
# carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
# carr['최초등록일자'].dtype
# print(carr.shape)
# print()

# # 30.5s
# carm = carr.sort_values(['최초등록일자', '차량말소여부'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
# print(carm.shape)
# print(carm['차량말소여부'].value_counts(dropna=False))
# print()

# # 3.7s
# carm = carm[carm['차량말소여부'] == 'N'].reset_index(drop=True)
# print(carm.shape)
# print()

# # 44.3s
# print(ered.shape)
# erc = ered.merge(carm, on='차대번호', how='left')
# print(erc.shape)

In [529]:
# 3.7s
print('cs :', cs['차량말소여부'].value_counts(dropna=False))
csm = cs[cs['차량말소여부'] == 'N'].reset_index(drop=True)
print('csm.shape :', csm.shape)
print()

# 44.3s
print('ered.shape :', ered.shape)
erc = ered.merge(csm, on='차대번호', how='left')
print('erc.shape :', erc.shape)

cs : 차량말소여부
N    25949517
Y     5549354
Name: count, dtype: int64


csm.shape : (25949517, 21)

ered.shape : (106804, 11)
erc.shape : (106804, 31)


In [530]:
erc.columns

Index(['차량번호', '차대번호', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호', '차대번호_상세',
       '정비부품명', '정비일자_상세', '정비내용', '정밀재검사접수번호_상세', '제원관리번호', '차량말소여부',
       '최초등록일자', '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형',
       '차명', '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '차량관리번호'],
      dtype='object')

In [531]:
cs['배출가스등급'].isnull().sum()

0

In [532]:
erc['배출가스등급'].isnull().sum()

99988

In [533]:
ered.shape[0] - erc['배출가스등급'].isnull().sum()

6816

### 2\. + 제원

In [534]:
# print(erc.shape)
# ercs = erc.merge(srcr, on='제원관리번호', how='left')
# print(ercs.shape)

In [535]:
ercs = erc.copy()

In [536]:
ercs['법정동코드'].isnull().sum()

99988

### 3\. + 법정동코드

In [537]:
# ercs['법정동코드'] = ercs['법정동코드'].astype('str')
# # 26.2s
# ercs['시도코드'] = ercs['법정동코드'].str[:2]
# ercs['시군구코드'] = ercs['법정동코드'].str[2:5]
# ercs['시도코드'] = pd.to_numeric(ercs['시도코드'], errors='coerce')
# ercs['시군구코드'] = pd.to_numeric(ercs['시군구코드'], errors='coerce')
# ercs['법정동코드'] = pd.to_numeric(ercs['법정동코드'], errors='coerce')

In [538]:
ercs.shape

(106804, 31)

In [539]:
# # 0.4s
# ec = ercs.merge(coder[['법정동코드', '시도', '시군구']], on='법정동코드', how='left')
# print(ec.shape)
# print(ec[['시도', '시군구']].isnull().sum())

In [540]:
ec = ercs.copy()
print(ec[['시도명', '시군구명']].isnull().sum())

시도명     99988
시군구명    99988
dtype: int64


In [541]:
ec.columns

Index(['차량번호', '차대번호', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호', '차대번호_상세',
       '정비부품명', '정비일자_상세', '정비내용', '정밀재검사접수번호_상세', '제원관리번호', '차량말소여부',
       '최초등록일자', '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형',
       '차명', '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '차량관리번호'],
      dtype='object')

### 정기&정밀 병합

In [542]:
jgtr['정기정밀구분'] = '정기'
eetr['정기정밀구분'] = '정밀'

In [543]:
jgtr.shape, eetr.shape

((67890732, 25), (105849354, 25))

In [544]:
# 22.0s
ins = pd.concat([jgtr, eetr], ignore_index=True)
ins.shape

(173740086, 25)

In [545]:
ins.columns

Index(['차대번호', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연판정1', '무부하매연판정2',
       '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연측정치1',
       '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6',
       '무부하매연허용치1', '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5',
       '무부하매연허용치6', '정기정밀구분'],
      dtype='object')

In [546]:
# 18.7s
ins = ins[[
    '차대번호',
    '검사일자',
    '주행거리',
    '검사종류',
    '검사방법',
    '검사판정',
    '무부하매연측정치1',
    '무부하매연허용치1',
    '무부하매연판정1',
    '무부하매연측정치2',
    '무부하매연허용치2',
    '무부하매연판정2',
    '무부하매연측정치3',
    '무부하매연허용치3',
    '무부하매연판정3',
    '무부하매연측정치4',
    '무부하매연허용치4',
    '무부하매연판정4',
    '무부하매연측정치5',
    '무부하매연허용치5',
    '무부하매연판정5',
    '무부하매연측정치6',
    '무부하매연허용치6',
    '무부하매연판정6',
    '정기정밀구분',
]]

In [547]:
ec.shape

(106804, 31)

In [548]:
ec.columns

Index(['차량번호', '차대번호', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호', '차대번호_상세',
       '정비부품명', '정비일자_상세', '정비내용', '정밀재검사접수번호_상세', '제원관리번호', '차량말소여부',
       '최초등록일자', '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형',
       '차명', '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '차량관리번호'],
      dtype='object')

In [549]:
# # 
# bf_er = ec[[
#     '차량번호',
#     '차대번호',
#     '정비일자',
#     '전문정비사업소명',
#     '정비접수번호',
#     # '정밀재검사접수번호',
#     # '차대번호_상세',
#     '정비내용',
#     # '정비일자_상세',
#     '정비부품명',
#     # '정밀재검사접수번호_상세',
#     '제원관리번호',
#     # '차량말소여부',
#     # '최초등록일자',
#     '연식',
#     '차종',
#     '용도',
#     '법정동코드',
#     '배출가스등급',
#     '연료',
#     '제작사명',
#     '차종유형',
#     '차명',
#     '자동차형식',
#     '엔진형식',
#     '총중량',
#     '배기량',
#     '엔진출력',
#     '시도',
#     '시군구',
# ]]
# bf_er.shape

In [550]:
# 
bf_er = ec[[
    '차량번호',
    '차대번호',
    '정비일자',
    '전문정비사업소명',
    '정비접수번호',
    # '정밀재검사접수번호',
    # '차대번호_상세',
    '정비내용',
    # '정비일자_상세',
    '정비부품명',
    # '정밀재검사접수번호_상세',
    '제원관리번호',
    # '차량말소여부',
    # '최초등록일자',
    '연식',
    '차종',
    '용도',
    '법정동코드',
    '배출가스등급',
    '연료',
    '제작사명',
    '차종유형',
    '차명',
    '자동차형식',
    '엔진형식',
    '총중량',
    '배기량',
    '엔진출력',
    '시도명',
    '시군구명',
]]
bf_er.shape

(106804, 24)

### 전문정비 추가 데이터

#### 데이터 확인

In [551]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_det_ex_202312061021.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv',
 '과제4_전처리완료_20231221_PM_033316.csv',
 '과제4_전처리완료_20240105_PM_090451.csv',
 '과제4_전처리완료_20240108_PM_100718.csv',
 '과제4_전처리완료_20240116_PM_042303.csv',
 '과제4_전처리완료_20240118_AM_091837.csv',
 '과제4_전처리완료_20240118_AM_103015.csv',
 '과제4_전처리완료_20240118_AM_111805.csv',
 '과제4_전처리완료_20240118_PM_025809.csv',
 '과제4_전처리완료_20240119_AM_095418.csv',
 '과제4_전처리완료_전문정비통합_20240109_AM_025533.csv',
 '과제4_전처리완료_전문정비통합_20240116_PM_042303.csv',
 '과제4_전처리완료_전문정비통합_20240118_AM_091837.csv',
 '과제4_전처리완료_전문정비통합_20240118_AM_103015.csv',
 '과제4_전처리완료_전문정비통합_20240118_AM_111805.csv',
 '과제4_전처리완료_전문정비통합_20240118_PM_025809.csv',
 '과제4_전처리완료_전문정비통합_20240119_AM_095418.csv',
 '전국_전문정비사업자_현황_

In [552]:
len(os.listdir(raw4_fold))

33

In [553]:
print('tb_rsd_ge_list'.upper())
print('tb_rsd_ge_list_ex'.upper())
print('tb_rsd_ge_sub'.upper())
print('tb_rse_out_result'.upper())
print('tb_rse_reg'.upper())
print('tb_rse_std'.upper())
print('tb_rsm_org'.upper())
print('tb_rso_reg'.upper())
print('tb_us_org'.upper())

TB_RSD_GE_LIST
TB_RSD_GE_LIST_EX
TB_RSD_GE_SUB
TB_RSE_OUT_RESULT
TB_RSE_REG
TB_RSE_STD
TB_RSM_ORG
TB_RSO_REG
TB_US_ORG


In [554]:
raw4_fold

'D:/data/processing/big3/raw/BD4'

In [555]:
# 0.3s
name = 'tb_rsd_ge_list_202311071035'
file_name = f'{name}.csv'
rsd_ge_list = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71065 entries, 0 to 71064
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ge_idx             71065 non-null  int64  
 1   rsd_month          71065 non-null  int64  
 2   car_ser_no         71065 non-null  object 
 3   snt                61457 non-null  float64
 4   ge_cnt             71065 non-null  int64  
 5   search1            11799 non-null  float64
 6   search2            2351 non-null   float64
 7   search3            52534 non-null  float64
 8   search4            61061 non-null  float64
 9   car_reg_no         71065 non-null  object 
 10  car_nm             71065 non-null  object 
 11  usage              71065 non-null  object 
 12  car_year           71065 non-null  int64  
 13  car_type_nm        71065 non-null  object 
 14  car_weight         71065 non-null  int64  
 15  tot_weight         71065 non-null  int64  
 16  engine_capacity    710

In [556]:
stat1 = rsd_ge_list.isnull().sum().reset_index()
stat1 = stat1.rename(columns={'index':'컬럼명', 0:'결측치 개수'})
stat1['총 행수'] = rsd_ge_list.shape[0]
stat1['번호'] = [x for x in range(1, stat1.shape[0] + 1)]
stat1 = stat1[['번호', '컬럼명', '결측치 개수', '총 행수']]
stat1.head()

,번호,컬럼명,결측치 개수,총 행수
0,1,ge_idx,0,71065
1,2,rsd_month,0,71065
2,3,car_ser_no,0,71065
3,4,snt,9608,71065
4,5,ge_cnt,0,71065


In [557]:
for file_name in os.listdir(raw4_fold):
    print(file_name)

tb_rsd_ge_list_202311071035.csv
tb_rsd_ge_list_ex_202311071035.csv
tb_rsd_ge_sub_202311071035.csv
tb_rse_det_ex_202312061021.csv
tb_rse_out_result_202311071036.csv
tb_rse_reg_202311071036.csv
tb_rse_std_202311071036.csv
tb_rsm_org_202311071037.csv
tb_rso_reg_202311071037.csv
tb_us_org_202311071037.csv
과제4_전처리완료_20231221_PM_033316.csv
과제4_전처리완료_20240105_PM_090451.csv
과제4_전처리완료_20240108_PM_100718.csv
과제4_전처리완료_20240116_PM_042303.csv
과제4_전처리완료_20240118_AM_091837.csv
과제4_전처리완료_20240118_AM_103015.csv
과제4_전처리완료_20240118_AM_111805.csv
과제4_전처리완료_20240118_PM_025809.csv
과제4_전처리완료_20240119_AM_095418.csv
과제4_전처리완료_전문정비통합_20240109_AM_025533.csv
과제4_전처리완료_전문정비통합_20240116_PM_042303.csv
과제4_전처리완료_전문정비통합_20240118_AM_091837.csv
과제4_전처리완료_전문정비통합_20240118_AM_103015.csv
과제4_전처리완료_전문정비통합_20240118_AM_111805.csv
과제4_전처리완료_전문정비통합_20240118_PM_025809.csv
과제4_전처리완료_전문정비통합_20240119_AM_095418.csv
전국_전문정비사업자_현황_20231213.xlsx
전국_전문정비사업자_현황_코드_20231214.xlsx
전문정비_20231208.xlsx
전문정비내역수정_20231208.csv
전문정비내역수정_20231208.xl

In [558]:
tb_list = [
    'tb_rsd_ge_list_202311071035.csv',
    'tb_rsd_ge_list_ex_202311071035.csv',
    'tb_rsd_ge_sub_202311071035.csv',
    'tb_rse_det_ex_202312061021.csv',
    'tb_rse_out_result_202311071036.csv',
    'tb_rse_reg_202311071036.csv',
    'tb_rse_std_202311071036.csv',
    'tb_rsm_org_202311071037.csv',
    'tb_rso_reg_202311071037.csv',
    'tb_us_org_202311071037.csv', 
]
len(tb_list)

10

In [559]:
# # 12.8s
# for file_name in tb_list:
#     name = file_name.split('.')[0]
#     df = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
#     print(f'{name} : {df.shape}')

#### [출력] 컬럼, 데이터 정보들

In [560]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2024.01.19'

In [561]:
# # 12.8s
# with pd.ExcelWriter(os.path.join(an4_fold, f'건설기계_추가데이터_현황({today_date}).xlsx')) as writer:
#     for file_name in os.listdir(raw4_fold):
#         name = file_name.split('.')[0]
#         df = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
#         temp = df.isnull().sum().reset_index()
#         temp = temp.rename(columns={'index':'컬럼명', 0:'결측치 개수'})
#         temp['총 행수'] = df.shape[0]
#         temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
#         temp = temp[['번호', '컬럼명', '결측치 개수', '총 행수']]
#         temp.to_excel(writer, sheet_name=name, index=False)

### 3\. TB_RSD_GE_LIST(과다배출선별(최종)(연속과다+차대번호중복제거))

In [562]:
# 0.3s
name = 'tb_rsd_ge_list_202311071035'
file_name = f'{name}.csv'
rsd_ge_list = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71065 entries, 0 to 71064
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ge_idx             71065 non-null  int64  
 1   rsd_month          71065 non-null  int64  
 2   car_ser_no         71065 non-null  object 
 3   snt                61457 non-null  float64
 4   ge_cnt             71065 non-null  int64  
 5   search1            11799 non-null  float64
 6   search2            2351 non-null   float64
 7   search3            52534 non-null  float64
 8   search4            61061 non-null  float64
 9   car_reg_no         71065 non-null  object 
 10  car_nm             71065 non-null  object 
 11  usage              71065 non-null  object 
 12  car_year           71065 non-null  int64  
 13  car_type_nm        71065 non-null  object 
 14  car_weight         71065 non-null  int64  
 15  tot_weight         71065 non-null  int64  
 16  engine_capacity    710

In [563]:
rsd_ge_list_dict = {
    'ge_idx':'과다선별_일련번호',
    'rsd_month':'측정년월',
    'car_ser_no':'차대번호',
    'snt':'총 측정횟수',
    'ge_cnt':'총 과다횟수',
    'search1':'연속과다_구분',
    'search2':'2회이상과다_구분',
    'search3':'연속과다대기_구분(최종_측정과다_1회)',
    'search4':'1회측정과다_구분',
    'car_reg_no':'차량번호',
    'car_nm':'차명',
    'usage':'용도',
    'car_year':'연식',
    'car_type_nm':'차종+차종유형',
    'car_weight':'중량',
    'tot_weight':'총중량',
    'engine_capacity':'엔진배기량',
    'engine_power':'엔진출력',
    'car_fuel_nm':'연료명',
    'first_reg_date':'최초등록일',
    'smo_chk_exp_date':'정밀검사만료일자',
    'chk_ge_fr':'유효기간_시작일',
    'chk_ge_to':'유효기간_종료일',
    'rso_reg_status':'개선명령_등록여부',
    'make_reg_date':'제작일자',
    'car_fuel_cd':'연료코드',
    'car_type_cd':'차종코드',
    'reg_region':'소재지',
    'sido':'시도',
    'sigungu':'시군구',
    'usage_cd':'용도코드',
    'usage_ex':'용도기준',
    'ge_reg_type':'결과등록_타입(1-원격,2-노상)',
    'last_chk_exp_date':'최종_정밀검사일자',
    'car_type_nm2':'차종명(노상-대기법)',
    'car_maker_nm':'제작사명',
    'engine_type':'엔진형식',
    'mileage':'주행거리',
    'road_cont':'노상-위반사유_코드',
    'road_type':'노상-처분종류',
    'road_money':'노상-과태료',
    'road_reason':'노상-위반사유-상세',
    'conf_yn':'확정여부',
    'conf_flag':'측정플래그',
    'selection_date':'선별일자',
    'selection_user':'선별자',
    'confirm_date':'확정일자',
    'confirm_user':'확정자',
    'imps_reason':'명령불가사유',
    'sido_cd':'법정동_시도코드',
    'sigungu_cd':'법정동_시군구코드',
    'sido_nm':'법정동_시도명',
    'sigungu_nm':'법정동_시군구명',
    'cncl_yn':'CNCL_구분',
    'note':'메모',
}
rsd_ge_listr = rsd_ge_list.rename(columns=rsd_ge_list_dict)
rsd_ge_listr.columns

Index(['과다선별_일련번호', '측정년월', '차대번호', '총 측정횟수', '총 과다횟수', '연속과다_구분', '2회이상과다_구분',
       '연속과다대기_구분(최종_측정과다_1회)', '1회측정과다_구분', '차량번호', '차명', '용도', '연식',
       '차종+차종유형', '중량', '총중량', '엔진배기량', '엔진출력', '연료명', '최초등록일', '정밀검사만료일자',
       '유효기간_시작일', '유효기간_종료일', '개선명령_등록여부', '제작일자', '연료코드', '차종코드', '소재지',
       '시도', '시군구', '용도코드', '용도기준', '결과등록_타입(1-원격,2-노상)', '최종_정밀검사일자',
       '차종명(노상-대기법)', '제작사명', '엔진형식', '주행거리', '노상-위반사유_코드', '노상-처분종류',
       '노상-과태료', '노상-위반사유-상세', '확정여부', '측정플래그', '선별일자', '선별자', '확정일자', '확정자',
       '명령불가사유', '법정동_시도코드', '법정동_시군구코드', '법정동_시도명', '법정동_시군구명', 'CNCL_구분',
       '메모'],
      dtype='object')

In [564]:
rsd_ge_listr['주행거리'].describe()

count               9608.00
mean        115644599111.14
std       11335508143510.87
min                    0.00
25%               130623.50
50%               196997.50
75%               278993.25
max     1111111111111111.00
Name: 주행거리, dtype: float64

In [565]:
fuel_cd_df = rsd_ge_listr.groupby(['연료명', '연료코드'], dropna=False)['과다선별_일련번호'].count().reset_index()
fuel_cd_df = fuel_cd_df.sort_values(['연료명', '연료코드', '과다선별_일련번호'])
fuel_cd_df

,연료명,연료코드,과다선별_일련번호
0,CNG,J,1342
1,가스,C,177
2,가스,J,6
3,경유,B,8986
4,엘피지,C,20349
5,하이브리드(LPG+전기),N,100
6,하이브리드(휘발유+전기),L,382
7,휘발유,A,13487
8,휘발유,G,1
9,휘발유,H,164


In [566]:
# CNG : J
# 가스 : C, J
# 엘피지 : C
# 경유 : B
# 하이브리드(LPG+전기) : N
# 하이브리드(휘발유+전기) : L
# 휘발유 : A, H, G
# 휘발유(무연) : H

In [567]:
# CNG : J
# LPG : C
# 경유 : B
# 하이브리드(LPG+전기) : N
# 하이브리드(휘발유+전기) : L
# 휘발유 : A, H, G

### 4\. [✔] TB_RSD_GE_LIST_EX(정밀검사2회부적합 제원+결과)

In [568]:
# 0.3s
name = 'tb_rsd_ge_list_ex_202311071035'
file_name = f'{name}.csv'
rsd_ge_list_ex = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_list_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119898 entries, 0 to 119897
Data columns (total 70 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ge_idx                 119898 non-null  int64  
 1   rsd_date               119898 non-null  int64  
 2   car_reg_no             119898 non-null  object 
 3   car_ser_no             119866 non-null  object 
 4   car_nm                 119896 non-null  object 
 5   car_type_cd            62630 non-null   float64
 6   car_year               119896 non-null  object 
 7   usage_cd               62630 non-null   float64
 8   car_fuel_cd            119894 non-null  object 
 9   car_weight             119501 non-null  float64
 10  tot_weight             119495 non-null  float64
 11  engine_capacity        119501 non-null  float64
 12  engine_power           119502 non-null  object 
 13  engine_type            119494 non-null  object 
 14  mileage                119898 non-nu

In [569]:
rsd_ge_list_ex_dict = {
    'ge_idx':'일련번호', 
    'rsd_date':'점검일자', 
    'car_reg_no':'차량번호', 
    'car_ser_no':'차대번호', 
    'car_nm':'차명', 
    'car_type_cd':'차종', 
    'car_year':'연식', 
    'usage_cd':'용도', 
    'car_fuel_cd':'연료', 
    'car_weight':'중량', 
    'tot_weight':'총중량', 
    'engine_capacity':'엔진배기량', 
    'engine_power':'엔진출력', 
    'engine_type':'엔진형식', 
    'mileage':'주행거리', 
    'make_reg_date':'제작일자', 
    'first_reg_date':'최초등록일', 
    'smo_chk_exp_date':'정밀검사만료일자', 
    'sido':'시도', 
    'sigungu':'시군구', 
    'org_id':'전문정비사업자코드', 
    'cer_exp_year':'보증기간_년', 
    'cer_exp_km':'보증기간_Km', 
    'mistake_yn':'보증기간내_고의_또는_과실여부',  
    'repair_name':'정비사성명', 
    'repair_comment':'정비사의견', 
    'car_age':'차령', 
    'det_yn':'정비여부', 
    'rec_date':'생성일자', 
    'rec_nm':'생성자명', 
    'las_chg_nm':'최종수정자명', 
    'las_chg_date':'최종수정일자', 
    'cer_repair_yn':'보증수리여부', 
    'reg_region':'소재지', 
    'doc_name':'소유자명', 
    'reg_no':'접수번호', 
    'print_yn':'출력여부', 
    'print_date':'출력일자', 
    'print_rec_nm':'출력자명', 
    'sido_cd':'시도코드', 
    'sigungu_cd':'시군구코드', 
    'sido_nm':'시도명', 
    'sigungu_nm':'시군구명', 
    'smo_chk_yn':'정밀검사유무', 
    'smo_chk_date':'정밀검사일자', 
    'smo_chk_org_nm':'정밀검사소명', 
    'smo_chk_method':'정밀검사측정방법', 
    'smo_chk_judge_yn':'정밀검사판정', 
    'unload_smo_val1':'무부하매연측정치1', 
    'unload_smo_lim1':'무부하매연허용치1', 
    'unload_smo_judge1':'무부하매연판정1', 
    'unload_smo_val2':'무부하매연측정치2', 
    'unload_smo_lim2':'무부하매연허용치2', 
    'unload_smo_judge2':'무부하매연판정2', 
    'unload_smo_val3':'무부하매연측정치3', 
    'unload_smo_lim3':'무부하매연허용치3', 
    'unload_smo_judge3':'무부하매연판정3', 
    'unload_smo_val4':'무부하매연측정치4', 
    'unload_smo_lim4':'무부하매연허용치4', 
    'unload_smo_judge4':'무부하매연판정4', 
    'unload_smo_val5':'무부하매연측정치5', 
    'unload_smo_lim5':'무부하매연허용치5', 
    'unload_smo_judge5':'무부하매연판정5', 
    'unload_smo_val6':'무부하매연측정치6', 
    'unload_smo_lim6':'무부하매연허용치6', 
    'unload_smo_judge6':'무부하매연판정6', 
    'smo_fun_chk_judge_yn':'관능검사판정', 
    'smo_fun_chk_fail_item':'관능검사부적합내용', 
    'smo_update_date':'정밀검사수정일', 
    'conf_yn':'확정여부', 
}
rsd_ge_list_exr = rsd_ge_list_ex.rename(columns=rsd_ge_list_ex_dict)
rsd_ge_list_exr.columns

Index(['일련번호', '점검일자', '차량번호', '차대번호', '차명', '차종', '연식', '용도', '연료', '중량',
       '총중량', '엔진배기량', '엔진출력', '엔진형식', '주행거리', '제작일자', '최초등록일', '정밀검사만료일자',
       '시도', '시군구', '전문정비사업자코드', '보증기간_년', '보증기간_Km', '보증기간내_고의_또는_과실여부',
       '정비사성명', '정비사의견', '차령', '정비여부', '생성일자', '생성자명', '최종수정자명', '최종수정일자',
       '보증수리여부', '소재지', '소유자명', '접수번호', '출력여부', '출력일자', '출력자명', '시도코드',
       '시군구코드', '시도명', '시군구명', '정밀검사유무', '정밀검사일자', '정밀검사소명', '정밀검사측정방법',
       '정밀검사판정', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2',
       '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3',
       '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5',
       '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '관능검사판정', '관능검사부적합내용',
       '정밀검사수정일', '확정여부'],
      dtype='object')

In [570]:
rsd_ge_list_exr.shape, len(rsd_ge_list_exr['일련번호'].unique())

((119898, 70), 119898)

In [571]:
rsd_ge_list_exr['일련번호'].describe()

count   119898.00
mean     60550.66
std      34776.82
min          1.00
25%      30516.25
50%      60490.50
75%      90622.75
max     120932.00
Name: 일련번호, dtype: float64

In [572]:
rsd_ge_list_exr[['일련번호', '점검일자', '차량번호', '차대번호', '차명', '차종', '연식', '용도', '연료', '중량', '총중량', ]].head()

,일련번호,점검일자,차량번호,차대번호,차명,차종,연식,용도,연료,중량,총중량
0,3,20130805,85거9409,KPALE4A811P134760,코란도-밴,331.00,2001,2.00,B,1705.00,2335.00
1,11,20130806,35머4273,KMHMH81VP1U097748,트라제XG,152.00,2001,2.00,B,1845.00,2300.00
2,12,20130806,96도3224,KMFFE17APSU238435,포터초장축더블캡,311.00,1995,2.00,B,1495.00,2825.00
3,15,20130807,85오2664,KMFMA17HPNU008954,현대4.5톤카고트럭-초장축,312.00,1992,2.00,B,4640.00,9305.00
4,16,20130807,33서4306,KMHNN81WP1U012631,테라칸,113.00,2001,2.00,B,2430.00,2430.00


In [573]:
rsd_ge_list_exr[['엔진배기량', '엔진출력', '엔진형식', '주행거리', '제작일자', '최초등록일', '시도코드', '시군구코드', '시도명', '시군구명', '전문정비사업자코드', '정비여부',]].head()

,엔진배기량,엔진출력,엔진형식,주행거리,제작일자,최초등록일,시도코드,시군구코드,시도명,시군구명,전문정비사업자코드,정비여부
0,2874.00,95/4000,662,105598.00,20010827.00,20010830.00,27,290,대구광역시,달서구,ZC05,Y
1,1991.00,115/4000,D4EA,199014.00,20010725.00,20010730.00,44,130,충청남도,천안시,ZK01,Y
2,2476.00,80/4200,D4EA,299215.00,20030711.00,19950803.00,27,290,대구광역시,달서구,ZC02,Y
3,7545.00,0167/2900,D6BH,24587.00,19920918.00,19920918.00,27,140,대구광역시,동구,ZC01,Y
4,2476.00,103/3800,D4BH,197038.00,20010705.00,20010705.00,27,230,대구광역시,북구,ZC06,Y


In [574]:
rsd_ge_list_exr[['시도코드', '시군구코드', '시도명', '시군구명', '전문정비사업자코드', '정비여부', '정밀검사소명']].head()

,시도코드,시군구코드,시도명,시군구명,전문정비사업자코드,정비여부,정밀검사소명
0,27,290,대구광역시,달서구,ZC05,Y,스피드1급검사정비
1,44,130,충청남도,천안시,ZK01,Y,천안자동차검사소
2,27,290,대구광역시,달서구,ZC02,Y,성서자동차검사정비
3,27,140,대구광역시,동구,ZC01,Y,반야월자동차정비공장
4,27,230,대구광역시,북구,ZC06,Y,이현자동차검사소


In [575]:
rsd_ge_list_exr[['점검일자', '제작일자', '최초등록일', '생성일자', '최종수정일자']].isnull().sum()

점검일자           0
제작일자       55596
최초등록일        396
생성일자           0
최종수정일자    109623
dtype: int64

#### 연료 코드 해석

In [576]:
rsd_ge_list_exr['연료'].unique()

array(['B', 'C', 'H', 'A', 'G', 'L', nan, 'J', 'N', 'Z'], dtype=object)

In [577]:
fuel_cd_df

,연료명,연료코드,과다선별_일련번호
0,CNG,J,1342
1,가스,C,177
2,가스,J,6
3,경유,B,8986
4,엘피지,C,20349
5,하이브리드(LPG+전기),N,100
6,하이브리드(휘발유+전기),L,382
7,휘발유,A,13487
8,휘발유,G,1
9,휘발유,H,164


In [578]:
# CNG : J
# LPG : C
# 경유 : B
# 하이브리드(LPG+전기) : N
# 하이브리드(휘발유+전기) : L
# 휘발유 : A, H, G

In [579]:
fuel_dict = {
    'A':'휘발유',
    'B':'경유',
    'C':'LPG',
    'D':'등유',
    'E':'전기',
    'F':'알코올',
    'G':'휘발유', # 유연
    'H':'휘발유', # 무연
    'I':'태양열', 
    'J':'CNG',
    'K':'LNG',
    'L':'하이브리드(휘발유+전기)',
    'M':'하이브리드(경유+전기)',
    'N':'하이브리드(LPG+전기)',
    'O':'하이브리드(CNG+전기)',
    'P':'하이브리드(LNG+전기)',
    'Z':'기타연료',  
}
rsd_ge_list_exr['연료'] = rsd_ge_list_exr['연료'].replace(fuel_dict)
rsd_ge_list_exr['연료'].unique()

array(['경유', 'LPG', '휘발유', '하이브리드(휘발유+전기)', nan, 'CNG', '하이브리드(LPG+전기)',
       '기타연료'], dtype=object)

#### 차종 코드 해석

In [580]:
rsd_ge_list_exr['차종'].unique()

array([331., 152., 311., 312., 113., 132., 342., 112., 133., 212., 121.,
       323., 153., 111., 322., 114., 332., 321., 341., 441., 313., 422.,
       122., 213., 211., 131., 442., 343., 242., 334., 421., 214., 123.,
       241., 243., 443., 433., 314., 151., 324., 363., 344., 362.,  nan,
       124., 134., 154., 431., 444.])

In [581]:
car_type_cd_dict = {
    111:'승용', # 승용차일반형소형
    112:'승용', # 승용차일반형중형
    113:'승용', # 승용차일반형대형
    114:'승용', # 승용차일반형경형
    121:'승용', # 승용차겸화물형소형
    122:'승용', # 승용차겸화물형중형
    123:'승용', # 승용차겸화물형대형
    124:'승용', # 승용차겸화물형경형
    131:'승용', # 승용차다목적형소형
    132:'승용', # 승용차다목적형중형
    133:'승용', # 승용차다목적형대형
    134:'승용', # 승용차다목적형경형
    151:'승용', # 승용차기타형소형
    152:'승용', # 승용차기타형중형
    153:'승용', # 승용차기타형대형
    154:'승용', # 승용차기타형경형
    211:'승합', # 승합차일반형소형
    212:'승합', # 승합차일반형중형
    213:'승합', # 승합차일반형대형
    214:'승합', # 승합차일반형경형
    241:'승합', # 승합차특수형소형
    242:'승합', # 승합차특수형중형
    243:'승합', # 승합차특수형대형
    244:'승합', # 승합차특수형경형
    311:'화물', # 화물차일반형소형
    312:'화물', # 화물차일반형중형
    313:'화물', # 화물차일반형대형
    314:'화물', # 화물차일반형경형
    321:'화물', # 화물차덤프형소형
    322:'화물', # 화물차덤프형중형
    323:'화물', # 화물차덤프형대형
    324:'화물', # 화물차덤프형경형
    331:'화물', # 화물차밴형소형
    332:'화물', # 화물차밴형중형
    333:'화물', # 화물차밴형대형
    334:'화물', # 화물차밴형경형
    341:'화물', # 화물차특수용도형소형
    342:'화물', # 화물차특수용도형중형
    343:'화물', # 화물차특수용도형대형
    344:'화물', # 화물차특수용도형경형
    351:'화물', # 화물차피견인형저상형
    352:'화물', # 화물차피견인형고상형
    353:'화물', # 화물차피견인형적재함형
    354:'화물', # 화물차피견인형샷시형
    355:'화물', # 화물차피견인형기타형
    361:'화물', # 화물차탱크로리형소형
    362:'화물', # 화물차탱크로리형중형
    363:'화물', # 화물차탱크로리형대형
    364:'화물', # 화물차탱크로리형경형
    421:'특수', # 특수차구난형소형
    422:'특수', # 특수차구난형중형
    423:'특수', # 특수차구난형대형
    424:'특수', # 특수차구난형경형
    431:'특수', # 특수차견인형소형
    432:'특수', # 특수차견인형중형
    433:'특수', # 특수차견인형대형
    434:'특수', # 특수차견인형경형
    441:'특수', # 특수차특수작업형소형
    442:'특수', # 특수차특수작업형중형
    443:'특수', # 특수차특수작업형대형
    444:'특수', # 특수차특수작업형경형
    511:'이륜', # 2륜차일반형소형
    512:'이륜', # 2륜차일반형중형
    513:'이륜', # 2륜차일반형대형
    541:'이륜', # 2륜차특수형소형
    542:'이륜', # 2륜차특수형중형
    543:'이륜', # 2륜차특수형대형
}
car_type_cd_dict2 = {
    111:'소형',
    112:'중형',
    113:'대형',
    114:'경형',
    121:'소형',
    122:'중형',
    123:'대형',
    124:'경형',
    131:'소형',
    132:'중형',
    133:'대형',
    134:'경형',
    151:'소형',
    152:'중형',
    153:'대형',
    154:'경형',
    211:'소형',
    212:'중형',
    213:'대형',
    214:'경형',
    241:'소형',
    242:'중형',
    243:'대형',
    244:'경형',
    311:'소형',
    312:'중형',
    313:'대형',
    314:'경형',
    321:'소형',
    322:'중형',
    323:'대형',
    324:'경형',
    331:'소형',
    332:'중형',
    333:'대형',
    334:'경형',
    341:'소형',
    342:'중형',
    343:'대형',
    344:'경형',
    351:'상형',
    352:'상형',
    353:'함형',
    354:'시형',
    355:'타형',
    361:'소형',
    362:'중형',
    363:'대형',
    364:'경형',
    421:'소형',
    422:'중형',
    423:'대형',
    424:'경형',
    431:'소형',
    432:'중형',
    433:'대형',
    434:'경형',
    441:'소형',
    442:'중형',
    443:'대형',
    444:'경형',
    511:'소형',
    512:'중형',
    513:'대형',
    541:'소형',
    542:'중형',
    543:'대형',
}
rsd_ge_list_exr['차종유형'] = rsd_ge_list_exr['차종'].replace(car_type_cd_dict2)
rsd_ge_list_exr['차종'] = rsd_ge_list_exr['차종'].replace(car_type_cd_dict)
print(rsd_ge_list_exr['차종'].unique())
print(rsd_ge_list_exr['차종유형'].unique())

['화물' '승용' '승합' '특수' nan]
['소형' '중형' '대형' '경형' nan]


#### 용도 코드 해석

In [582]:
rsd_ge_list_exr['용도'].unique()

array([ 2.,  3.,  1.,  4., nan])

In [583]:
usage_cd_dict = {
    1:'관용',
    2:'자가용',
    3:'영업용',
    4:'개인택시',
}
rsd_ge_list_exr['용도'] = rsd_ge_list_exr['용도'].replace(usage_cd_dict)
rsd_ge_list_exr['용도'].unique()

array(['자가용', '영업용', '관용', '개인택시', nan], dtype=object)

### 5\. TB_RSD_GE_SUB(운행차 수시점검 내역(과다배출선별))

In [584]:
# 0.3s
name = 'tb_rsd_ge_sub_202311071035'
file_name = f'{name}.csv'
rsd_ge_sub = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469327 entries, 0 to 469326
Data columns (total 69 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sub_idx            469327 non-null  int64  
 1   fst                459692 non-null  float64
 2   chk_sts            469272 non-null  object 
 3   ge_chk             26834 non-null   object 
 4   rsd_month          469327 non-null  int64  
 5   rsd_date           469327 non-null  int64  
 6   rsd_time           469325 non-null  object 
 7   sitecode           469326 non-null  object 
 8   car_nm             469327 non-null  object 
 9   usage              469327 non-null  object 
 10  car_year           469327 non-null  int64  
 11  car_type_nm        469327 non-null  object 
 12  car_weight         469327 non-null  int64  
 13  tot_weight         469327 non-null  int64  
 14  engine_capacity    469326 non-null  float64
 15  engine_power       469327 non-null  object 
 16  sp

In [585]:
rsd_ge_sub_dict = {
    'sub_idx':'측정결과_일련번호',
    'fst':'측정순번',
    'chk_sts':'측정기간_유효여부',
    'ge_chk':'과다배출_여부',
    'rsd_month':'측정년월',
    'rsd_date':'측정일자',
    'rsd_time':'측정시간',
    'sitecode':'측정장소',
    'car_nm':'차명',
    'usage':'용도',
    'car_year':'연식',
    'car_type_nm':'차종명',
    'car_weight':'중량',
    'tot_weight':'총중량',
    'engine_capacity':'엔진용량',
    'engine_power':'엔진출력',
    'speed':'속도',
    'accel':'가속도',
    'vsp':'VSP',
    'uvsmoke':'매연UV_측정값',
    'irsmoke':'매연IR_측정값',
    'co':'CO_측정값',
    'ppmhc':'HC_측정값',
    'ppmno':'NOX_측정값',
    'co_ex':'CO_기준값',
    'ppmhc_ex':'HC_기준값',
    'ppmno_ex':'NO_기준값',
    'car_ser_no':'차대번호',
    'car_reg_no':'차량번호',
    'usage_ex':'용도_기준값',
    'fuel_ex':'연료_기준값',
    'car_type_ex':'차종_기준값',
    'first_reg_date':'최초등록일',
    'smo_chk_exp_date':'정밀검사만료일',
    'smo_chk_mon':'정밀검사만료일_날짜형식',
    'chk_ge_fr':'측정_유효기간_시작일',
    'chk_ge_to':'측정_유효기간_종료일',
    'f__':'계산일자_1',
    'd__':'계산일자_2',
    'e__':'계산일자_3',
    'smo_chk_method':'정밀검사_검사방법',
    'unload_smo_val1':'정밀검사_측정값_1',
    'unload_smo_val2':'정밀검사_측정값_2',
    'unload_smo_val3':'정밀검사_측정값_3',
    'unload_smo_val4':'정밀검사_측정값_4',
    'unload_smo_val5':'정밀검사_측정값_5',
    'unload_smo_val6':'정밀검사_측정값_6',
    'unload_smo_lim1':'정밀검사_기준값_1',
    'unload_smo_lim2':'정밀검사_기준값_2',
    'unload_smo_lim3':'정밀검사_기준값_3',
    'unload_smo_lim4':'정밀검사_기준값_4',
    'unload_smo_lim5':'정밀검사_기준값_5',
    'unload_smo_lim6':'정밀검사_기준값_6',
    'make_reg_date':'제작일자',
    'file_nm':'제원정보파일명',
    'rsd_file_nm':'원격측정업로드파일명',
    'smo_chk_cd':'정밀검사방법코드',
    'std_tag':'허용기준(부하)',
    'std_tag2':'허용기준(무부하)',
    'imagename':'원격측정이미지파일명',
    'ge_reg_type':'결과등록 타입(1-원격,2-노상)',
    'last_chk_exp_date':'최종_정밀검사일자',
    'air':'공기과잉률측정치',
    'air_ex_from':'공기과잉률_허용치_FROM',
    'air_ex_to':'공기과잉률_허용치_TO',
    'ppmhchex':'PPMHCHEX',
    'samples':'샘플수',
    'rec_date':'등록일',
    'air_type':'공기과잉률_종류',

}
rsd_ge_subr = rsd_ge_sub.rename(columns=rsd_ge_sub_dict)
rsd_ge_subr.columns

Index(['측정결과_일련번호', '측정순번', '측정기간_유효여부', '과다배출_여부', '측정년월', '측정일자', '측정시간',
       '측정장소', '차명', '용도', '연식', '차종명', '중량', '총중량', '엔진용량', '엔진출력', '속도',
       '가속도', 'VSP', '매연UV_측정값', '매연IR_측정값', 'CO_측정값', 'HC_측정값', 'NOX_측정값',
       'CO_기준값', 'HC_기준값', 'NO_기준값', '차대번호', '차량번호', '용도_기준값', '연료_기준값',
       '차종_기준값', '최초등록일', '정밀검사만료일', '정밀검사만료일_날짜형식', '측정_유효기간_시작일',
       '측정_유효기간_종료일', '계산일자_1', '계산일자_2', '계산일자_3', '정밀검사_검사방법', '정밀검사_측정값_1',
       '정밀검사_측정값_2', '정밀검사_측정값_3', '정밀검사_측정값_4', '정밀검사_측정값_5', '정밀검사_측정값_6',
       '정밀검사_기준값_1', '정밀검사_기준값_2', '정밀검사_기준값_3', '정밀검사_기준값_4', '정밀검사_기준값_5',
       '정밀검사_기준값_6', '제작일자', '제원정보파일명', '원격측정업로드파일명', '정밀검사방법코드', '허용기준(부하)',
       '허용기준(무부하)', '원격측정이미지파일명', '결과등록 타입(1-원격,2-노상)', '최종_정밀검사일자',
       '공기과잉률측정치', '공기과잉률_허용치_FROM', '공기과잉률_허용치_TO', 'PPMHCHEX', '샘플수', '등록일',
       '공기과잉률_종류'],
      dtype='object')

In [586]:
set(rsd_ge_subr['공기과잉률_종류'].unique())

{0.0, nan, 1.0, 2.0}

In [587]:
rsd_ge_subr['측정순번'].describe()

count   459692.00
mean         1.51
std          1.63
min          1.00
25%          1.00
50%          1.00
75%          2.00
max         70.00
Name: 측정순번, dtype: float64

In [588]:
rsd_ge_subr['측정일자'].describe()

count     469327.00
mean    20133274.76
std        10817.22
min     20000104.00
25%     20130726.00
50%     20130930.00
75%     20131113.00
max     21090428.00
Name: 측정일자, dtype: float64

In [589]:
rsd_ge_subr['허용기준(부하)'].head()

0   22.00
1   28.00
2   22.00
3    6.00
4   29.00
Name: 허용기준(부하), dtype: float64

### 6\. TB_RSE_OUT_RESULT(확인검사OUT결과)

In [590]:
# 0.3s
name = 'tb_rse_out_result_202311071036'
file_name = f'{name}.csv'
rse_out_result = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rse_out_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3640 entries, 0 to 3639
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rse_idx                   3640 non-null   int64  
 1   rse_date                  3640 non-null   int64  
 2   rse_org_cd                3623 non-null   object 
 3   car_reg_no                3640 non-null   object 
 4   car_ser_no                3640 non-null   object 
 5   cer_exp_year              3263 non-null   object 
 6   cer_exp_km                3263 non-null   float64
 7   cer_repair_yn             3632 non-null   object 
 8   mileage                   3530 non-null   object 
 9   mistake_yn                3632 non-null   object 
 10  repair_name               3262 non-null   object 
 11  repair_comment            3245 non-null   object 
 12  before_smo_val            1788 non-null   float64
 13  before_smo_lim            1789 non-null   float64
 14  before_c

In [591]:
rse_out_result_dict = {
    'rse_idx':'일련번호(TB_RSE_REG.RSE_IDX)',
    'rse_date':'배출가스검사일자(확인검사일자)',
    'rse_org_cd':'검사소코드(TB_US_ORG.ORG_ID)',
    'car_reg_no':'차량번호',
    'car_ser_no':'차대번호',
    'cer_exp_year':'보증기간_년',
    'cer_exp_km':'보증기간_Km',
    'cer_repair_yn':'보증수리여부',
    'mileage':'주행거리',
    'mistake_yn':'보증기간내_고의_또는_과실여부',
    'repair_name':'정비사성명',
    'repair_comment':'정비사의견',
    'before_smo_val':'개선전_매연',
    'before_smo_lim':'개선전_매연허용치',
    'before_co_val':'개선전_일산화탄소(CO)',
    'before_co_lim':'개선전_일산화탄소(CO)허용치',
    'before_hc_val':'개선전_탄화수소(HC)',
    'before_hc_lim':'개선전_탄화수소(HC)허용치',
    'before_nox_val':'개선전_질소산화물(NOX)',
    'before_nox_lim':'개선전_질소산화물(NOX)허용치',
    'before_air_over_val':'개선전_공기과잉률',
    'before_air_over_lim':'개선전_공기과잉률_허용치',
    'after_smo_val1':'개선후_매연측정치',
    'after_smo_lim1':'개선후_매연허용치',
    'after_co_val1':'개선후_일산화탄소(CO)측정치',
    'after_co_lim1':'개선후_일산화탄소(CO)허용치',
    'after_hc_val1':'개선후_탄화수소(HC)측정치',
    'after_hc_lim1':'개선후_탄화수소(HC)허용치',
    'after_nox_val1':'개선후_질소산화물(NOX)측정치',
    'after_nox_lim1':'개선후_질소산화물(NOX)허용치',
    'after_air_over_val1':'개선후_공기과잉률측정치',
    'after_air_over_lim1_from':'개선후_공기과잉률허용치_FROM',
    'after_air_over_lim1_to':'개선후_공기과잉률허용치_TO',
    'after_smo_val2':'개선후_매연측정치',
    'after_smo_lim2':'개선후_매연허용치',
    'after_co_val2':'개선후_일산화탄소(CO)측정치',
    'after_co_lim2':'개선후_일산화탄소(CO)허용치',
    'after_hc_val2':'개선후_탄화수소(HC)측정치',
    'after_hc_lim2':'개선후_탄화수소(HC)허용치',
    'after_nox_val2':'개선후_질소산화물(NOX)측정치',
    'after_nox_lim2':'개선후_질소산화물(NOX)허용치',
    'after_air_over_val2':'개선후_공기과잉률측정치',
    'after_air_over_lim2_from':'개선후_공기과잉률측정치_FROM',
    'after_air_over_lim2_to':'개선후_공기과잉률측정치_TO',
    'rec_date':'생성일자',
    'rec_nm':'생성자명',
    'las_chg_nm':'최종수정자명',
    'las_chg_date':'최종수정일자',
    'rse_out_final_yn':'확인검사결과',
    'car_age':'차령',
    'before_air_over_lim2':'개선전공기과잉률_TO',
    'filename1':'확인검사이미지(앞면)',
    'filename2':'확인검사이미지(뒷면)',
}
rse_out_resultr = rse_out_result.rename(columns=rse_out_result_dict)
rse_out_resultr.columns

Index(['일련번호(TB_RSE_REG.RSE_IDX)', '배출가스검사일자(확인검사일자)',
       '검사소코드(TB_US_ORG.ORG_ID)', '차량번호', '차대번호', '보증기간_년', '보증기간_Km',
       '보증수리여부', '주행거리', '보증기간내_고의_또는_과실여부', '정비사성명', '정비사의견', '개선전_매연',
       '개선전_매연허용치', '개선전_일산화탄소(CO)', '개선전_일산화탄소(CO)허용치', '개선전_탄화수소(HC)',
       '개선전_탄화수소(HC)허용치', '개선전_질소산화물(NOX)', '개선전_질소산화물(NOX)허용치', '개선전_공기과잉률',
       '개선전_공기과잉률_허용치', '개선후_매연측정치', '개선후_매연허용치', '개선후_일산화탄소(CO)측정치',
       '개선후_일산화탄소(CO)허용치', '개선후_탄화수소(HC)측정치', '개선후_탄화수소(HC)허용치',
       '개선후_질소산화물(NOX)측정치', '개선후_질소산화물(NOX)허용치', '개선후_공기과잉률측정치',
       '개선후_공기과잉률허용치_FROM', '개선후_공기과잉률허용치_TO', '개선후_매연측정치', '개선후_매연허용치',
       '개선후_일산화탄소(CO)측정치', '개선후_일산화탄소(CO)허용치', '개선후_탄화수소(HC)측정치',
       '개선후_탄화수소(HC)허용치', '개선후_질소산화물(NOX)측정치', '개선후_질소산화물(NOX)허용치',
       '개선후_공기과잉률측정치', '개선후_공기과잉률측정치_FROM', '개선후_공기과잉률측정치_TO', '생성일자', '생성자명',
       '최종수정자명', '최종수정일자', '확인검사결과', '차령', '개선전공기과잉률_TO', '확인검사이미지(앞면)',
       '확인검사이미지(뒷면)'],
      dtype='object')

### 7\. [❌] TB_RSE_REG(확인검사접수)

In [592]:
# 0.3s
name = 'tb_rse_reg_202311071036'
file_name = f'{name}.csv'
rse_reg = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rse_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4140 entries, 0 to 4139
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rse_idx          4140 non-null   int64  
 1   rso_idx          3769 non-null   float64
 2   car_reg_no       3766 non-null   object 
 3   car_ser_no       3766 non-null   object 
 4   org_id           4140 non-null   object 
 5   org_nm           4133 non-null   object 
 6   reg_no           4140 non-null   int64  
 7   reg_no_full      4140 non-null   object 
 8   rse_date         4140 non-null   int64  
 9   rse_org_cd       4109 non-null   object 
 10  rse_org_nm       4130 non-null   object 
 11  rse_reg_no       4140 non-null   int64  
 12  rse_reg_no_full  4140 non-null   object 
 13  rse_pay_gb       4140 non-null   int64  
 14  rse_how_gb       4140 non-null   object 
 15  rse_etc_gb       4126 non-null   float64
 16  rse_status       4140 non-null   int64  
 17  rec_date      

In [593]:
rse_reg_dict = {
    'rse_idx':'일련번호_MAX', 
    'rso_idx':'개선명령_IDX', 
    'car_reg_no':'차량번호', 
    'car_ser_no':'차대번호', 
    'org_id':'업체코드_접수업체', # 중요
    'org_nm':'업체명_접수업체', # 중요
    'reg_no':'순번_접수업체', 
    'reg_no_full':'접수번호_접수업체', 
    'rse_date':'접수일자', 
    'rse_org_cd':'업체코드_검사업체', # 중요
    'rse_org_nm':'업체명_검사업체', # 중요
    'rse_reg_no':'순번_검사업체', 
    'rse_reg_no_full':'접수번호_검사업체', 
    'rse_pay_gb':'수납구분', 
    'rse_how_gb':'검사방법', 
    'rse_etc_gb':'특이사항', 
    'rse_status':'확인검사상태', 
    'rec_date':'생성일자', 
    'rec_nm':'생성자명', 
    'las_chg_date':'최종수정일자', 
    'las_chg_nm':'최종수정자명', 
    'det_yn':'정비내역등록여부', 
    'print_yn':'출력여부', 
    'print_date':'출력일시', 
    'rse_amt':'수수료', 
    'scale_gb':'규모구분', 
}
rse_regr = rse_reg.rename(columns=rse_reg_dict)
rse_regr.columns

Index(['일련번호_MAX', '개선명령_IDX', '차량번호', '차대번호', '업체코드_접수업체', '업체명_접수업체',
       '순번_접수업체', '접수번호_접수업체', '접수일자', '업체코드_검사업체', '업체명_검사업체', '순번_검사업체',
       '접수번호_검사업체', '수납구분', '검사방법', '특이사항', '확인검사상태', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '정비내역등록여부', '출력여부', '출력일시', '수수료', '규모구분'],
      dtype='object')

In [594]:
rse_regr['일련번호_MAX'].describe()

count   4140.00
mean    2189.04
std     1271.14
min        1.00
25%     1083.75
50%     2190.50
75%     3296.25
max     4379.00
Name: 일련번호_MAX, dtype: float64

In [595]:
rse_regr['업체명_접수업체'].unique()[:5]

array(['(주)영등포자동차정비검사소', '비케이프라임산업주식회사', '신흥자동차정비검사(주)', '솔로몬자동차공업사',
       '인영자동차공업사(주)'], dtype=object)

In [596]:
rse_regr['업체코드_접수업체'].unique()[:5]

array(['ZA09', 'ZA08', 'ZA25', 'ZA26', 'ZD02'], dtype=object)

### 8\. TB_RSE_STD(확인검사허용기준(확정))

In [597]:
# 0.3s
name = 'tb_rse_std_202311071036'
file_name = f'{name}.csv'
rse_std = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rse_std.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3410 entries, 0 to 3409
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rse_idx            3410 non-null   int64  
 1   rse_reg_no         3410 non-null   object 
 2   rse_how_gb         3410 non-null   object 
 3   gb_co_lim          1306 non-null   float64
 4   gb_hc_lim          1306 non-null   float64
 5   gb_nox_lim         1306 non-null   float64
 6   gb_bh_hp_lim       3410 non-null   float64
 7   db_1mod_lim        3410 non-null   int64  
 8   db_2mod_lim        3410 non-null   int64  
 9   db_3mod_lim        3410 non-null   int64  
 10  db_op_lim          3363 non-null   float64
 11  db_revc_lim        3410 non-null   int64  
 12  gm_co_lim          1305 non-null   float64
 13  gm_hc_lim          1305 non-null   float64
 14  gm_air_over_lim1   3410 non-null   float64
 15  gm_air_over_lim2   3410 non-null   float64
 16  dm_smo_lim         3410 

In [598]:
rse_std_dict = {
    'rse_idx':'일련번호(TB_RSE_REG.RSE_IDX)',
    'rse_reg_no':'접수번호',
    'rse_how_gb':'검사방법',
    'gb_co_lim':'GB(CO기준치)',
    'gb_hc_lim':'GB(HC기준치)',
    'gb_nox_lim':'GB(NOX기준치)',
    'gb_bh_hp_lim':'GB(부하마력 기준치)',
    'db_1mod_lim':'DB(1모드 기준치)',
    'db_2mod_lim':'DB(2모드 기준치)',
    'db_3mod_lim':'DB(3모드 기준치)',
    'db_op_lim':'DB(출력 기준치)',
    'db_revc_lim':'DB(회전수 기준치)',
    'gm_co_lim':'GM(CO기준치)',
    'gm_hc_lim':'GM(HC기준치)',
    'gm_air_over_lim1':'GM(공기과잉률 기준치1)',
    'gm_air_over_lim2':'GM(공기과잉률 기준치2)',
    'dm_smo_lim':'DM(매연기준치)',
    'dm_revc_lim':'DM(회전수기준치)',
    'gt_co_lim':'GT(CO기준치)',
    'gt_hc_lim':'GT(HC기준치)',
    'gt_air_over_lim1':'GT(공기과잉률기준치1)',
    'gt_air_over_lim2':'GT(공기과잉률기준치2)',
    'dk_smo_lim':'DK(매연기준치)',
    'dk_car_weight_lim':'DK(차량중량기준치)',
    'dk_op_lim':'DK(출력기준치)',
    'dk_car_type_cd':'DK(차종코드:자동차법)',
    'gi_co_b_lim':'GI(CO_B기준치)',
    'gi_hc_b_lim':'GI(HC_B기준치)',
    'gi_co_m_lim':'GI(CO_M기준치)',
    'gi_hc_m_lim':'GI(HC_M기준치)',
    'gi_nox_lim':'GI(NOX기준치)',
    'gi_bh_hp_lim':'GI(부하마력기준치)',
    'gi_air_over_lim1':'GI(공기과잉률기준치1)',
    'gi_air_over_lim2':'GI(공기과잉률기준치2)',
    'rse_time':'검사시간(시:분:초) TB_RSE_OBD_RESULT 마지막수정시간',
    'rse_yn':'검사여부',
    'rse_user_id':'관능검사자ID',
    'usage_cd':'연료코드',
    'kagumgi':'과급기',
    'car_year':'년식',
    'atm_type_gb':'차종(대기법)',
    'combustion_yn':'희박연소엔진여부',
    'car_reg_no':'차량번호',
}
rse_stdr = rse_std.rename(columns=rse_std_dict)
rse_stdr.columns

Index(['일련번호(TB_RSE_REG.RSE_IDX)', '접수번호', '검사방법', 'GB(CO기준치)', 'GB(HC기준치)',
       'GB(NOX기준치)', 'GB(부하마력 기준치)', 'DB(1모드 기준치)', 'DB(2모드 기준치)',
       'DB(3모드 기준치)', 'DB(출력 기준치)', 'DB(회전수 기준치)', 'GM(CO기준치)', 'GM(HC기준치)',
       'GM(공기과잉률 기준치1)', 'GM(공기과잉률 기준치2)', 'DM(매연기준치)', 'DM(회전수기준치)',
       'GT(CO기준치)', 'GT(HC기준치)', 'GT(공기과잉률기준치1)', 'GT(공기과잉률기준치2)', 'DK(매연기준치)',
       'DK(차량중량기준치)', 'DK(출력기준치)', 'DK(차종코드:자동차법)', 'GI(CO_B기준치)',
       'GI(HC_B기준치)', 'GI(CO_M기준치)', 'GI(HC_M기준치)', 'GI(NOX기준치)',
       'GI(부하마력기준치)', 'GI(공기과잉률기준치1)', 'GI(공기과잉률기준치2)',
       '검사시간(시:분:초) TB_RSE_OBD_RESULT 마지막수정시간', '검사여부', '관능검사자ID', '연료코드',
       '과급기', '년식', '차종(대기법)', '희박연소엔진여부', '차량번호'],
      dtype='object')

In [599]:
rse_stdr['접수번호'].unique()[:5]

array(['41K12013090390001', 'H0392013102190002', 'A0482013112590002',
       'D0212014012190003', 'A0482014020690001'], dtype=object)

In [600]:
rse_stdr['연료코드'].unique()

array(['C', 'B', 'H', 'A', 'J', 'L', 'N', 'G'], dtype=object)

### 9\. [✔] TB_RSM_ORG(전문정비사업자)

In [601]:
# 0.3s
name = 'tb_rsm_org_202311071037'
file_name = f'{name}.csv'
rsm_org = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsm_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   org_id             652 non-null    int64  
 1   org_code           567 non-null    object 
 2   sido               221 non-null    object 
 3   sigungu            221 non-null    object 
 4   receipte_no        652 non-null    object 
 5   receipte_date      652 non-null    int64  
 6   send_date          651 non-null    float64
 7   org_nm             652 non-null    object 
 8   org_owner          652 non-null    object 
 9   org_addr1          652 non-null    object 
 10  org_addr2          492 non-null    object 
 11  org_owner_tel_no   652 non-null    object 
 12  org_post_no        652 non-null    object 
 13  org_simple_addr    652 non-null    object 
 14  org_tel_no         652 non-null    object 
 15  ins_item_group_id  0 non-null      float64
 16  rec_date           652 non

In [602]:
rsm_org_dict = {
    'org_id':'전문정비사업자SEQ',
    'org_code':'사업자코드',
    'sido':'시도',
    'sigungu':'시군구',
    'receipte_no':'등록번호',
    'receipte_date':'등록일자',
    'send_date':'입력일자',
    'org_nm':'상호',
    'org_owner':'성명',
    'org_addr1':'주소1',
    'org_addr2':'주소2',
    'org_owner_tel_no':'전화번호',
    'org_post_no':'우편번호',
    'org_simple_addr':'사업소재지',
    'org_tel_no':'사업소재지전화번호',
    'ins_item_group_id':'확인검사항목ID',
    'rec_date':'등록일자',
    'rec_nm':'등록자',
    'las_chg_date':'수정일자',
    'las_chg_nm':'수정자',
    'co':'일산화탄소',
    'ch':'탄화수소',
    'no':'질소산화물',
    'air':'공기과잉률_여부',
    'smoke':'매연',
    'org_reg_no':'사업자번호',
    'us_org_id':'전문정비사업자코드',
    'org_status':'등록_승인_상태',
    'org_gld':'운행차배출가스전문정비업',
    'org_ele_status':'전산_승인_상태',
    'rsm_type_cd':'사업자_구분',
    'rsm_join_yn':'대행여부',
    'rsm_join_org_nm':'대행업체명',
    'owner_reg_no':'성명',
    'nos':'소음',
    'tcp_yn':'TCP여부',
    'sido_cd':'법정동_시도코드',
    'sigungu_cd':'법정도_시군구코드',
    'x_coord':'X좌표',
    'y_coord':'Y좌표 ',
    'sido_nm':'법정동_시도명',
    'sigungu_nm':'법정동_시군구명',
}
rsm_orgr = rsm_org.rename(columns=rsm_org_dict)
rsm_orgr.columns

Index(['전문정비사업자SEQ', '사업자코드', '시도', '시군구', '등록번호', '등록일자', '입력일자', '상호', '성명',
       '주소1', '주소2', '전화번호', '우편번호', '사업소재지', '사업소재지전화번호', '확인검사항목ID', '등록일자',
       '등록자', '수정일자', '수정자', '일산화탄소', '탄화수소', '질소산화물', '공기과잉률_여부', '매연',
       '사업자번호', '전문정비사업자코드', '등록_승인_상태', '운행차배출가스전문정비업', '전산_승인_상태', '사업자_구분',
       '대행여부', '대행업체명', '성명', '소음', 'TCP여부', '법정동_시도코드', '법정도_시군구코드', 'X좌표',
       'Y좌표 ', '법정동_시도명', '법정동_시군구명'],
      dtype='object')

In [603]:
rsm_orgr['대행업체명'].unique()

array([nan, '성동그린 모터스'], dtype=object)

In [604]:
rsm_orgr[['시도', '시군구', '법정동_시도명', '법정동_시군구명', '법정동_시도코드', '법정도_시군구코드', '상호', '전문정비사업자코드']].head()

,시도,시군구,법정동_시도명,법정동_시군구명,법정동_시도코드,법정도_시군구코드,상호,전문정비사업자코드
0,NaN,NaN,경상남도,김해시,48,250,김해자동차정비공업사,ZO12
1,NaN,NaN,대구광역시,서구,27,170,경북1급종합정비,ZN12
2,NaN,NaN,경상북도,안동시,47,170,영호카서비스,ZN10
3,NaN,NaN,대구광역시,동구,27,140,스피드1급카센타,ZC26
4,NaN,NaN,서울특별시,마포구,11,440,삼진자동차공업사,ZA88


In [605]:
rsm_orgr['전문정비사업자SEQ'].head()

0    316
1    404
2    380
3    419
4    401
Name: 전문정비사업자SEQ, dtype: int64

In [606]:
rsm_orgr['사업소재지'].head()

0        경상남도 김해시 주촌면 골든루트로 274
1    대구광역시 서구  북비산로17길 12 (이현동)
2       경상북도 안동시  경동로 376 (송현동)
3          대구광역시 동구 송라로 34(신천동)
4     서울특별시 마포구  새터산길 1-5 (성산동)
Name: 사업소재지, dtype: object

In [607]:
rsm_orgr['주소1'].head()

0             경상남도 김해시 주촌면 골든루트로 274
1         대구광역시 서구  북비산로17길 12 (이현동)
2            경상북도 안동시  경동로 376 (송현동)
3    대구광역시 동구  신암남로 50 (신암동,신암건영캐스빌)
4          서울특별시 마포구  새터산길 1-5 (성산동)
Name: 주소1, dtype: object

In [608]:
rsm_orgr['주소2'].head()

0    김해자동차정비공업사
1      이현동 14-4
2    송현동 271-45
3    107동 1307호
4            1층
Name: 주소2, dtype: object

In [609]:
rsm_orgr['소음'].unique()

array(['N', nan, 'Y'], dtype=object)

### 10\. TB_RSO_REG(개선명령)

In [610]:
# 0.3s
name = 'tb_rso_reg_202311071037'
file_name = f'{name}.csv'
rso_reg = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rso_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rso_idx          4192 non-null   int64  
 1   ge_idx           4191 non-null   float64
 2   sub_idx          4186 non-null   float64
 3   reg_no           4192 non-null   int64  
 4   up_rso_date      4191 non-null   float64
 5   up_exp_date      4192 non-null   int64  
 6   rso_reason       2848 non-null   object 
 7   rso_type         4189 non-null   float64
 8   doc_name         3994 non-null   object 
 9   doc_birth        2657 non-null   object 
 10  doc_post         2854 non-null   object 
 11  doc_addr1        2827 non-null   object 
 12  doc_addr2        2825 non-null   object 
 13  doc_tel_no       2853 non-null   object 
 14  doc_last_site    2923 non-null   object 
 15  doc_last_time    2923 non-null   object 
 16  rso_reg_status   4192 non-null   object 
 17  rse_status    

In [611]:
rso_reg_dict = {
    'rso_idx':'일련번호',
    'ge_idx':'TB_RSD_GE_LIST의_GE_IDX',
    'sub_idx':'TB_RSD_GE_SUB의_SUB_IDX',
    'reg_no':'접수번호(지역별_자동순번)',
    'up_rso_date':'개선명령일',
    'up_exp_date':'개선기한',
    'rso_reason':'명령사유',
    'rso_type':'명령구분',
    'doc_name':'소유자_성명',
    'doc_birth':'소유자_생년월일',
    'doc_post':'소유자_우편번호',
    'doc_addr1':'소유자_주소1',
    'doc_addr2':'소유자_주소2',
    'doc_tel_no':'소유자_전화번호',
    'doc_last_site':'최종점검장소(원격측정)',
    'doc_last_time':'최종점검일시(원격측정)',
    'rso_reg_status':'개선명령여부(Y:명령완료,R:이의접수)',
    'rse_status':'확인검사여부(Y:완료,N:미완료)',
    'reg_region':'사업장소재지',
    'reg_tel_no':'사업장전화번호',
    'org_id':'지자체_ORGID',
    'rsd_month':'RSD년월',
    'rec_date':'등록일자',
    'rec_nm':'등록자',
    'las_chg_date':'최종수정일자',
    'las_chg_nm':'최종수정자',
    'reg_no_full':'접수번호(명령서용)',
    'rsd_idx':'TB_RSD_GE_LIST의_RSD_IDX',
    'rso_reason_stop':'운행정지사유',
    'stop_from_date':'운행정지시작일',
    'stop_to_date':'운행정지만료일',
    'rso_stop_type':'운행정지코드(항상2)',
    'print_yn':'출력여부',
    'print_date':'출력일시',
    'print_rec':'출력자',
    'rso_stop_status':'운행정지상태',
    'r_reason':'이의접수사유',
}
rso_regr = rso_reg.rename(columns=rso_reg_dict)
rso_regr.columns

Index(['일련번호', 'TB_RSD_GE_LIST의_GE_IDX', 'TB_RSD_GE_SUB의_SUB_IDX',
       '접수번호(지역별_자동순번)', '개선명령일', '개선기한', '명령사유', '명령구분', '소유자_성명', '소유자_생년월일',
       '소유자_우편번호', '소유자_주소1', '소유자_주소2', '소유자_전화번호', '최종점검장소(원격측정)',
       '최종점검일시(원격측정)', '개선명령여부(Y:명령완료,R:이의접수)', '확인검사여부(Y:완료,N:미완료)', '사업장소재지',
       '사업장전화번호', '지자체_ORGID', 'RSD년월', '등록일자', '등록자', '최종수정일자', '최종수정자',
       '접수번호(명령서용)', 'TB_RSD_GE_LIST의_RSD_IDX', '운행정지사유', '운행정지시작일', '운행정지만료일',
       '운행정지코드(항상2)', '출력여부', '출력일시', '출력자', '운행정지상태', '이의접수사유'],
      dtype='object')

In [612]:
rso_regr['확인검사여부(Y:완료,N:미완료)'].unique()

array(['Y', 'N'], dtype=object)

In [613]:
rso_regr['접수번호(지역별_자동순번)'].describe()

count   4192.00
mean      51.97
std       62.52
min        1.00
25%        8.00
50%       31.00
75%       73.00
max      343.00
Name: 접수번호(지역별_자동순번), dtype: float64

### 11\. TB_US_ORG(MECAR 업무기관)

In [614]:
# 0.3s
name = 'tb_us_org_202311071037'
file_name = f'{name}.csv'
us_org = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
us_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15711 entries, 0 to 15710
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   org_id            15711 non-null  object 
 1   biz_cd            15711 non-null  object 
 2   org_nm            15711 non-null  object 
 3   user_grp_cd       15711 non-null  object 
 4   area_cd           15711 non-null  object 
 5   owner_nm          15711 non-null  object 
 6   owner_post_no     10714 non-null  object 
 7   owner_addr1       10756 non-null  object 
 8   owner_addr2       10243 non-null  object 
 9   org_post_no       14659 non-null  object 
 10  org_addr1         14755 non-null  object 
 11  org_addr2         13466 non-null  object 
 12  org_tel_no        12752 non-null  object 
 13  dedicate_no       1 non-null      float64
 14  dedicate_date     1 non-null      float64
 15  reg_date          15072 non-null  object 
 16  reg_id            14950 non-null  object

In [615]:
us_org_dict = {
    'org_id':'기관코드',
    'biz_cd':'업무유형코드',
    'org_nm':'기관명',
    'user_grp_cd':'사용자권한그룹코드',
    'area_cd':'AREA_CD',
    'owner_nm':'대표자명',
    'owner_post_no':'대표자우편번호',
    'owner_addr1':'대표자주소1',
    'owner_addr2':'대표자주소2',
    'org_post_no':'기관우편번호',
    'org_addr1':'기관주소1',
    'org_addr2':'기관주소2',
    'org_tel_no':'기관연락처',
    'dedicate_no':'DEDICATE_NO',
    'dedicate_date':'DEDICATE_DATE',
    'reg_date':'등록일자',
    'reg_id':'등록자ID',
    'sido_cd':'시도코드',
    'sigungu_cd':'시군구코드',
    'max_user_cnt':'허용사용자수',
    'auto_lock_period':'자동잠금기간',
}
us_orgr = us_org.rename(columns=us_org_dict)
us_orgr.columns

Index(['기관코드', '업무유형코드', '기관명', '사용자권한그룹코드', 'AREA_CD', '대표자명', '대표자우편번호',
       '대표자주소1', '대표자주소2', '기관우편번호', '기관주소1', '기관주소2', '기관연락처', 'DEDICATE_NO',
       'DEDICATE_DATE', '등록일자', '등록자ID', '시도코드', '시군구코드', '허용사용자수', '자동잠금기간'],
      dtype='object')

In [616]:
us_orgr['자동잠금기간'].unique()

array([ 90, 365], dtype=int64)

In [617]:
us_orgr['허용사용자수'].describe()

count   14070.00
mean        5.82
std         2.48
min         0.00
25%         5.00
50%         5.00
75%         5.00
max       150.00
Name: 허용사용자수, dtype: float64

In [618]:
us_orgr['DEDICATE_NO'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: DEDICATE_NO, dtype: float64

In [619]:
us_orgr['DEDICATE_NO'].value_counts(dropna=False)

DEDICATE_NO
NaN          15710
111111.00        1
Name: count, dtype: int64

In [620]:
us_orgr['DEDICATE_DATE'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: DEDICATE_DATE, dtype: float64

In [621]:
us_orgr['DEDICATE_DATE'].value_counts(dropna=False)

DEDICATE_DATE
NaN          15710
111111.00        1
Name: count, dtype: int64

### 12\. [✔] TB_RSE_DET_EX(확인검사 상세 실행)

In [622]:
# 0.3s
name = 'tb_rse_det_ex_202312061021'
file_name = f'{name}.csv'
rse_det_ex = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep=',')
rse_det_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675495 entries, 0 to 675494
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   rse_idx          675495 non-null  int64  
 1   det_date         0 non-null       float64
 2   repair_dev_cd    675495 non-null  object 
 3   repair_part_cd   675495 non-null  object 
 4   repair_part1_yn  675495 non-null  object 
 5   repair_part2_yn  675495 non-null  object 
 6   repair_part3_yn  675495 non-null  object 
 7   repair_part4_yn  675495 non-null  object 
 8   rec_date         675495 non-null  int64  
 9   rec_nm           112410 non-null  object 
 10  las_chg_date     37849 non-null   float64
 11  las_chg_nm       0 non-null       float64
dtypes: float64(3), int64(2), object(7)
memory usage: 61.8+ MB


In [623]:
rse_det_ex.columns.to_list()

['rse_idx',
 'det_date',
 'repair_dev_cd',
 'repair_part_cd',
 'repair_part1_yn',
 'repair_part2_yn',
 'repair_part3_yn',
 'repair_part4_yn',
 'rec_date',
 'rec_nm',
 'las_chg_date',
 'las_chg_nm']

In [624]:
det_chc = {
    'rse_idx':'일련번호', 
    'det_date':'상세일자', 
    'repair_dev_cd':'정비내용', # 내용상 정비내용
    'repair_part_cd':'정비부품명', # 내용상 정비부품명
    'repair_part1_yn':'교환유무', 
    'repair_part2_yn':'점검조정유무', 
    'repair_part3_yn':'분해탈착유무', 
    'repair_part4_yn':'정비불가능유무', 
    'rec_date':'생성일자', 
    'rec_nm':'생성자명', 
    'las_chg_date':'최종수정일자', 
    'las_chg_nm':'최종수정자명', 
}
rse_det_exr = rse_det_ex.rename(columns=det_chc)
rse_det_exr.columns

Index(['일련번호', '상세일자', '정비내용', '정비부품명', '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무',
       '생성일자', '생성자명', '최종수정일자', '최종수정자명'],
      dtype='object')

In [625]:
rse_det_exr['일련번호'].describe()

count   675495.00
mean     31215.46
std      36814.28
min          1.00
25%       6296.00
50%      11379.00
75%      53477.50
max     121144.00
Name: 일련번호, dtype: float64

In [626]:
rse_det_exr.shape, len(rse_det_exr['일련번호'].unique())

((675495, 12), 117546)

In [627]:
rse_det_exr[['상세일자', '생성일자']].isnull().sum()

상세일자    675495
생성일자         0
dtype: int64

#### 정비내용 해석

In [628]:
rse_det_exr['정비내용'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype=object)

In [629]:
repair_dev_cd_dict = {
    'A':'점화계통', 
    'B':'연료장치', 
    'C':'엔진계통', 
    'D':'배출가스전환장치', 
    'E':'배출가스재순환장치', 
    'F':'연료증발가스방지장치', 
    'G':'블로바이가스환원장치', 
    'H':'각종센서', 
    'I':'흡기계통', 
    'J':'배기계통', 
}
rse_det_exr['정비내용'] = rse_det_exr['정비내용'].replace(repair_dev_cd_dict)
rse_det_exr['정비내용'].unique()

array(['점화계통', '연료장치', '엔진계통', '배출가스전환장치', '배출가스재순환장치', '연료증발가스방지장치',
       '블로바이가스환원장치', '각종센서', '흡기계통', '배기계통'], dtype=object)

#### 정비부품명 해석

In [630]:
rse_det_exr['정비부품명'].unique()

array(['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7',
       'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'E1', 'E2', 'F1',
       'F2', 'G1', 'G2', 'H1', 'H2', 'H3', 'H4', 'I1', 'I2', 'I3', 'J1',
       'J2'], dtype=object)

In [631]:
repair_part_cd_dict = {
    'A1':'점화플러그', 
    'A2':'점화배선', 
    'A3':'배전기(Distributor)', 
    'A4':'점화시기(분사시기)조정', 
    'B1':'인젝터(휘발유)', 
    'B2':'기화기(믹서포함)', 
    'B3':'연료압력조절기', 
    'B4':'연료휠터', 
    'B5':'연료분사펌프(조속기조정,봉인)', 
    'B6':'노즐(디젤인젝터포함)', 
    'B7':'공연비(혼합비)', 
    'C1':'엔진정비', 
    'C2':'엔진오일', 
    'C3':'냉각계통', 
    'C4':'공회전속도', 
    'C5':'전자제어장치(ECU)', 
    'D1':'정화용촉매(CatalyticConverter)', 
    'D2':'디젤산화촉매(DOC)', 
    'D3':'매연여과장치(DPF)', 
    'E1':'EGR밸브', 
    'E2':'EGR제어용서머밸브', 
    'F1':'정화조절밸브(PurgeControlValve)', 
    'F2':'캐니스터및필터', 
    'G1':'PCV밸브', 
    'G2':'블로바이가스호스', 
    'H1':'산소센서', 
    'H2':'공기유량센서', 
    'H3':'드로틀포지션센서', 
    'H4':'냉각수온센서', 
    'I1':'에어크리너', 
    'I2':'드로틀바디', 
    'I3':'과급기', 
    'J1':'소음기', 
    'J2':'배기관', 
    }
rse_det_exr['정비부품명'] = rse_det_exr['정비부품명'].replace(repair_part_cd_dict)
rse_det_exr['정비부품명'].unique()

array(['점화플러그', '점화배선', '배전기(Distributor)', '점화시기(분사시기)조정', '인젝터(휘발유)',
       '기화기(믹서포함)', '연료압력조절기', '연료휠터', '연료분사펌프(조속기조정,봉인)', '노즐(디젤인젝터포함)',
       '공연비(혼합비)', '엔진정비', '엔진오일', '냉각계통', '공회전속도', '전자제어장치(ECU)',
       '정화용촉매(CatalyticConverter)', '디젤산화촉매(DOC)', '매연여과장치(DPF)', 'EGR밸브',
       'EGR제어용서머밸브', '정화조절밸브(PurgeControlValve)', '캐니스터및필터', 'PCV밸브',
       '블로바이가스호스', '산소센서', '공기유량센서', '드로틀포지션센서', '냉각수온센서', '에어크리너',
       '드로틀바디', '과급기', '소음기', '배기관'], dtype=object)

In [632]:
rse_det_exr.columns

Index(['일련번호', '상세일자', '정비내용', '정비부품명', '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무',
       '생성일자', '생성자명', '최종수정일자', '최종수정자명'],
      dtype='object')

In [633]:
rse_det_exr[['상세일자', '생성일자', '최종수정일자']].isnull().sum()

상세일자      675495
생성일자           0
최종수정일자    637646
dtype: int64

In [634]:
rse_det_exr[['일련번호', '정비내용', '정비부품명', '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '상세일자', '최종수정일자', ]].head()

,일련번호,정비내용,정비부품명,교환유무,점검조정유무,분해탈착유무,정비불가능유무,생성일자,상세일자,최종수정일자
0,9,점화계통,점화플러그,N,N,N,N,20130806,NaN,NaN
1,9,점화계통,점화배선,N,N,N,N,20130806,NaN,NaN
2,9,점화계통,배전기(Distributor),N,N,N,N,20130806,NaN,NaN
3,9,점화계통,점화시기(분사시기)조정,N,N,N,N,20130806,NaN,NaN
4,9,연료장치,인젝터(휘발유),Y,N,N,N,20130806,NaN,NaN


In [635]:
rse_det_exr.loc[rse_det_exr['정비내용'] == '연료장치', '정비부품명'].unique()

array(['인젝터(휘발유)', '기화기(믹서포함)', '연료압력조절기', '연료휠터', '연료분사펌프(조속기조정,봉인)',
       '노즐(디젤인젝터포함)', '공연비(혼합비)'], dtype=object)

#### 정비 내용 수정

In [636]:
# 정비내용 : 연료장치와 엔지계통 뒤바뀌어 있음
print(rse_det_exr.loc[rse_det_exr['정비내용'] == '연료장치', '정비부품명'].unique())
print()
print(rse_det_exr.loc[rse_det_exr['정비내용'] == '엔진계통', '정비부품명'].unique())


['인젝터(휘발유)' '기화기(믹서포함)' '연료압력조절기' '연료휠터' '연료분사펌프(조속기조정,봉인)' '노즐(디젤인젝터포함)'
 '공연비(혼합비)']

['엔진정비' '엔진오일' '냉각계통' '공회전속도' '전자제어장치(ECU)']


In [637]:
rse_det_exr['정비내용'] = rse_det_exr['정비내용'].replace({'연료장치':'엔진계통', '엔진계통':'연료장치'})

In [638]:
# 정비내용 : 연료장치와 엔지계통 뒤바뀌어 있음
print(rse_det_exr.loc[rse_det_exr['정비내용'] == '연료장치', '정비부품명'].unique())
print()
print(rse_det_exr.loc[rse_det_exr['정비내용'] == '엔진계통', '정비부품명'].unique())


['엔진정비' '엔진오일' '냉각계통' '공회전속도' '전자제어장치(ECU)']

['인젝터(휘발유)' '기화기(믹서포함)' '연료압력조절기' '연료휠터' '연료분사펌프(조속기조정,봉인)' '노즐(디젤인젝터포함)'
 '공연비(혼합비)']


## 전처리

### TB_RSD_GE_LIST_EX, TB_RSE_DET_EX, TB_RSM_ORG 병합
- 1. TB_RSE_DET_EX : '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무' 중 단 하나라도 'Y'가 있는 샘플만 추출
- 2. TB_RSD_GE_LIST_EX(일련번호), TB_RSE_DET_EX(일련번호) 기준 병합
- 3. 2번과 TB_RSM_ORG '전문정비사업자코드' 기준으로 병합

In [639]:
gl = rsd_ge_list_exr.copy()
det = rse_det_exr.copy()
rsd_ge_list_ex.shape, rse_det_exr.shape, gl.shape, det.shape

((119898, 70), (675495, 12), (119898, 71), (675495, 12))

In [640]:
det.columns

Index(['일련번호', '상세일자', '정비내용', '정비부품명', '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무',
       '생성일자', '생성자명', '최종수정일자', '최종수정자명'],
      dtype='object')

In [641]:
det['교환유무'].unique()

array(['N', 'Y'], dtype=object)

#### 1\. TB_RSE_DET_EX : '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무' 중 단 하나라도 'Y'가 있는 샘플만 추출

In [642]:
det01 = det[
    (det['교환유무'] == 'Y') | 
    (det['점검조정유무'] == 'Y') | 
    (det['분해탈착유무'] == 'Y') | 
    (det['정비불가능유무'] == 'Y')
    ].reset_index(drop=True)
det01.shape

(267003, 12)

In [643]:
det01.shape, len(det01['일련번호'].unique())

((267003, 12), 117540)

In [644]:
det01.loc[det01['일련번호'].duplicated(keep=False), ['일련번호', '정비내용', '정비부품명', '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자']].head()

,일련번호,정비내용,정비부품명,교환유무,점검조정유무,분해탈착유무,정비불가능유무,생성일자
1,14,배출가스재순환장치,EGR밸브,Y,N,N,N,20130807
2,14,배출가스재순환장치,EGR제어용서머밸브,Y,N,N,N,20130807
3,19,엔진계통,"연료분사펌프(조속기조정,봉인)",Y,N,N,N,20130808
4,19,배출가스재순환장치,EGR밸브,Y,N,N,N,20130808
5,19,배출가스재순환장치,EGR제어용서머밸브,Y,N,N,N,20130808


In [645]:
det01.shape

(267003, 12)

In [646]:
det01.head()

,일련번호,상세일자,정비내용,정비부품명,교환유무,점검조정유무,분해탈착유무,정비불가능유무,생성일자,생성자명,최종수정일자,최종수정자명
0,9,NaN,엔진계통,인젝터(휘발유),Y,N,N,N,20130806,NaN,NaN,NaN
1,14,NaN,배출가스재순환장치,EGR밸브,Y,N,N,N,20130807,NaN,20130807.00,NaN
2,14,NaN,배출가스재순환장치,EGR제어용서머밸브,Y,N,N,N,20130807,NaN,20130807.00,NaN
3,19,NaN,엔진계통,"연료분사펌프(조속기조정,봉인)",Y,N,N,N,20130808,NaN,NaN,NaN
4,19,NaN,배출가스재순환장치,EGR밸브,Y,N,N,N,20130808,NaN,NaN,NaN


#### 2\. TB_RSD_GE_LIST_EX(일련번호), TB_RSE_DET_EX(일련번호) 기준 병합

In [647]:
gl.columns

Index(['일련번호', '점검일자', '차량번호', '차대번호', '차명', '차종', '연식', '용도', '연료', '중량',
       '총중량', '엔진배기량', '엔진출력', '엔진형식', '주행거리', '제작일자', '최초등록일', '정밀검사만료일자',
       '시도', '시군구', '전문정비사업자코드', '보증기간_년', '보증기간_Km', '보증기간내_고의_또는_과실여부',
       '정비사성명', '정비사의견', '차령', '정비여부', '생성일자', '생성자명', '최종수정자명', '최종수정일자',
       '보증수리여부', '소재지', '소유자명', '접수번호', '출력여부', '출력일자', '출력자명', '시도코드',
       '시군구코드', '시도명', '시군구명', '정밀검사유무', '정밀검사일자', '정밀검사소명', '정밀검사측정방법',
       '정밀검사판정', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2',
       '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3',
       '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5',
       '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '관능검사판정', '관능검사부적합내용',
       '정밀검사수정일', '확정여부', '차종유형'],
      dtype='object')

In [648]:
gl = gl[[
    '일련번호',
    '점검일자',
    '차량번호',
    '차대번호',
    '차명',
    '차종',
    '연식',
    '용도',
    '연료',
    '중량',
    '총중량',
    '엔진배기량',
    '엔진출력',
    '엔진형식',
    '주행거리',
    '제작일자',
    '최초등록일',
    # '정밀검사만료일자',
    # '시도',
    # '시군구',
    '전문정비사업자코드',
    # '보증기간_년',
    # '보증기간_Km',
    # '보증기간내_고의_또는_과실여부',
    # '정비사성명',
    # '정비사의견',
    # '차령',
    '정비여부',
    # '생성일자',
    # '생성자명',
    # '최종수정자명',
    # '최종수정일자',
    # '보증수리여부',
    # '소재지',
    # '소유자명',
    # '접수번호',
    # '출력여부',
    # '출력일자',
    # '출력자명',
    '시도코드',
    '시군구코드',
    '시도명',
    '시군구명',
    # '정밀검사유무',
    # '정밀검사일자',
    # # '정밀검사소명',
    # '정밀검사측정방법',
    # '정밀검사판정',
    # '무부하매연측정치1',
    # '무부하매연허용치1',
    # '무부하매연판정1',
    # '무부하매연측정치2',
    # '무부하매연허용치2',
    # '무부하매연판정2',
    # '무부하매연측정치3',
    # '무부하매연허용치3',
    # '무부하매연판정3',
    # '무부하매연측정치4',
    # '무부하매연허용치4',
    # '무부하매연판정4',
    # '무부하매연측정치5',
    # '무부하매연허용치5',
    # '무부하매연판정5',
    # '무부하매연측정치6',
    # '무부하매연허용치6',
    # '무부하매연판정6',
    # '관능검사판정',
    # '관능검사부적합내용',
    # '정밀검사수정일',
    # '확정여부',
]]

In [649]:
gl.shape, len(gl['일련번호'].unique())

((119898, 23), 119898)

In [650]:
det01.shape, len(det01['일련번호'].unique())

((267003, 12), 117540)

In [651]:
gd = gl.merge(det01, on='일련번호', how='left')
gd.shape

(268752, 34)

In [652]:
gd.columns

Index(['일련번호', '점검일자', '차량번호', '차대번호', '차명', '차종', '연식', '용도', '연료', '중량',
       '총중량', '엔진배기량', '엔진출력', '엔진형식', '주행거리', '제작일자', '최초등록일', '전문정비사업자코드',
       '정비여부', '시도코드', '시군구코드', '시도명', '시군구명', '상세일자', '정비내용', '정비부품명', '교환유무',
       '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자', '최종수정자명'],
      dtype='object')

#### 3\. 2번과 TB_RSM_ORG '전문정비사업자코드' 기준으로 병합

In [653]:
rsm_orgr.columns

Index(['전문정비사업자SEQ', '사업자코드', '시도', '시군구', '등록번호', '등록일자', '입력일자', '상호', '성명',
       '주소1', '주소2', '전화번호', '우편번호', '사업소재지', '사업소재지전화번호', '확인검사항목ID', '등록일자',
       '등록자', '수정일자', '수정자', '일산화탄소', '탄화수소', '질소산화물', '공기과잉률_여부', '매연',
       '사업자번호', '전문정비사업자코드', '등록_승인_상태', '운행차배출가스전문정비업', '전산_승인_상태', '사업자_구분',
       '대행여부', '대행업체명', '성명', '소음', 'TCP여부', '법정동_시도코드', '법정도_시군구코드', 'X좌표',
       'Y좌표 ', '법정동_시도명', '법정동_시군구명'],
      dtype='object')

In [654]:
len(set(gd['전문정비사업자코드'].unique()) - set(rsm_orgr['전문정비사업자코드'].unique()))

16

In [655]:
rsm_orgr[['시도', '시군구', '법정동_시도명', '법정동_시군구명', '법정동_시도코드', '법정도_시군구코드', '상호', '전문정비사업자코드']].head()

,시도,시군구,법정동_시도명,법정동_시군구명,법정동_시도코드,법정도_시군구코드,상호,전문정비사업자코드
0,NaN,NaN,경상남도,김해시,48,250,김해자동차정비공업사,ZO12
1,NaN,NaN,대구광역시,서구,27,170,경북1급종합정비,ZN12
2,NaN,NaN,경상북도,안동시,47,170,영호카서비스,ZN10
3,NaN,NaN,대구광역시,동구,27,140,스피드1급카센타,ZC26
4,NaN,NaN,서울특별시,마포구,11,440,삼진자동차공업사,ZA88


In [656]:
rsm_orgr = rsm_orgr.rename(columns={'법정동_시도명':'전문정비사업자_시도명', '법정동_시군구명':'전문정비사업자_시군구명', '법정동_시도코드':'전문정비사업자_시도코드', '법정도_시군구코드':'전문정비사업자_시군구코드', '상호':'전문정비사업소명'})
rsm_orgr.columns

Index(['전문정비사업자SEQ', '사업자코드', '시도', '시군구', '등록번호', '등록일자', '입력일자', '전문정비사업소명',
       '성명', '주소1', '주소2', '전화번호', '우편번호', '사업소재지', '사업소재지전화번호', '확인검사항목ID',
       '등록일자', '등록자', '수정일자', '수정자', '일산화탄소', '탄화수소', '질소산화물', '공기과잉률_여부',
       '매연', '사업자번호', '전문정비사업자코드', '등록_승인_상태', '운행차배출가스전문정비업', '전산_승인_상태',
       '사업자_구분', '대행여부', '대행업체명', '성명', '소음', 'TCP여부', '전문정비사업자_시도코드',
       '전문정비사업자_시군구코드', 'X좌표', 'Y좌표 ', '전문정비사업자_시도명', '전문정비사업자_시군구명'],
      dtype='object')

In [657]:
# 1 : 등록, 2:승인, 3:취소
rsm_orgr['등록_승인_상태'].value_counts(dropna=False)

등록_승인_상태
1    324
2    298
3     30
Name: count, dtype: int64

In [658]:
temp = rsm_orgr.loc[((rsm_orgr['등록_승인_상태'] == 1) | (rsm_orgr['등록_승인_상태'] == 2)) & (rsm_orgr['전문정비사업자코드'].notnull()), ['전문정비사업자코드', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드', '전문정비사업자_시군구코드', '전문정비사업소명']]
temp.shape

(545, 6)

In [659]:
len(temp['전문정비사업자코드'].unique())

545

In [660]:
gd.shape

(268752, 34)

In [661]:
# 전문정비사업소명(등록, 승인만) 정보 추가
gdo = gd.merge(rsm_orgr.loc[((rsm_orgr['등록_승인_상태'] == 1) | (rsm_orgr['등록_승인_상태'] == 2)) & (rsm_orgr['전문정비사업자코드'].notnull()), ['전문정비사업자코드', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드', '전문정비사업자_시군구코드', '전문정비사업소명']], on='전문정비사업자코드', how='left')
gdo.shape

(268752, 39)

In [662]:
gdo['연료'].unique()

array(['경유', 'LPG', '휘발유', '하이브리드(휘발유+전기)', nan, 'CNG', '하이브리드(LPG+전기)',
       '기타연료'], dtype=object)

In [663]:
len(gdo['일련번호'].unique())

119898

In [664]:
gdo['일련번호'].isnull().sum()

0

In [665]:
ggrp1 = gdo.groupby(['전문정비사업자_시도명']).agg({'전문정비사업자코드':lambda x:len(x.unique())}).reset_index()
ggrp1.loc[ggrp1.shape[0], ['전문정비사업자_시도명', '전문정비사업자코드']] = ['합계', ggrp1['전문정비사업자코드'].sum()]
ggrp1

,전문정비사업자_시도명,전문정비사업자코드
0,강원도,14.00
1,경기도,55.00
2,경상남도,38.00
3,경상북도,45.00
4,광주광역시,16.00
5,대구광역시,39.00
6,대전광역시,22.00
7,부산광역시,17.00
8,서울특별시,76.00
9,세종특별자치시,3.00


### 정기/정밀 검사 정보

#### 검사 측정치 음수 확인
- 수정 : -9999 -> np.nan

In [666]:
ins.loc[ins['무부하매연측정치1'] < 0, '무부하매연측정치1'].shape

(6201575,)

In [667]:
ins.loc[ins['무부하매연측정치2'] < 0, '무부하매연측정치2'].shape

(73231712,)

In [668]:
ins.loc[ins['무부하매연측정치3'] < 0, '무부하매연측정치3'].shape

(75662369,)

In [669]:
ins['무부하매연측정치1'].describe()

count   173740086.00
mean         -349.78
std          1856.56
min         -9999.00
25%             0.00
50%             0.02
75%             8.00
max         97938.00
Name: 무부하매연측정치1, dtype: float64

In [670]:
ins.loc[ins['무부하매연측정치1'] < 0, '무부하매연측정치1'].unique()

array([-9999.])

In [671]:
ins.loc[ins['무부하매연측정치2'] < 0, '무부하매연측정치2'].unique()

array([-9999.])

In [672]:
ins.loc[ins['무부하매연측정치3'] < 0, '무부하매연측정치3'].unique()

array([-9999.])

In [673]:
ins.loc[ins['무부하매연측정치1'] < 0, '무부하매연측정치1'] = np.nan
ins.loc[ins['무부하매연측정치2'] < 0, '무부하매연측정치2'] = np.nan
ins.loc[ins['무부하매연측정치3'] < 0, '무부하매연측정치3'] = np.nan

### 전문, gdo concat
- 과거 전문정비
    - 전문정비 + 상세 + 등록 + 제원 + 법정동 : bf_er
- 최근 전문정비
    - TB_RSD_GE_LIST_EX + TB_RSE_DET_EX + TB_RSM_ORG : gdo

#### 1\.과거, 최신 데이터 병합

In [674]:
gdo['점검일자'].dtype

dtype('int64')

In [675]:
gdo['점검일자'].head()

0    20130805
1    20130806
2    20130806
3    20130806
4    20130806
Name: 점검일자, dtype: int64

In [676]:
gdo.shape, gdo['점검일자'].isnull().sum()

((268752, 39), 0)

In [677]:
bf_er.columns

Index(['차량번호', '차대번호', '정비일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명'],
      dtype='object')

In [678]:
bf_er['정비내용'].unique()

array(['엔진계통', '연료장치', '각종센서', '흡기계통', '배출가스재순환장치', '배출가스전환장치', '배기계통',
       '점화계통', '연료증발가스방지장치', '블로바이가스환원장치', nan], dtype=object)

In [679]:
bf_er['정비부품명'].unique()

array(['연료휠터', '엔진오일', '연료압력조절기', '냉각수온센서', '연료분사펌프(조속기조정,봉인)', '에어크리너',
       '엔진정비', '노즐(디젤인젝터포함)', 'EGR밸브', '기화기(믹서포함)',
       '정화용촉매(CatalyticConverter)', '소음기', '냉각계통', '배기관', 'EGR제어용서머밸브',
       '공연비(혼합비)', '점화시기(분사시기)조정', '점화플러그', '공회전속도', '디젤산화촉매(DOC)',
       '공기유량센서', '과급기', '산소센서', '점화배선', '드로틀바디',
       '정화조절밸브(PurgeControlValve)', '블로바이가스호스', '캐니스터및필터', '인젝터(휘발유)',
       '드로틀포지션센서', '배전기(Distributor)', '전자제어장치(ECU)', 'PCV밸브',
       '매연여과장치(DPF)', nan], dtype=object)

In [680]:
bf_er01 = bf_er.rename(columns={'정비일자':'점검일자'})
bf_er01.columns

Index(['차량번호', '차대번호', '점검일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명'],
      dtype='object')

In [681]:
gdo.columns

Index(['일련번호', '점검일자', '차량번호', '차대번호', '차명', '차종', '연식', '용도', '연료', '중량',
       '총중량', '엔진배기량', '엔진출력', '엔진형식', '주행거리', '제작일자', '최초등록일', '전문정비사업자코드',
       '정비여부', '시도코드', '시군구코드', '시도명', '시군구명', '상세일자', '정비내용', '정비부품명', '교환유무',
       '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자', '최종수정자명',
       '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드', '전문정비사업자_시군구코드',
       '전문정비사업소명'],
      dtype='object')

In [682]:
bf_er01.shape[0] + gdo.shape[0]

375556

In [683]:
total_err = pd.concat([bf_er01, gdo], ignore_index=True)
total_err.shape

(375556, 47)

In [684]:
len(total_err['차대번호'].unique())

101388

In [685]:
total_err['차대번호'].isnull().sum()

7355

In [686]:
total_err.columns

Index(['차량번호', '차대번호', '점검일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '일련번호', '중량',
       '엔진배기량', '주행거리', '제작일자', '최초등록일', '전문정비사업자코드', '정비여부', '시도코드', '시군구코드',
       '상세일자', '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드',
       '전문정비사업자_시군구코드'],
      dtype='object')

In [687]:
total_err = total_err.drop('주행거리', axis=1)
total_err.shape

(375556, 46)

##### 연료 수정
- LPG, LPG(액화석유가스) -> LPG로 통합

In [688]:
total_err['연료'].unique()

array(['경유', nan, 'LPG(액화석유가스)', '휘발유', 'LPG', '하이브리드(휘발유+전기)', 'CNG',
       '하이브리드(LPG+전기)', '기타연료'], dtype=object)

In [689]:
total_err.loc[total_err['연료'] == 'LPG(액화석유가스)', '연료'] = 'LPG'
total_err['연료'].unique()

array(['경유', nan, 'LPG', '휘발유', '하이브리드(휘발유+전기)', 'CNG', '하이브리드(LPG+전기)',
       '기타연료'], dtype=object)

##### 연식 수정

In [690]:
total_err.loc[total_err['연식'].str.contains('-', regex=False, na=False), '연식'].shape

(55,)

In [691]:
total_err.loc[total_err['연식'].str.contains('-', regex=False, na=False), '연식'].head()

128168    2012-11-6
128169    2012-11-6
128170    2012-11-6
128171    2012-11-6
128172    2012-11-6
Name: 연식, dtype: object

In [692]:
total_err.loc[total_err['연식'].str.contains('-', regex=False, na=False), '연식'] = total_err.loc[total_err['연식'].str.contains('-', regex=False, na=False), '연식'].str.split('-').str[0]

In [693]:
total_err.loc[total_err['연식'].str.contains('-', regex=False, na=False), '연식'].shape

(0,)

In [694]:
total_err.loc[128172, '연식']

'2012'

In [695]:
total_err['연식'] = pd.to_numeric(total_err['연식'], errors='coerce')

#### 2\. 정기정밀 검사 병합

In [696]:
total_err.columns

Index(['차량번호', '차대번호', '점검일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '일련번호', '중량',
       '엔진배기량', '제작일자', '최초등록일', '전문정비사업자코드', '정비여부', '시도코드', '시군구코드', '상세일자',
       '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드',
       '전문정비사업자_시군구코드'],
      dtype='object')

In [697]:
total_err['점검일자'].dtype

dtype('int64')

In [698]:
total_err['점검일자'].head()

0    20080110
1    20080123
2    20080108
3    20080118
4    20080118
Name: 점검일자, dtype: int64

In [699]:
ins.columns

Index(['차대번호', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정기정밀구분'],
      dtype='object')

##### 검사일자 타입 변경

In [700]:
ins['검사일자'].dtype

dtype('O')

In [701]:
ins['검사일자'].head()

0    2019-10-07 00:00:00.000000
1    2019-08-28 00:00:00.000000
2    2019-08-28 00:00:00.000000
3    2019-08-28 00:00:00.000000
4    2019-08-28 00:00:00.000000
Name: 검사일자, dtype: object

In [702]:
# 1m 45.7s
ins['검사일자'] = pd.to_datetime(ins['검사일자'], errors='coerce')
ins['검사일자'] = ins['검사일자'].astype('str')
ins['검사일자'] = ins['검사일자'].str.replace('-', '')
ins['검사일자'] = pd.to_numeric(ins['검사일자'], errors='coerce')
ins['검사일자'].head()

0    20191007
1    20190828
2    20190828
3    20190828
4    20190828
Name: 검사일자, dtype: int64

In [703]:
ins['검사종류'].unique()

array(['정기검사', '재검사(2부재검사)', '정기(경과)검사', '재검사(1부재검사)', '특정재검사(배출)',
       '종합검사(경과)', '2부재검사(종합)', '종합검사', '정밀검사', '특정검사', '관능재검사', '배출재검사',
       '정밀검사(경과)', '특정재검사(관능)', '특정검사(경과)', '1부재검사(종합)', '종합(정밀)',
       '종합(정밀) 2부재검사', '종합(경과:정밀)', '종합(정밀) 1부재검사', '구조변경검사', '배출재검사(구변)',
       '9999', '재검사(관능)', '재검사(배출)', '관능재검사(임시)', '임시검사'], dtype=object)

In [704]:
total_err.shape, ins.shape

((375556, 46), (173740086, 25))

In [705]:
# 1m 53.1s
pr4 = total_err.merge(ins, on='차대번호', how='left')
pr4.shape

(5452157, 70)

### 시도명, 전문정비사업자_시도명 수정
- 강원도 -> 강원특별자치도
- 제주도 -> 제주특별자치도

In [706]:
total_err.columns

Index(['차량번호', '차대번호', '점검일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '일련번호', '중량',
       '엔진배기량', '제작일자', '최초등록일', '전문정비사업자코드', '정비여부', '시도코드', '시군구코드', '상세일자',
       '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드',
       '전문정비사업자_시군구코드'],
      dtype='object')

In [707]:
print(total_err['시도명'].unique())
total_err.loc[total_err['시도명'] == '강원도', '시도명'] = '강원특별자치도'
total_err.loc[total_err['시도명'] == '제주도', '시도명'] = '제주특별자치도'
print(total_err['시도명'].unique())

['경기도' nan '부산광역시' '경상북도' '충청북도' '울산광역시' '인천광역시' '전라남도' '충청남도' '전라북도'
 '경상남도' '대구광역시' '제주특별자치도' '강원특별자치도' '대전광역시' '서울특별시' '광주광역시' '세종특별자치시'
 '강원도' '제주도']
['경기도' nan '부산광역시' '경상북도' '충청북도' '울산광역시' '인천광역시' '전라남도' '충청남도' '전라북도'
 '경상남도' '대구광역시' '제주특별자치도' '강원특별자치도' '대전광역시' '서울특별시' '광주광역시' '세종특별자치시']


In [708]:
print(pr4['시도명'].unique())
pr4.loc[pr4['시도명'] == '강원도', '시도명'] = '강원특별자치도'
pr4.loc[pr4['시도명'] == '제주도', '시도명'] = '제주특별자치도'
print(pr4['시도명'].unique())

['경기도' nan '부산광역시' '경상북도' '충청북도' '울산광역시' '인천광역시' '전라남도' '충청남도' '전라북도'
 '경상남도' '대구광역시' '제주특별자치도' '강원특별자치도' '대전광역시' '서울특별시' '광주광역시' '세종특별자치시'
 '강원도' '제주도']
['경기도' nan '부산광역시' '경상북도' '충청북도' '울산광역시' '인천광역시' '전라남도' '충청남도' '전라북도'
 '경상남도' '대구광역시' '제주특별자치도' '강원특별자치도' '대전광역시' '서울특별시' '광주광역시' '세종특별자치시']


In [709]:
# 전문정비사업자_시도명
print(total_err['전문정비사업자_시도명'].unique())
total_err.loc[total_err['전문정비사업자_시도명'] == '강원도', '전문정비사업자_시도명'] = '강원특별자치도'
print(total_err['전문정비사업자_시도명'].unique())

[nan '대구광역시' '충청남도' '경기도' '서울특별시' '광주광역시' '전라북도' '경상남도' '대전광역시' '경상북도'
 '강원도' '인천광역시' '울산광역시' '충청북도' '부산광역시' '전라남도' '세종특별자치시' '제주특별자치도']
[nan '대구광역시' '충청남도' '경기도' '서울특별시' '광주광역시' '전라북도' '경상남도' '대전광역시' '경상북도'
 '강원특별자치도' '인천광역시' '울산광역시' '충청북도' '부산광역시' '전라남도' '세종특별자치시' '제주특별자치도']


In [710]:
# 전문정비사업자_시도명
print(pr4['전문정비사업자_시도명'].unique())
pr4.loc[pr4['전문정비사업자_시도명'] == '강원도', '전문정비사업자_시도명'] = '강원특별자치도'
print(pr4['전문정비사업자_시도명'].unique())

[nan '대구광역시' '충청남도' '경기도' '서울특별시' '광주광역시' '전라북도' '경상남도' '대전광역시' '경상북도'
 '강원도' '인천광역시' '울산광역시' '충청북도' '부산광역시' '전라남도' '세종특별자치시' '제주특별자치도']
[nan '대구광역시' '충청남도' '경기도' '서울특별시' '광주광역시' '전라북도' '경상남도' '대전광역시' '경상북도'
 '강원특별자치도' '인천광역시' '울산광역시' '충청북도' '부산광역시' '전라남도' '세종특별자치시' '제주특별자치도']


In [711]:
set(pr4['시도명'].unique()) - set(pr4['전문정비사업자_시도명'].unique())

set()

### [출력] 과제4 전처리 완료

In [712]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104022'

In [713]:
# 41.4s
total_err.to_csv(os.path.join(raw4_fold, f'과제4_전처리완료_전문정비통합_{today_date_dtl}.csv'), index=False)
print(os.path.join(raw4_fold, f'제4_전처리완료_전문정비통합_{today_date_dtl}.csv'))

D:/data/processing/big3/raw/BD4\제4_전처리완료_전문정비통합_20240119_AM_104022.csv


In [714]:
# 41.4s
pr4.to_csv(os.path.join(raw4_fold, f'과제4_전처리완료_{today_date_dtl}.csv'), index=False)
print(os.path.join(raw4_fold, f'과제4_전처리완료_{today_date_dtl}.csv'))

D:/data/processing/big3/raw/BD4\과제4_전처리완료_20240119_AM_104022.csv


### [로드] 과제4 전처리 완료


In [715]:
# # 과제4_전처리완료_전문정비통합_20240109_AM_025533 : STD_CEG_GRD_LST로 생성

# # 1.2s
# name = '과제4_전처리완료_전문정비통합_20240109_AM_025533'
# file_name = f'{name}.csv'
# total_err = pd.read_csv(os.path.join(raw4_fold, file_name))
# total_err.info()

In [716]:
# # 과제4_전처리완료_20231221_PM_033316 : 등록&제원로 생성
# # 과제4_전처리완료_20240108_PM_100718 : STD_CEG_GRD_LST로 생성

# # 23.5s
# name = '과제4_전처리완료_20240108_PM_100718'
# file_name = f'{name}.csv'
# pr4 = pd.read_csv(os.path.join(raw4_fold, file_name))
# pr4.info()

In [717]:
pr4.columns

Index(['차량번호', '차대번호', '점검일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '일련번호', '중량',
       '엔진배기량', '제작일자', '최초등록일', '전문정비사업자코드', '정비여부', '시도코드', '시군구코드', '상세일자',
       '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드',
       '전문정비사업자_시군구코드', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정기정밀구분'],
      dtype='object')

In [718]:
pr4['정비내용'].unique()

array(['엔진계통', '연료장치', '각종센서', '흡기계통', '배출가스재순환장치', '배출가스전환장치', '배기계통',
       '점화계통', '연료증발가스방지장치', '블로바이가스환원장치', nan], dtype=object)

In [719]:
pr4['정비부품명'].unique()

array(['연료휠터', '엔진오일', '연료압력조절기', '냉각수온센서', '연료분사펌프(조속기조정,봉인)', '에어크리너',
       '엔진정비', '노즐(디젤인젝터포함)', 'EGR밸브', '기화기(믹서포함)',
       '정화용촉매(CatalyticConverter)', '소음기', '냉각계통', '배기관', 'EGR제어용서머밸브',
       '공연비(혼합비)', '점화시기(분사시기)조정', '점화플러그', '공회전속도', '디젤산화촉매(DOC)',
       '공기유량센서', '과급기', '산소센서', '점화배선', '드로틀바디',
       '정화조절밸브(PurgeControlValve)', '블로바이가스호스', '캐니스터및필터', '인젝터(휘발유)',
       '드로틀포지션센서', '배전기(Distributor)', '전자제어장치(ECU)', 'PCV밸브',
       '매연여과장치(DPF)', nan], dtype=object)

## 분석

In [720]:
pr4['연료'].value_counts(dropna=False)

연료
경유               3836747
NaN              1279351
휘발유               196628
LPG               134535
하이브리드(휘발유+전기)       4620
CNG                  125
기타연료                 112
하이브리드(LPG+전기)         39
Name: count, dtype: int64

In [721]:
total_err['차대번호'].isnull().sum()

7355

In [722]:
len(total_err['차량번호'].unique()), len(total_err['차대번호'].unique())

(101528, 101388)

In [723]:
# 2.9s
srt1 = total_err.groupby(['전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업소명', '차대번호'], dropna=False).agg({'차량번호':lambda x:len(x.unique()), '점검일자':'count'}).reset_index()
srt1 = srt1.rename(columns={'차량번호':'정비차량대수', '점검일자':'정비건수'})
srt1.head()

,전문정비사업자_시도명,전문정비사업자_시군구명,전문정비사업소명,차대번호,정비차량대수,정비건수
0,강원특별자치도,강릉시,유산모터스,KLAGA691DFB006781,1,3
1,강원특별자치도,강릉시,유산모터스,KLAGA691DFB010502,1,3
2,강원특별자치도,강릉시,유산모터스,KLAJA69YDBK150733,1,3
3,강원특별자치도,강릉시,유산모터스,KLALA69PD7B107512,1,3
4,강원특별자치도,강릉시,유산모터스,KLALA69PD8B400983,1,2


In [724]:
srt1.drop_duplicates(['전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업소명']).shape

(707, 6)

In [725]:
srt1[['정비차량대수', '정비건수']].sum()

정비차량대수    149674
정비건수      375556
dtype: int64

In [726]:
srt1.loc[srt1['정비차량대수'] != 1].head()

,전문정비사업자_시도명,전문정비사업자_시군구명,전문정비사업소명,차대번호,정비차량대수,정비건수
799,강원특별자치도,원주시,스피드메이트대현점,KMHFT41CP2A201909,2,6
1500,강원특별자치도,춘천시,진성종합정비공장,KMHWP81JP2U446629,2,2
1566,강원특별자치도,춘천시,진성종합정비공장,KPBLC3AC11P111272,2,2
5548,경기도,의정부시,애니카랜드 호원점,KMHMJ81VP4U237727,2,4
5551,경기도,의정부시,애니카랜드 호원점,KMHSD81VP2U343131,2,5


In [727]:
srt1.loc[(srt1['정비차량대수'] != 1) & (srt1['차대번호'].notnull())].head()

,전문정비사업자_시도명,전문정비사업자_시군구명,전문정비사업소명,차대번호,정비차량대수,정비건수
799,강원특별자치도,원주시,스피드메이트대현점,KMHFT41CP2A201909,2,6
1500,강원특별자치도,춘천시,진성종합정비공장,KMHWP81JP2U446629,2,2
1566,강원특별자치도,춘천시,진성종합정비공장,KPBLC3AC11P111272,2,2
5548,경기도,의정부시,애니카랜드 호원점,KMHMJ81VP4U237727,2,4
5551,경기도,의정부시,애니카랜드 호원점,KMHSD81VP2U343131,2,5


In [728]:
srt1.loc[(srt1['정비차량대수'] != 1) & (srt1['차대번호'].notnull()), '정비차량대수'].unique()

array([2, 4], dtype=int64)

In [729]:
# 차량번호 1개당 차대번호 여러개 존재하는지 확인
temp = total_err.groupby(['차량번호']).agg({'차대번호':lambda x:len(x.unique())}).reset_index()
print('차량번호 1개당 차대번호 여러개인 차량번호 개수 :', temp[temp['차대번호'] != 1].shape[0])

차량번호 1개당 차대번호 여러개인 차량번호 개수 : 3213


In [730]:
# 동일한 전문정비사업소명이 다른 지역에도 존재하는지 확인
temp = total_err.groupby(['전문정비사업소명']).agg({'전문정비사업자_시도명':lambda x:len(x.unique())}).reset_index()

# 지역별 동일전문정비사업소명이 2개 이상인 샘플
print('지역별 동일전문정비사업소명이 2개 이상인 전문정비사업소 수 :', temp[temp['전문정비사업자_시도명'] > 1].shape[0])
print()
print('지역별 동일전문정비사업소명이 2개 이상인 전문정비사업소 리스트')
print(temp.loc[temp['전문정비사업자_시도명'] > 1, ['전문정비사업소명', '전문정비사업자_시도명']])

지역별 동일전문정비사업소명이 2개 이상인 전문정비사업소 수 : 33

지역별 동일전문정비사업소명이 2개 이상인 전문정비사업소 리스트
           전문정비사업소명  전문정비사업자_시도명
11          (자)유승기업            2
31         (주)동신공업사            2
34         (주)동양공업사            2
44      (주)사능자동차공업사            2
77        (주)원진엠엔에스            2
87    (주)중산자동차정비검사소            2
99   (주)현대일산자동차정비센타            2
114         1급청솔모터스            2
124         e-디젤시스템            2
173           국일카센타            2
178       그린1급정비공업사            2
196      남부자동차공업(주)            2
210           대구부란자            2
235        대한자동차공업사            2
256      동일자동차공업(주)            2
263          동환종합정비            2
294      반야월자동차정비공장            2
314       빛나는자동차공업사            2
326        삼진자동차공업사            2
330       상신정비공업(주)            2
355       성서자동차검사정비            2
356      성우자동차공업(주)            2
358            성일디젤            2
391      신대우종합정비(주)            2
406    신흥자동차정비검사(주)            2
558        진영자동차공업사            2
581      충북1급자동차공업사            2
59

### 4.1.1 STD_BD41_1_ERR_RES_RGN_CURSTT
- 전문정비 사업자 지역별 현황
    - 지역별 사업자 수, 검사수, 검사차량대수

In [731]:
srt1.columns

Index(['전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업소명', '차대번호', '정비차량대수', '정비건수'], dtype='object')

In [732]:
grp1 = srt1.groupby('전문정비사업자_시도명', dropna=False).agg({'전문정비사업소명':lambda x:len(x.unique()), '정비차량대수':'sum', '정비건수':'sum'}).reset_index()
grp1 = grp1.rename(columns={'전문정비사업자_시도명':'시도', '전문정비사업소명':'전문정비사업소_수'})
# grp1.loc[grp1.shape[0], ['시도', '전문정비사업소_수', '정비차량대수', '정비건수']] = ['합계', grp1['전문정비사업소_수'].sum(), grp1['정비차량대수'].sum(), grp1['정비건수'].sum()]
grp1

,시도,전문정비사업소_수,정비차량대수,정비건수
0,강원특별자치도,14.00,1575.00,4381.00
1,경기도,55.00,4715.00,21735.00
2,경상남도,38.00,3087.00,8528.00
3,경상북도,45.00,5205.00,12248.00
4,광주광역시,16.00,1427.00,4545.00
5,대구광역시,39.00,6202.00,12611.00
6,대전광역시,22.00,3541.00,8321.00
7,부산광역시,17.00,2641.00,6691.00
8,서울특별시,76.00,8266.00,24463.00
9,세종특별자치시,3.00,392.00,1102.00


In [733]:
srt1[['정비차량대수', '정비건수']].sum()

정비차량대수    149674
정비건수      375556
dtype: int64

#### [출력] STD_BD41_1_ERR_RES_RGN_CURSTT

In [734]:
grp1.columns

Index(['시도', '전문정비사업소_수', '정비차량대수', '정비건수'], dtype='object')

In [736]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104122'

In [735]:
# grp1_dict = {
#     '전문정비사업자_시도명':'SPCY_OVHUL_BZMN_CTPV_NM', 
#     '전문정비사업소_수':'SPCY_OVHUL_BIZPLC_CNT', 
#     '정비차량대수':'OVHUL_VHCL_MKCNT', 
#     '정비건수':'OVHUL_NOCS', 
# }
# STD_BD41_1_ERR_RES_RGN_CURSTT = grp1.rename(columns=grp1_dict)
# STD_BD41_1_ERR_RES_RGN_CURSTT['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD41_1_ERR_RES_RGN_CURSTT.columns)

# #
# if export_opt == 'y':
#     STD_BD41_1_ERR_RES_RGN_CURSTT.to_csv(os.path.join(df4_fold, f'STD_BD41_1_ERR_RES_RGN_CURSTT_{today_date_dtl}.csv'), index=False)
#     print('STD_BD41_1_ERR_RES_RGN_CURSTT 출력 완료.')
# else:
#     pass

In [738]:
if export_opt == 'y':
    grp1.to_excel(os.path.join(df4_fold, f'STD_BD41_1_ERR_RES_RGN_CURSTT_{today_date_dtl}.xlsx'), index=False)
else:
    pass

### 4.1.2 STD_BD41_2_ERR_RES_FLFMT_CURSTT
- 전문정비 사업자 수행현황
    - 업자별 차량대수, 건수

In [739]:
srt1.head()

,전문정비사업자_시도명,전문정비사업자_시군구명,전문정비사업소명,차대번호,정비차량대수,정비건수
0,강원특별자치도,강릉시,유산모터스,KLAGA691DFB006781,1,3
1,강원특별자치도,강릉시,유산모터스,KLAGA691DFB010502,1,3
2,강원특별자치도,강릉시,유산모터스,KLAJA69YDBK150733,1,3
3,강원특별자치도,강릉시,유산모터스,KLALA69PD7B107512,1,3
4,강원특별자치도,강릉시,유산모터스,KLALA69PD8B400983,1,2


In [740]:
grp2_1 = srt1.groupby('전문정비사업소명', dropna=False).agg({'정비차량대수':'sum', '정비건수':'sum'}).reset_index()
grp2_1 = grp2_1.sort_values('정비건수', ascending=False).reset_index(drop=True)
# grp2_1.loc[grp2_1.shape[0], ['전문정비사업소명', '정비차량대수', '정비건수']] = ['합계', grp2_1['정비차량대수'].sum(), grp2_1['정비건수'].sum()]
grp2_1

,전문정비사업소명,정비차량대수,정비건수
0,NaN,51946.00,119733.00
1,만수카정비,295.00,10565.00
2,청원자동차정비,915.00,9058.00
3,그린1급정비공업사,680.00,7795.00
4,박병철자동차공업사,158.00,6882.00
...,...,...,...
670,한일자동차종합정비공업사,1.00,1.00
671,기아자동차(주)대구서비스센터,1.00,1.00
672,르노코리아자동차(주)대구사업소,1.00,1.00
673,(주)안일자동차써비스,1.00,1.00


In [741]:
grp2_2 = srt1.groupby('전문정비사업소명', dropna=False).agg({'정비차량대수':'sum', '정비건수':'sum'}).reset_index()
grp2_2 = grp2_2.sort_values('정비차량대수', ascending=False).reset_index(drop=True)
# grp2_2.loc[grp2_2.shape[0], ['전문정비사업소명', '정비차량대수', '정비건수']] = ['합계', grp2_2['정비차량대수'].sum(), grp2_2['정비건수'].sum()]
grp2_2

,전문정비사업소명,정비차량대수,정비건수
0,NaN,51946.00,119733.00
1,대구부란자,2039.00,2446.00
2,반야월자동차정비공장,1374.00,2638.00
3,카프자동차정비(주),1304.00,2146.00
4,구미디젤,1296.00,2883.00
...,...,...,...
670,SK스피드메이트상도점,1.00,3.00
671,이화모터스,1.00,2.00
672,현대자동차(주)수원서비스센터,1.00,5.00
673,성정점 현대자동차,1.00,7.00


In [742]:
srt1[['정비차량대수', '정비건수']].sum()

정비차량대수    149674
정비건수      375556
dtype: int64

#### [출력] STD_BD41_2_ERR_RES_FLFMT_CURSTT

In [743]:
grp2_1.columns, grp2_2.columns

(Index(['전문정비사업소명', '정비차량대수', '정비건수'], dtype='object'),
 Index(['전문정비사업소명', '정비차량대수', '정비건수'], dtype='object'))

In [745]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104123'

In [744]:
# grp2_dict = {
#     '전문정비사업소명':'SPCY_OVHUL_BIZPLC_NM', 
#     '정비차량대수':'OVHUL_VHCL_MKCNT', 
#     '정비건수':'OVHUL_NOCS', 
# }
# STD_BD41_2_ERR_RES_FLFMT_CURSTT = grp2_1.rename(columns=grp2_dict)
# STD_BD41_2_ERR_RES_FLFMT_CURSTT['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD41_2_ERR_RES_FLFMT_CURSTT.columns)

# #
# if export_opt == 'y':
#     STD_BD41_2_ERR_RES_FLFMT_CURSTT.to_excel(os.path.join(df4_fold, f'STD_BD41_2_ERR_RES_FLFMT_CURSTT_{today_date_dtl}.xlsx'), index=False)
#     print('STD_BD41_2_ERR_RES_FLFMT_CURSTT 출력 완료.')
# else:
#     pass

In [746]:
if export_opt == 'y':
    with pd.ExcelWriter(os.path.join(df4_fold, f'STD_BD41_2_ERR_RES_FLFMT_CURSTT_{today_date_dtl}.xlsx')) as writer:
        grp2_1.to_excel(writer, sheet_name='정비건수_순_정렬', index=False)
        grp2_2.to_excel(writer, sheet_name='정비차량대수_순_정렬', index=False)
else:
    pass

### [❌] 4.1.3 STD_BD41_3_BZMN_TKCG_VHCL
- 전문정비 사업자별 담당차량
    - 업자별 차량 대수, 건수
- 4.1.1, 4.1.2 와 동일한 내용

### [❌] 4.1.4 STD_BD41_4_INSP_FAIL_RT
- 제도현황 분석
    - 검사불합격율 : 차량별 검사판정

In [747]:
pr4.columns

Index(['차량번호', '차대번호', '점검일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '일련번호', '중량',
       '엔진배기량', '제작일자', '최초등록일', '전문정비사업자코드', '정비여부', '시도코드', '시군구코드', '상세일자',
       '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드',
       '전문정비사업자_시군구코드', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정기정밀구분'],
      dtype='object')

In [748]:
pr4[['차량번호', '차대번호', '점검일자', '정비접수번호']].isnull().sum()

차량번호            0
차대번호         7355
점검일자            0
정비접수번호    3992195
dtype: int64

In [749]:
pr4['검사판정'].value_counts(dropna=False)

검사판정
Y      3285497
N      2077378
NaN      66115
M        23167
Name: count, dtype: int64

In [750]:
pr4['검사판정'].value_counts(dropna=False, normalize=True)

검사판정
Y     0.60
N     0.38
NaN   0.01
M     0.00
Name: proportion, dtype: float64

In [751]:
# 검사판정 Y, N 만 활용
grp4 = pr4[(pr4['검사판정'] == 'Y') | (pr4['검사판정'] == 'N')].groupby(['전문정비사업자_시도명', '전문정비사업소명', '차대번호', '차량번호', '검사판정'], dropna=False).agg({'점검일자':'count'}).unstack('검사판정').reset_index()
grp4.head()

전문정비사업자_시도명 전문정비사업소명               차대번호       차량번호   점검일자       
검사판정                                                         N      Y
0        강원특별자치도    강원부란자       280134307541  충남81사1700  48.00 204.00
1        강원특별자치도    강원부란자  KLAGA691DFB067876    35러4547  28.00  21.00
2        강원특별자치도    강원부란자  KMFWBX7JBBU338779    87가8914  75.00 130.00
3        강원특별자치도    강원부란자  KMFXKD7BP3U649397    95더8598 153.00  90.00
4        강원특별자치도    강원부란자  KMFZCN7HP5U107229    80보3158  12.00  60.00

In [752]:
grp4.columns = ['전문정비사업자_시도명', '전문정비사업소명', '차대번호', '차량번호', '검사판정N', '검사판정Y']
grp4['불합격율'] = grp4['검사판정N'] / (grp4['검사판정Y'] + grp4['검사판정N'])
grp4['불합격율'] = grp4['불합격율'].round(2)
grp4.head()

,전문정비사업자_시도명,전문정비사업소명,차대번호,차량번호,검사판정N,검사판정Y,불합격율
0,강원특별자치도,강원부란자,280134307541,충남81사1700,48.00,204.00,0.19
1,강원특별자치도,강원부란자,KLAGA691DFB067876,35러4547,28.00,21.00,0.57
2,강원특별자치도,강원부란자,KMFWBX7JBBU338779,87가8914,75.00,130.00,0.37
3,강원특별자치도,강원부란자,KMFXKD7BP3U649397,95더8598,153.00,90.00,0.63
4,강원특별자치도,강원부란자,KMFZCN7HP5U107229,80보3158,12.00,60.00,0.17


In [753]:
pr4[['차대번호', '차량번호']].isnull().sum()

차대번호    7355
차량번호       0
dtype: int64

In [754]:
grp4.shape

(121949, 7)

In [755]:
srt1[['정비차량대수', '정비건수']].sum()

정비차량대수    149674
정비건수      375556
dtype: int64

#### [❌][출력] STD_BD41_4_INSP_FAIL_RT

In [756]:
grp4.columns

Index(['전문정비사업자_시도명', '전문정비사업소명', '차대번호', '차량번호', '검사판정N', '검사판정Y', '불합격율'], dtype='object')

In [757]:
grp4_1 = grp4[['전문정비사업자_시도명', '전문정비사업소명', '차대번호', '차량번호', '불합격율']]
# grp4_1_dict = {
#     '전문정비사업자_시도명':'', 
#     '전문정비사업소명':'', 
#     '차대번호':'', 
#     '차량번호':'', 
#     '불합격율':'', 
# }
# STD_BD41_4_INSP_FAIL_RT = grp4_1.rename(columns={grp4_1_dict})
# STD_BD41_4_INSP_FAIL_RT['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# STD_BD41_4_INSP_FAIL_RT.columns

In [758]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104126'

In [759]:
# # 6.5s
# if export_opt == 'y':
#     grp4_1.to_excel(os.path.join(df4_fold, f'STD_BD41_4_INSP_FAIL_RT_{today_date_dtl}.xlsx'), index=False)
# else:
#     pass

### [❌] 4.1.5 STD_BD41_5_FAIL_VHCL_CURSTT
- 제도현황 분석
    - 불합격 차량 현황 : 시도, 시군구, 연료, 등급

In [760]:
srt5 = pr4[pr4['검사판정'] == 'N'].reset_index(drop=True)
srt5.shape

(2077378, 70)

In [761]:
srt5['연료'].unique()

array([nan, '경유', 'LPG', '휘발유', '하이브리드(휘발유+전기)', 'CNG', '하이브리드(LPG+전기)',
       '기타연료'], dtype=object)

In [762]:
grp5 = srt5.groupby(['전문정비사업자_시도명', '전문정비사업자_시군구명', '연료', '배출가스등급', '차대번호'], dropna=False).agg({'차량번호':lambda x:len(x.unique())}).reset_index()
grp5 = grp5.rename(columns={'차량번호':'정비차량대수'})
grp5

,전문정비사업자_시도명,전문정비사업자_시군구명,연료,배출가스등급,차대번호,정비차량대수
0,강원특별자치도,강릉시,LPG,NaN,KLALA69PD7B107512,1
1,강원특별자치도,강릉시,LPG,NaN,KLALA69PD8B400983,1
2,강원특별자치도,강릉시,LPG,NaN,KLY2B11ZDCC145040,1
3,강원특별자치도,강릉시,LPG,NaN,KLY2B51EDDC045128,1
4,강원특별자치도,강릉시,LPG,NaN,KMHAC51RPYU252792,1
...,...,...,...,...,...,...
113411,NaN,NaN,NaN,NaN,YV2H2CEC4RA228355,1
113412,NaN,NaN,NaN,NaN,YV2H2CEC7SA238805,1
113413,NaN,NaN,NaN,NaN,YV2J4CFC04A576107,1
113414,NaN,NaN,NaN,NaN,YV2JN60C35A607472,1


In [763]:
grp5['정비차량대수'].sum()

113570

In [764]:
grp5_1 = grp5.groupby(['전문정비사업자_시도명', '전문정비사업자_시군구명', '연료', '배출가스등급'], dropna=False)['정비차량대수'].sum().reset_index()
grp5_1

,전문정비사업자_시도명,전문정비사업자_시군구명,연료,배출가스등급,정비차량대수
0,강원특별자치도,강릉시,LPG,NaN,11
1,강원특별자치도,강릉시,경유,NaN,333
2,강원특별자치도,강릉시,하이브리드(휘발유+전기),NaN,1
3,강원특별자치도,강릉시,휘발유,NaN,21
4,강원특별자치도,원주시,LPG,NaN,26
...,...,...,...,...,...
409,NaN,NaN,휘발유,3,34
410,NaN,NaN,휘발유,4,46
411,NaN,NaN,휘발유,5,2
412,NaN,NaN,휘발유,NaN,2964


In [765]:
grp5_1['배출가스등급'].unique()

array([nan, '2', '3', '4', '5', '9999'], dtype=object)

In [766]:
grp5_1['정비차량대수'].sum()

113570

#### [❌][출력] STD_BD41_5_FAIL_VHCL_CURSTT

In [767]:
grp5_1.columns

Index(['전문정비사업자_시도명', '전문정비사업자_시군구명', '연료', '배출가스등급', '정비차량대수'], dtype='object')

In [768]:
# grp5_1_dict = {
#     '전문정비사업자_시도명':'', 
#     '전문정비사업자_시군구명':'', 
#     '연료':'', 
#     '배출가스등급':'', 
#     '정비차량대수':'', 
# }
# STD_BD41_5_FAIL_VHCL_CURSTT = grp5_1.rename(columns={grp5_1_dict})
# STD_BD41_5_FAIL_VHCL_CURSTT['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# STD_BD41_5_FAIL_VHCL_CURSTT.columns

In [769]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104131'

In [770]:
# # 6.5s
# if export_opt == 'y':
#     grp5_1.to_excel(os.path.join(df4_fold, f'STD_BD41_5_FAIL_VHCL_CURSTT_{today_date_dtl}.xlsx'), index=False)
# else:
#     pass

### 4.1.6 STD_BD41_6_FAIL_RESN

#### 검사판정 N df 추출(srt6)

In [771]:
grp6 = total_err.groupby(['정비내용'], dropna=False)['차량번호'].count().reset_index()
grp6 = grp6.rename(columns={'차량번호':'정비건수'})
# grp6.loc[grp6.shape[0], ['정비내용', '정비건수']] = ['합계', grp6['정비건수'].sum()]
grp6

,정비내용,정비건수
0,각종센서,21322.00
1,배기계통,35080.00
2,배출가스재순환장치,28877.00
3,배출가스전환장치,15991.00
4,블로바이가스환원장치,3780.00
5,엔진계통,129347.00
6,연료장치,68921.00
7,연료증발가스방지장치,2361.00
8,점화계통,15720.00
9,흡기계통,51518.00


#### [출력] STD_BD41_6_FAIL_RESN

In [772]:
grp6.columns

Index(['정비내용', '정비건수'], dtype='object')

In [774]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104131'

In [773]:
# grp6_dict = {
#     '정비내용':'OVHUL_CN', 
#     '정비건수':'OVHUL_NOCS',
# }
# STD_BD41_6_FAIL_RESN = grp6.rename(columns={grp6_dict})
# STD_BD41_6_FAIL_RESN['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD41_6_FAIL_RESN.columns)

# if export_opt == 'y':
#     STD_BD41_6_FAIL_RESN.to_excel(os.path.join(df4_fold, f'STD_BD41_6_FAIL_RESN_{today_date_dtl}.xlsx'), index=False)
#     print('STD_BD41_6_FAIL_RESN 출력 완료.')
# else:
#     pass

In [775]:
# 0.0s
if export_opt == 'y':
    grp6.to_excel(os.path.join(df4_fold, f'STD_BD41_6_FAIL_RESN_{today_date_dtl}.xlsx'), index=False)
else:
    pass

### 4.1.7 STD_BD41_7_OVHUL_ITM

In [776]:
grp7 = total_err.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
grp7 = grp7.rename(columns={'차량번호':'정비건수'})
grp7 = grp7.sort_values('정비건수', ascending=False).reset_index(drop=True)
# grp7.loc[grp7.shape[0], ['정비내용', '정비건수']] = ['합계', grp7['정비건수'].sum()]
grp7

,정비내용,정비부품명,정비건수
0,엔진계통,"연료분사펌프(조속기조정,봉인)",60649.00
1,흡기계통,에어크리너,40113.00
2,연료장치,엔진오일,34366.00
3,엔진계통,연료휠터,29346.00
4,배출가스재순환장치,EGR밸브,24565.00
5,엔진계통,노즐(디젤인젝터포함),22924.00
6,배기계통,소음기,19755.00
7,배기계통,배기관,15325.00
8,연료장치,엔진정비,14597.00
9,연료장치,냉각계통,12880.00


#### [출력] STD_BD41_7_OVHUL_ITM

In [777]:
grp7.columns

Index(['정비내용', '정비부품명', '정비건수'], dtype='object')

In [779]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104131'

In [778]:
# grp7_dict = {
#     '정비내용':'OVHUL_CN', 
#     '정비부품명':'OVHUL_CMPNT_NM', 
#     '정비건수':'OVHUL_NOCS', 
# }
# STD_BD41_7_OVHUL_ITM = grp7.rename(columns={grp7_dict})
# STD_BD41_7_OVHUL_ITM['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD41_7_OVHUL_ITM.columns)

# #
# if export_opt == 'y':
#     STD_BD41_7_OVHUL_ITM.to_excel(os.path.join(df4_fold, f'STD_BD41_7_OVHUL_ITM_{today_date_dtl}.xlsx'), index=False)
#     print('STD_BD41_7_OVHUL_ITM 출력 완료.')
# else:
#     pass

In [780]:
# 0.0s
if export_opt == 'y':
    grp7.to_excel(os.path.join(df4_fold, f'STD_BD41_7_OVHUL_ITM_{today_date_dtl}.xlsx'), index=False)
else:
    pass

### [🔺] 4.2.8 STD_BD42_8_OVHUL_ITM
- total_err과 가장 최근 검사 기준 차대번호 중복 제거한 ins 병합

In [781]:
ins.shape

(173740086, 25)

In [782]:
ins.columns

Index(['차대번호', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정기정밀구분'],
      dtype='object')

In [783]:
ins['검사일자'].dtype

dtype('int64')

In [784]:
ins['검사일자'].isnull().sum()

0

#### 정기/정밀 검사 차대번호 중복 제거

In [785]:
# 3m 41.6s
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
insm.shape

(32067437, 25)

In [786]:
total_err.shape

(375556, 46)

In [787]:
len(set(total_err['차대번호'].to_list()) - set(insm['차대번호'].to_list()))

15427

In [788]:
srt8 = total_err.merge(insm, on='차대번호', how='left')
srt8.shape

(375556, 70)

In [789]:
srt8[['무부하매연판정1', '무부하매연판정2', '무부하매연판정3']].isnull().sum()

무부하매연판정1    66115
무부하매연판정2    66115
무부하매연판정3    66115
dtype: int64

In [790]:
srt8['연료'].unique()

array(['경유', nan, 'LPG', '휘발유', '하이브리드(휘발유+전기)', 'CNG', '하이브리드(LPG+전기)',
       '기타연료'], dtype=object)

In [791]:
srt8['연료'] = srt8['연료'].replace({'LPG(액화석유가스)':'LPG'})
srt8['연료'].unique()

array(['경유', nan, 'LPG', '휘발유', '하이브리드(휘발유+전기)', 'CNG', '하이브리드(LPG+전기)',
       '기타연료'], dtype=object)

In [792]:
srt8_jt1n = srt8[(srt8['무부하매연판정1'] == 'N') & (srt8['연료'] == '경유')].reset_index(drop=True)
srt8_jt2n = srt8[(srt8['무부하매연판정1'] == 'N') & ( (srt8['연료'] == '휘발유') | (srt8['연료'] == 'LPG'))].reset_index(drop=True)
srt8_jt3n = srt8[(srt8['무부하매연판정2'] == 'N') & ( (srt8['연료'] == '휘발유') | (srt8['연료'] == 'LPG'))].reset_index(drop=True)
srt8_jt4n = srt8[(srt8['무부하매연판정3'] == 'N') & ( (srt8['연료'] == '휘발유') | (srt8['연료'] == 'LPG'))].reset_index(drop=True)
srt8_jt1n.shape, srt8_jt2n.shape, srt8_jt3n.shape, srt8_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [793]:
print(srt8_jt1n['연료'].unique())
print(srt8_jt2n['연료'].unique())
print(srt8_jt3n['연료'].unique())
print(srt8_jt4n['연료'].unique())

['경유']
['휘발유' 'LPG']
['휘발유' 'LPG']
['LPG' '휘발유']


In [794]:
# grp8_1 = srt8_jt1n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp8_1 = grp8_1.rename(columns={'차량번호':'정비건수'})
# grp8_1.loc[grp8_1.shape[0], ['정비내용', '정비건수']] = ['합계', grp8_1['정비건수'].sum()]
# display(grp8_1.tail())
# print()

# grp8_2 = srt8_jt2n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp8_2 = grp8_2.rename(columns={'차량번호':'정비건수'})
# grp8_2.loc[grp8_2.shape[0], ['정비내용', '정비건수']] = ['합계', grp8_2['정비건수'].sum()]
# display(grp8_2.tail())
# print()

# grp8_3 = srt8_jt3n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp8_3 = grp8_3.rename(columns={'차량번호':'정비건수'})
# grp8_3.loc[grp8_3.shape[0], ['정비내용', '정비건수']] = ['합계', grp8_3['정비건수'].sum()]
# display(grp8_3.tail())
# print()

# grp8_4 = srt8_jt4n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp8_4 = grp8_4.rename(columns={'차량번호':'정비건수'})
# grp8_4.loc[grp8_4.shape[0], ['정비내용', '정비건수']] = ['합계', grp8_4['정비건수'].sum()]
# display(grp8_4.tail())
# print()

In [904]:
def func8_grp(df):
    grp = df.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
    grp = grp.rename(columns={'차량번호':'정비건수'})
    grp = grp.sort_values('정비건수', ascending=False).reset_index(drop=True)
    # grp.loc[grp.shape[0], ['정비내용', '정비건수']] = ['합계', grp['정비건수'].sum()]
    return grp

In [905]:
for i, one in enumerate([srt8_jt1n, srt8_jt2n, srt8_jt3n, srt8_jt4n]):
    globals()[f'grp8_{i+1}'] = func8_grp(one)

In [906]:
grp8_1['불합격내역'] = '매연'
grp8_2['불합격내역'] = 'CO'
grp8_3['불합격내역'] = 'HC'
grp8_4['불합격내역'] = 'NOx'

In [907]:
grp8 = pd.concat([grp8_1, grp8_2, grp8_3, grp8_4], ignore_index=True)
grp8.tail()

,정비내용,정비부품명,정비건수,불합격내역
135,배출가스전환장치,매연여과장치(DPF),11,NOx
136,엔진계통,"연료분사펌프(조속기조정,봉인)",10,NOx
137,엔진계통,노즐(디젤인젝터포함),10,NOx
138,흡기계통,과급기,10,NOx
139,연료증발가스방지장치,정화조절밸브(PurgeControlValve),8,NOx


In [908]:
grp8.columns

Index(['정비내용', '정비부품명', '정비건수', '불합격내역'], dtype='object')

#### [출력] STD_BD42_8_OVHUL_ITM 

In [798]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104504'

In [ ]:
# STD_BD42_8_OVHUL_ITM = grp8[[
#     '정비내용',
#     '정비부품명',
#     '정비건수',
#     '불합격내역',
# ]]
# grp8_dict = {
#     '정비내용':'OVHUL_CN',
#     '정비부품명':'OVHUL_CMPNT_NM',
#     '정비건수':'OVHUL_NOCS',
#     '불합격내역':'FAIL_BRKWN',
# }
# STD_BD42_8_OVHUL_ITM = STD_BD42_8_OVHUL_ITM.rename(columns=grp8_dict)
# STD_BD42_8_OVHUL_ITM['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD42_8_OVHUL_ITM.columns)

# if export_opt == 'y':
#     STD_BD42_8_OVHUL_ITM.to_csv(os.path.join(df4_fold, f'STD_BD42_8_OVHUL_ITM_{today_date_dtl}.csv'), index=False)
#     print('STD_BD42_8_OVHUL_ITM 출력 완료.')
# else:
#     pass

In [799]:
if export_opt == 'y':
    with pd.ExcelWriter(os.path.join(df4_fold, f'STD_BD42_8_OVHUL_ITM_{today_date_dtl}.xlsx')) as writer:
        grp8_1.to_excel(writer, sheet_name='판정1_매연', index=False)
        grp8_2.to_excel(writer, sheet_name='판정1_CO', index=False)
        grp8_3.to_excel(writer, sheet_name='판정2_HC', index=False)
        grp8_4.to_excel(writer, sheet_name='판정3_NOx', index=False)
else:
    pass

### [🔺] 4.2.9 STD_BD42_9_OVHUL_CL

In [800]:
srt8_jt1n.shape, srt8_jt2n.shape, srt8_jt3n.shape, srt8_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [801]:
srt9_jt1n = srt8_jt1n.copy()
srt9_jt2n = srt8_jt2n.copy()
srt9_jt3n = srt8_jt3n.copy()
srt9_jt4n = srt8_jt4n.copy()
srt9_jt1n.shape, srt9_jt2n.shape, srt9_jt3n.shape, srt9_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [802]:
srt9_jt1n = srt9_jt1n[srt9_jt1n['제작사명'].notnull()].reset_index(drop=True)
srt9_jt2n = srt9_jt2n[srt9_jt2n['제작사명'].notnull()].reset_index(drop=True)
srt9_jt3n = srt9_jt3n[srt9_jt3n['제작사명'].notnull()].reset_index(drop=True)
srt9_jt4n = srt9_jt4n[srt9_jt4n['제작사명'].notnull()].reset_index(drop=True)
srt9_jt1n.shape, srt9_jt2n.shape, srt9_jt3n.shape, srt9_jt4n.shape

((327, 70), (12, 70), (3, 70), (13, 70))

In [803]:
# # srt9_jt1n : 무부하매연판정1 == N
# grp9_1 = srt9_jt1n.groupby(['연료', '차종', '제작사명'], dropna=False)['차량번호'].count().reset_index()
# grp9_1 = grp9_1.rename(columns={'차량번호':'정비건수'})
# grp9_1.loc[grp9_1.shape[0], ['연료', '정비건수']] = ['합계', grp9_1['정비건수'].sum()]
# display(grp9_1.tail())
# print()

# # srt9_jt2n : 무부하매연판정1 == N
# grp9_2 = srt9_jt2n.groupby(['연료', '차종', '제작사명'], dropna=False)['차량번호'].count().reset_index()
# grp9_2 = grp9_2.rename(columns={'차량번호':'정비건수'})
# grp9_2.loc[grp9_2.shape[0], ['연료', '정비건수']] = ['합계', grp9_2['정비건수'].sum()]
# display(grp9_2.tail())
# print()

# # srt9_jt3n : 무부하매연판정1 == N
# grp9_3 = srt9_jt3n.groupby(['연료', '차종', '제작사명'], dropna=False)['차량번호'].count().reset_index()
# grp9_3 = grp9_3.rename(columns={'차량번호':'정비건수'})
# grp9_3.loc[grp9_3.shape[0], ['연료', '정비건수']] = ['합계', grp9_3['정비건수'].sum()]
# display(grp9_3.tail())
# print()

# # srt9_jt4n : 무부하매연판정1 == N
# grp9_4 = srt9_jt4n.groupby(['연료', '차종', '제작사명'], dropna=False)['차량번호'].count().reset_index()
# grp9_4 = grp9_4.rename(columns={'차량번호':'정비건수'})
# grp9_4.loc[grp9_4.shape[0], ['연료', '정비건수']] = ['합계', grp9_4['정비건수'].sum()]
# display(grp9_4.tail())
# print()

In [911]:
def func9_grp(df):
    grp = df.groupby(['연료', '차종', '제작사명'], dropna=False)['차량번호'].count().reset_index()
    grp = grp.rename(columns={'차량번호':'정비건수'})
    grp = grp.sort_values('정비건수', ascending=False).reset_index(drop=True)
    # grp.loc[grp.shape[0], ['연료', '정비건수']] = ['합계', grp['정비건수'].sum()]
    return grp

In [912]:
for i, one in enumerate([srt9_jt1n, srt9_jt2n, srt9_jt3n, srt9_jt4n]):
    globals()[f'grp9_{i+1}'] = func9_grp(one)

In [913]:
grp9_1['불합격내역'] = '매연'
grp9_2['불합격내역'] = 'CO'
grp9_3['불합격내역'] = 'HC'
grp9_4['불합격내역'] = 'NOx'

In [914]:
grp9 = pd.concat([grp9_1, grp9_2, grp9_3, grp9_4], ignore_index=True)
grp9.tail()

,연료,차종,제작사명,정비건수,불합격내역
19,휘발유,승용,BENZ,1,CO
20,휘발유,승용,현대자동차(주),2,HC
21,휘발유,승용,BENZ,1,HC
22,휘발유,승용,현대자동차(주),8,NOx
23,LPG,승용,현대자동차(주),5,NOx


In [915]:
grp9.columns

Index(['연료', '차종', '제작사명', '정비건수', '불합격내역'], dtype='object')

#### [출력] STD_BD42_9_OVHUL_CL

In [806]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104504'

In [ ]:
# STD_BD42_9_OVHUL_CL = grp9[[
#     '연료',
#     '차종',
#     '제작사명',
#     '정비건수',
#     '불합격내역',
# ]]
# grp9_dict = {
#     '연료':'FUEL',
#     '차종':'VHCTY',
#     '제작사명':'MNFCTR_NM',
#     '정비건수':'OVHUL_NOCS',
#     '불합격내역':'FAIL_BRKWN',
# }
# STD_BD42_9_OVHUL_CL = STD_BD42_9_OVHUL_CL.rename(columns=grp9_dict)
# STD_BD42_9_OVHUL_CL['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD42_9_OVHUL_CL.columns)

# if export_opt == 'y':
#     STD_BD42_9_OVHUL_CL.to_csv(os.path.join(df4_fold, f'STD_BD42_9_OVHUL_CL_{today_date_dtl}.csv'), index=False)
#     print('STD_BD42_9_OVHUL_CL 출력 완료.')
# else:
#     pass

In [807]:
if export_opt == 'y':
    with pd.ExcelWriter(os.path.join(df4_fold, f'STD_BD42_9_OVHUL_CL_{today_date_dtl}.xlsx')) as writer:
        grp9_1.to_excel(writer, sheet_name='판정1_매연', index=False)
        grp9_2.to_excel(writer, sheet_name='판정1_CO', index=False)
        grp9_3.to_excel(writer, sheet_name='판정2_HC', index=False)
        grp9_4.to_excel(writer, sheet_name='판정3_NOx', index=False)
else:
    pass

### [❌] 4.2.10 STD_BD42_10_EXHST_GAS_EXCS

### [❌] 4.2.11 STD_BD42_11_BRKWN_CHRC

### 4.2.12 STD_BD42_12_RPR_IMPV

In [808]:
pr4.shape

(5452157, 70)

In [809]:
pr4[['점검일자', '검사일자']].dtypes

점검일자      int64
검사일자    float64
dtype: object

In [810]:
pr4[['점검일자', '검사일자']].isnull().sum()

점검일자        0
검사일자    66115
dtype: int64

In [811]:
# srt12_before : 점검일자 전의 검사 기록(수리 전 검사기록)
# srt12_after : 점검일자 후 또는 같을 때의 검사 기록(수리 후 검사기록)
srt12_before = pr4[pr4['점검일자'] > pr4['검사일자']].reset_index(drop=True)
srt12_after = pr4[pr4['점검일자'] <= pr4['검사일자']].reset_index(drop=True)
srt12_before.shape, srt12_after.shape

((2408596, 70), (2977446, 70))

In [812]:
pr4[(pr4['점검일자'].notnull()) & (pr4['검사일자'].notnull())].shape

(5386042, 70)

In [813]:
srt12_before.shape[0] + srt12_after.shape[0] == pr4[(pr4['점검일자'].notnull()) & (pr4['검사일자'].notnull())].shape[0]

True

In [814]:
grp12_1 = srt12_before.groupby(['연료', '정비내용', '정비부품명'], dropna=False).agg({'무부하매연측정치1':'mean', '무부하매연측정치2':'mean', '무부하매연측정치3':'mean'}).reset_index()
grp12_1[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']] = grp12_1[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].round(2)
grp12_1 = grp12_1.rename(columns={'무부하매연측정치1':'무부하매연측정치1_전', '무부하매연측정치2':'무부하매연측정치2_전', '무부하매연측정치3':'무부하매연측정치3_전'})
grp12_1.head()

,연료,정비내용,정비부품명,무부하매연측정치1_전,무부하매연측정치2_전,무부하매연측정치3_전
0,CNG,각종센서,산소센서,0.00,28.00,736.00
1,CNG,엔진계통,연료휠터,0.00,28.00,736.00
2,CNG,점화계통,점화플러그,0.00,28.00,736.00
3,CNG,흡기계통,드로틀바디,0.00,24.00,1908.00
4,CNG,흡기계통,에어크리너,0.00,24.00,1908.00


In [815]:
grp12_2 = srt12_after.groupby(['연료', '정비내용', '정비부품명'], dropna=False).agg({'무부하매연측정치1':'mean', '무부하매연측정치2':'mean', '무부하매연측정치3':'mean'}).reset_index()
grp12_2[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']] = grp12_2[['무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3']].round(2)
grp12_2 = grp12_2.rename(columns={'무부하매연측정치1':'무부하매연측정치1_후', '무부하매연측정치2':'무부하매연측정치2_후', '무부하매연측정치3':'무부하매연측정치3_후'})
grp12_2.head()

,연료,정비내용,정비부품명,무부하매연측정치1_후,무부하매연측정치2_후,무부하매연측정치3_후
0,CNG,각종센서,산소센서,0.31,240.35,204.89
1,CNG,배기계통,소음기,0.21,193.61,106.00
2,CNG,엔진계통,연료휠터,0.00,12.50,1964.00
3,CNG,점화계통,점화플러그,0.00,12.50,1964.00
4,CNG,흡기계통,드로틀바디,0.00,37.40,105.80


In [816]:
grp12_3 = grp12_1.merge(grp12_2, on=['연료', '정비내용', '정비부품명'])
grp12_3.head()

,연료,정비내용,정비부품명,무부하매연측정치1_전,무부하매연측정치2_전,무부하매연측정치3_전,무부하매연측정치1_후,무부하매연측정치2_후,무부하매연측정치3_후
0,CNG,각종센서,산소센서,0.00,28.00,736.00,0.31,240.35,204.89
1,CNG,엔진계통,연료휠터,0.00,28.00,736.00,0.00,12.50,1964.00
2,CNG,점화계통,점화플러그,0.00,28.00,736.00,0.00,12.50,1964.00
3,CNG,흡기계통,드로틀바디,0.00,24.00,1908.00,0.00,37.40,105.80
4,CNG,흡기계통,에어크리너,0.00,24.00,1908.00,0.18,173.05,105.98


In [817]:
grp12_3[grp12_3['무부하매연측정치1_전'] < 0].shape

(0, 9)

In [818]:
grp12_3[grp12_3['무부하매연측정치1_후'] < 0].shape

(0, 9)

In [819]:
grp12_3[grp12_3['무부하매연측정치2_전'] < 0].shape

(0, 9)

In [820]:
grp12_3[grp12_3['무부하매연측정치2_후'] < 0].shape

(0, 9)

In [821]:
grp12_3[grp12_3['무부하매연측정치3_전'] < 0].shape

(0, 9)

In [822]:
grp12_3[grp12_3['무부하매연측정치3_후'] < 0].shape

(0, 9)

#### [출력] STD_BD42_12_RPR_IMPV

In [823]:
grp12_3.columns

Index(['연료', '정비내용', '정비부품명', '무부하매연측정치1_전', '무부하매연측정치2_전', '무부하매연측정치3_전',
       '무부하매연측정치1_후', '무부하매연측정치2_후', '무부하매연측정치3_후'],
      dtype='object')

In [824]:
grp12_3 = grp12_3[[
    '연료', 
    '정비내용',
    '정비부품명',
    '무부하매연측정치1_전',
    '무부하매연측정치1_후',
    '무부하매연측정치2_전',
    '무부하매연측정치2_후',
    '무부하매연측정치3_전',
    '무부하매연측정치3_후',
]]

In [825]:
# grp12_dict = {
#     '연료':'FUEL',
#     '정비내용':'OVHUL_CN', 
#     '정비부품명':'OVHUL_CMPNT_NM', 
#     '무부하매연측정치1_전':'NOLOD_SMO_MEVLU1_BF',
#     '무부하매연측정치1_후':'NOLOD_SMO_MEVLU1_AFTR',
#     '무부하매연측정치2_전':'NOLOD_SMO_MEVLU2_BF',
#     '무부하매연측정치2_후':'NOLOD_SMO_MEVLU2_AFTR',
#     '무부하매연측정치3_전':'NOLOD_SMO_MEVLU3_BF',
#     '무부하매연측정치3_후':'NOLOD_SMO_MEVLU3_AFTR',
# }
# STD_BD42_12_RPR_IMPV = grp12_3.rename(columns={grp12_dict})
# STD_BD42_12_RPR_IMPV['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD42_12_RPR_IMPV.columns)

# if export_opt == 'y':
#     STD_BD42_12_RPR_IMPV.to_csv(os.path.join(df4_fold, f'STD_BD42_12_RPR_IMPV_{today_date_dtl}.csv'), index=False)
#     print('STD_BD42_12_RPR_IMPV 출력 완료.')
# else:
#     pass

In [826]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104512'

In [827]:
# 0.0s
if export_opt == 'y':
    grp12_3.to_excel(os.path.join(df4_fold, f'STD_BD42_12_RPR_IMPV_{today_date_dtl}.xlsx'), index=False)
else:
    pass

### [❌] 4.2.13 STD_BD42_13_ITM_EXCS_RT
- 정비 전후 비교
    - (수정사항) 연료별 전후 비교 : N

In [828]:
srt12_before.shape, srt12_after.shape

((2408596, 70), (2977446, 70))

In [829]:
srt13_before = srt12_before.copy()
srt13_after = srt12_after.copy()

In [830]:
# 동일항목 초과율 계산식
srt13_before['초과율1'] = srt13_before['무부하매연측정치1'] / srt13_before['무부하매연허용치1'] - 1
srt13_before['초과율2'] = srt13_before['무부하매연측정치2'] / srt13_before['무부하매연허용치2'] - 1
srt13_before['초과율3'] = srt13_before['무부하매연측정치3'] / srt13_before['무부하매연허용치3'] - 1
srt13_before[['무부하매연측정치1', '무부하매연허용치1', '초과율1', 
              '무부하매연측정치2', '무부하매연허용치2', '초과율2', 
              '무부하매연측정치3', '무부하매연허용치3', '초과율3']].head()

,무부하매연측정치1,무부하매연허용치1,초과율1,무부하매연측정치2,무부하매연허용치2,초과율2,무부하매연측정치3,무부하매연허용치3,초과율3
0,60.00,60.00,0.00,43.00,60.00,-0.28,58.00,60.00,-0.03
1,29.00,40.00,-0.28,26.00,40.00,-0.35,59.00,40.00,0.48
2,20.00,40.00,-0.50,21.00,40.00,-0.47,40.00,40.00,0.00
3,27.00,35.00,-0.23,21.00,35.00,-0.40,34.00,35.00,-0.03
4,60.00,60.00,0.00,43.00,60.00,-0.28,58.00,60.00,-0.03


In [831]:
# srt13_before.loc[srt13_before['초과율1'] == np.inf, ['무부하매연측정치1', '무부하매연허용치1', '초과율1']] # 10
# srt13_before.loc[srt13_before['초과율2'] == np.inf, ['무부하매연측정치2', '무부하매연허용치2', '초과율2']] # 10216
# srt13_before.loc[srt13_before['초과율3'] == np.inf, ['무부하매연측정치3', '무부하매연허용치3', '초과율3']] # 178

In [832]:
# 허용치가 0인 값의 초과율은 빈 값으로 처리(초과율 inf -> np.nan)
srt13_before.loc[srt13_before['초과율1'] == np.inf, '초과율1'] = np.nan
srt13_before.loc[srt13_before['초과율2'] == np.inf, '초과율2'] = np.nan
srt13_before.loc[srt13_before['초과율3'] == np.inf, '초과율3'] = np.nan

In [833]:
srt13_before[['초과율1', '초과율2', '초과율3']].mean()

초과율1   -0.08
초과율2   -0.32
초과율3   -0.04
dtype: float64

In [834]:
# 동일항목 초과율 계산식
srt13_after['초과율1'] = srt13_after['무부하매연측정치1'] / srt13_after['무부하매연허용치1'] - 1
srt13_after['초과율2'] = srt13_after['무부하매연측정치2'] / srt13_after['무부하매연허용치2'] - 1
srt13_after['초과율3'] = srt13_after['무부하매연측정치3'] / srt13_after['무부하매연허용치3'] - 1
srt13_after[['무부하매연측정치1', '무부하매연허용치1', '초과율1', 
              '무부하매연측정치2', '무부하매연허용치2', '초과율2', 
              '무부하매연측정치3', '무부하매연허용치3', '초과율3']].head()

,무부하매연측정치1,무부하매연허용치1,초과율1,무부하매연측정치2,무부하매연허용치2,초과율2,무부하매연측정치3,무부하매연허용치3,초과율3
0,24.00,35.00,-0.31,24.00,35.00,-0.31,64.00,35.00,0.83
1,23.00,35.00,-0.34,21.00,35.00,-0.40,50.00,35.00,0.43
2,17.00,35.00,-0.51,11.00,35.00,-0.69,12.00,35.00,-0.66
3,31.00,35.00,-0.11,29.00,35.00,-0.17,69.00,35.00,0.97
4,24.00,35.00,-0.31,17.00,35.00,-0.51,22.00,35.00,-0.37


In [835]:
# 허용치가 0인 값의 초과율은 빈 값으로 처리(초과율 inf -> np.nan)
srt13_after.loc[srt13_after['초과율1'] == np.inf, '초과율1'] = np.nan
srt13_after.loc[srt13_after['초과율2'] == np.inf, '초과율2'] = np.nan
srt13_after.loc[srt13_after['초과율3'] == np.inf, '초과율3'] = np.nan

In [836]:
srt13_after[['초과율1', '초과율2', '초과율3']].mean()

초과율1   -0.16
초과율2   -0.30
초과율3   -0.19
dtype: float64

In [837]:
grp13_1 = srt13_before.groupby(['정비내용', '정비부품명'], dropna=False).agg({'초과율1':'mean', '초과율2':'mean', '초과율3':'mean'}).reset_index()
grp13_1[['초과율1', '초과율2', '초과율3']] = grp13_1[['초과율1', '초과율2', '초과율3']].round(2)
grp13_1 = grp13_1.rename(columns={'초과율1':'초과율1_전', '초과율2':'초과율2_전', '초과율3':'초과율3_전'})
grp13_1.head()

,정비내용,정비부품명,초과율1_전,초과율2_전,초과율3_전
0,각종센서,공기유량센서,0.00,-0.45,0.34
1,각종센서,냉각수온센서,-0.06,-0.37,0.42
2,각종센서,드로틀포지션센서,-0.09,-0.50,0.97
3,각종센서,산소센서,-0.18,-0.55,0.98
4,배기계통,배기관,-0.07,-0.35,-0.15


In [838]:
grp13_2 = srt13_after.groupby(['정비내용', '정비부품명'], dropna=False).agg({'초과율1':'mean', '초과율2':'mean', '초과율3':'mean'}).reset_index()
grp13_2[['초과율1', '초과율2', '초과율3']] = grp13_2[['초과율1', '초과율2', '초과율3']].round(2)
grp13_2 = grp13_2.rename(columns={'초과율1':'초과율1_후', '초과율2':'초과율2_후', '초과율3':'초과율3_후'})
grp13_2.head()

,정비내용,정비부품명,초과율1_후,초과율2_후,초과율3_후
0,각종센서,공기유량센서,-0.06,-0.46,-0.34
1,각종센서,냉각수온센서,-0.18,-0.37,-0.19
2,각종센서,드로틀포지션센서,-0.08,-0.42,-0.16
3,각종센서,산소센서,-0.28,-0.58,0.12
4,배기계통,배기관,-0.09,-0.30,-0.26


In [839]:
grp13_3 = grp13_1.merge(grp13_2, on=['정비내용', '정비부품명'])
grp13_3.head()

,정비내용,정비부품명,초과율1_전,초과율2_전,초과율3_전,초과율1_후,초과율2_후,초과율3_후
0,각종센서,공기유량센서,0.00,-0.45,0.34,-0.06,-0.46,-0.34
1,각종센서,냉각수온센서,-0.06,-0.37,0.42,-0.18,-0.37,-0.19
2,각종센서,드로틀포지션센서,-0.09,-0.50,0.97,-0.08,-0.42,-0.16
3,각종센서,산소센서,-0.18,-0.55,0.98,-0.28,-0.58,0.12
4,배기계통,배기관,-0.07,-0.35,-0.15,-0.09,-0.30,-0.26


#### [❌][출력] STD_BD42_13_ITM_EXCS_RT

In [840]:
grp13_3.columns

Index(['정비내용', '정비부품명', '초과율1_전', '초과율2_전', '초과율3_전', '초과율1_후', '초과율2_후',
       '초과율3_후'],
      dtype='object')

In [841]:
grp13_3 = grp13_3[[
    '정비내용',
    '정비부품명',
    '초과율1_전',
    '초과율1_후',
    '초과율2_전',
    '초과율2_후',
    '초과율3_전',
    '초과율3_후',
]]

In [842]:
# grp13_dict = {
#     '정비내용':'',
#     '정비부품명':'',
#     '초과율1_전':'',
#     '초과율1_후':'',
#     '초과율2_전':'',
#     '초과율2_후':'',
#     '초과율3_전':'',
#     '초과율3_후':'',
# }
# STD_BD42_13_ITM_EXCS_RT = grp13_3.rename(columns={grp13_dict})
# STD_BD42_13_ITM_EXCS_RT['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# STD_BD42_13_ITM_EXCS_RT.columns

In [843]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104515'

In [844]:
# # 0.0s
# if export_opt == 'y':
#     grp13_3.to_excel(os.path.join(df4_fold, f'STD_BD42_13_ITM_EXCS_RT_{today_date_dtl}.xlsx'), index=False)
# else:
#     pass

### [🔺] 4.2.14 STD_BD42_14_RE_BRKWN_RT
- 문제
    - 재고장난 차량 대수는 차대번호 개수, 전체 차량 대수는 전체 데이터 수 or 전체 차대번호 수 ?

In [845]:
total_err.shape, len(total_err['차대번호'].unique())

((375556, 46), 101388)

In [846]:
dup_total_err = total_err[total_err['차대번호'].duplicated(keep=False)].drop_duplicates('차대번호').reset_index(drop=True)
dup_total_err.shape

(76191, 46)

In [847]:
# 
srt14 = dup_total_err.merge(ins, on='차대번호', how='left')
srt14.shape

(1136694, 70)

In [848]:
srt14['무부하매연판정1'].value_counts(dropna=False)['N']

289946

In [849]:
def func01(x):
    try:
        return x.value_counts(dropna=False)['N']
    except:
        return np.nan

In [850]:
grp14_1 = srt14.groupby(['차대번호']).agg({'무부하매연판정1':func01})
grp14_1

,무부하매연판정1
차대번호,
0142233972,4.00
01791301996,4.00
01901321635,2.00
01901321881,3.00
02794305028,1.00
...,...
kpace4ab19p059492,NaN
kpace4ab1ap064501,NaN
kpace4ab1bp097609,NaN


In [851]:
# 1N -> 1N : 48483
grp14_1[grp14_1['무부하매연판정1'] > 1].shape

(48483, 1)

In [852]:
grp14_2 = srt14.groupby(['차대번호']).agg({'무부하매연판정2':func01})
grp14_2

,무부하매연판정2
차대번호,
0142233972,2.00
01791301996,4.00
01901321635,2.00
01901321881,3.00
02794305028,2.00
...,...
kpace4ab19p059492,NaN
kpace4ab1ap064501,NaN
kpace4ab1bp097609,NaN


In [853]:
# 2N -> 2N : 20644
grp14_2[grp14_2['무부하매연판정2'] > 1].shape

(20644, 1)

In [854]:
grp14_3 = srt14.groupby(['차대번호']).agg({'무부하매연판정3':func01})
grp14_3

,무부하매연판정3
차대번호,
0142233972,1.00
01791301996,4.00
01901321635,NaN
01901321881,NaN
02794305028,2.00
...,...
kpace4ab19p059492,NaN
kpace4ab1ap064501,NaN
kpace4ab1bp097609,NaN


In [855]:
# 3N -> 3N : 21196
grp14_3[grp14_3['무부하매연판정3'] > 1].shape

(21196, 1)

In [856]:
grp14_4 = srt14.groupby(['차대번호']).agg({'검사판정':func01})
grp14_4

,검사판정
차대번호,
0142233972,6.00
01791301996,7.00
01901321635,3.00
01901321881,3.00
02794305028,3.00
...,...
kpace4ab19p059492,NaN
kpace4ab1ap064501,NaN
kpace4ab1bp097609,NaN


In [857]:
# N -> N : 57781
grp14_4[grp14_4['검사판정'] > 1].shape

(57781, 1)

In [858]:
grp14_5 = pd.DataFrame({
    '조건':['1n1n', '2n2n', '3n3n', 'nn'], 
    '해당조건의 차대번호 개수':[ grp14_1[grp14_1['무부하매연판정1'] > 1].shape[0], grp14_2[grp14_2['무부하매연판정2'] > 1].shape[0], grp14_3[grp14_3['무부하매연판정3'] > 1].shape[0], grp14_4[grp14_4['검사판정'] > 1].shape[0] ]
    })
grp14_5['전체 차대번호 개수'] = len(total_err['차대번호'].unique())
grp14_5['비율'] =  grp14_5['해당조건의 차대번호 개수'] / grp14_5['전체 차대번호 개수']
grp14_5

,조건,해당조건의 차대번호 개수,전체 차대번호 개수,비율
0,1n1n,48483,101388,0.48
1,2n2n,20644,101388,0.20
2,3n3n,21196,101388,0.21
3,nn,57781,101388,0.57


##### [출력] STD_BD42_14_RE_BRKWN_RT

In [859]:
grp14_5.columns

Index(['조건', '해당조건의 차대번호 개수', '전체 차대번호 개수', '비율'], dtype='object')

In [860]:
grp14_6 = grp14_5[[
    '조건', 
    '해당조건의 차대번호 개수', 
    '전체 차대번호 개수', 
    '비율', 
]]

In [862]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104633'

In [861]:
# grp14_dict = {
#     '조건':'CND',
#     '해당조건의 차대번호 개수':'CND_VIN_CNT',
#     '전체 차대번호 개수':'ALL_VIN_CNT',
#     '비율':'RT',
# }
# STD_BD42_14_RE_BRKWN_RT = grp14_6.rename(columns={grp14_dict})
# STD_BD42_14_RE_BRKWN_RT['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD42_14_RE_BRKWN_RT.columns)

# if export_opt == 'y':
#     STD_BD42_14_RE_BRKWN_RT.to_csv(os.path.join(df4_fold, f'STD_BD42_14_RE_BRKWN_RT_{today_date_dtl}.csv'), index=False)
#     print('STD_BD42_14_RE_BRKWN_RT 출력 완료.')
# else:
#     pass

In [863]:
# 0.0s
if export_opt == 'y':
    grp14_6.to_excel(os.path.join(df4_fold, f'STD_BD42_14_RE_BRKWN_RT_{today_date_dtl}.xlsx'), index=False)
else:
    pass

### [🔺] 4.2.15 STD_BD42_15_VHCL_ERSR_CYCL

### [🔺] 4.3.16 STD_BD43_16_EXHST_GAS_EXCS

In [864]:
srt8_jt1n.shape, srt8_jt2n.shape, srt8_jt3n.shape

((22468, 70), (1336, 70), (1354, 70))

In [865]:
srt16_1 = srt8_jt1n.copy()
srt16_1['초과1'] = srt16_1['무부하매연측정치1'] - srt16_1['무부하매연허용치1']
srt16_1['초과2'] = srt16_1['무부하매연측정치2'] - srt16_1['무부하매연허용치2']
srt16_1['초과3'] = srt16_1['무부하매연측정치3'] - srt16_1['무부하매연허용치3']
srt16_1[['초과1', '초과2', '초과3']].head()

,초과1,초과2,초과3
0,17.00,6.00,4.00
1,17.00,6.00,4.00
2,17.00,6.00,4.00
3,10.00,18.00,15.00
4,14.00,NaN,NaN


### [❌] 4.3.17 STD_BD43_17_OVHUL_FUEL

### [🔺] 4.3.18 STD_BD43_18_CRSH_BRKWN

In [866]:
srt8_jt1n.shape, srt8_jt2n.shape, srt8_jt3n.shape, srt8_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [867]:
srt18_jt1n = srt8_jt1n.copy()
srt18_jt2n = srt8_jt2n.copy()
srt18_jt3n = srt8_jt3n.copy()
srt18_jt4n = srt8_jt4n.copy()
srt18_jt1n.shape, srt18_jt2n.shape, srt18_jt3n.shape, srt18_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [868]:
srt18_jt1n.columns

Index(['차량번호', '차대번호', '점검일자', '전문정비사업소명', '정비접수번호', '정비내용', '정비부품명', '제원관리번호',
       '연식', '차종', '용도', '법정동코드', '배출가스등급', '연료', '제작사명', '차종유형', '차명',
       '자동차형식', '엔진형식', '총중량', '배기량', '엔진출력', '시도명', '시군구명', '일련번호', '중량',
       '엔진배기량', '제작일자', '최초등록일', '전문정비사업자코드', '정비여부', '시도코드', '시군구코드', '상세일자',
       '교환유무', '점검조정유무', '분해탈착유무', '정비불가능유무', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '전문정비사업자_시도명', '전문정비사업자_시군구명', '전문정비사업자_시도코드',
       '전문정비사업자_시군구코드', '검사일자', '주행거리', '검사종류', '검사방법', '검사판정', '무부하매연측정치1',
       '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2',
       '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4',
       '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6',
       '무부하매연허용치6', '무부하매연판정6', '정기정밀구분'],
      dtype='object')

In [916]:
def func18_grp(df):
    grp = df.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
    grp = grp.rename(columns={'차량번호':'정비건수'})
    grp = grp.sort_values('정비건수', ascending=False).reset_index(drop=True)
    # grp.loc[grp.shape[0], ['정비내용', '정비건수']] = ['합계', grp['정비건수'].sum()]
    return grp

In [917]:
for i, one in enumerate([srt18_jt1n, srt18_jt2n, srt18_jt3n, srt18_jt4n]):
    globals()[f'grp18_{i+1}'] = func18_grp(one)

In [918]:
grp18_1.tail()

,정비내용,정비부품명,정비건수
30,연료증발가스방지장치,정화조절밸브(PurgeControlValve),89
31,엔진계통,기화기(믹서포함),82
32,점화계통,점화플러그,78
33,점화계통,배전기(Distributor),76
34,점화계통,점화배선,76


In [872]:
# grp18_1 = srt18_jt1n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp18_1 = grp18_1.rename(columns={'차량번호':'정비건수'})
# grp18_1.loc[grp18_1.shape[0], ['정비내용', '정비건수']] = ['합계', grp18_1['정비건수'].sum()]
# display(grp18_1.tail())
# print()

# grp18_2 = srt18_jt2n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp18_2 = grp18_2.rename(columns={'차량번호':'정비건수'})
# grp18_2.loc[grp18_2.shape[0], ['정비내용', '정비건수']] = ['합계', grp18_2['정비건수'].sum()]
# display(grp18_2.tail())
# print()

# grp18_3 = srt18_jt3n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp18_3 = grp18_3.rename(columns={'차량번호':'정비건수'})
# grp18_3.loc[grp18_3.shape[0], ['정비내용', '정비건수']] = ['합계', grp18_3['정비건수'].sum()]
# display(grp18_3.tail())
# print()

# grp18_4 = srt18_jt4n.groupby(['정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp18_4 = grp18_4.rename(columns={'차량번호':'정비건수'})
# grp18_4.loc[grp18_4.shape[0], ['정비내용', '정비건수']] = ['합계', grp18_4['정비건수'].sum()]
# display(grp18_4.tail())
# print()

In [919]:
grp18_1['불합격내역'] = '매연'
grp18_2['불합격내역'] = 'CO'
grp18_3['불합격내역'] = 'HC'
grp18_4['불합격내역'] = 'NOx'

In [920]:
grp18 = pd.concat([grp18_1, grp18_2, grp18_3, grp18_4], ignore_index=True)
grp18.tail()

,정비내용,정비부품명,정비건수,불합격내역
135,배출가스전환장치,매연여과장치(DPF),11,NOx
136,엔진계통,"연료분사펌프(조속기조정,봉인)",10,NOx
137,엔진계통,노즐(디젤인젝터포함),10,NOx
138,흡기계통,과급기,10,NOx
139,연료증발가스방지장치,정화조절밸브(PurgeControlValve),8,NOx


In [921]:
grp18.columns

Index(['정비내용', '정비부품명', '정비건수', '불합격내역'], dtype='object')

##### [출력] STD_BD43_18_CRSH_BRKWN 

In [873]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104633'

In [ ]:
# STD_BD43_18_CRSH_BRKWN = grp18[[
#     '정비내용',
#     '정비부품명',
#     '정비건수',
#     '불합격내역',
# ]]
# grp18_dict = {
#     '정비내용':'OVHUL_CN',
#     '정비부품명':'OVHUL_CMPNT_NM',
#     '정비건수':'OVHUL_NOCS',
#     '불합격내역':'FAIL_BRKWN',
# }
# STD_BD43_18_CRSH_BRKWN = STD_BD43_18_CRSH_BRKWN.rename(columns=grp18_dict)
# STD_BD43_18_CRSH_BRKWN['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD43_18_CRSH_BRKWN.columns)

# if export_opt == 'y':
#     STD_BD43_18_CRSH_BRKWN.to_csv(os.path.join(df4_fold, f'STD_BD43_18_CRSH_BRKWN_{today_date_dtl}.csv'), index=False)
#     print('STD_BD43_18_CRSH_BRKWN 출력 완료.')
# else:
#     pass

In [874]:
if export_opt == 'y':
    with pd.ExcelWriter(os.path.join(df4_fold, f'STD_BD43_18_CRSH_BRKWN_{today_date_dtl}.xlsx')) as writer:
        grp18_1.to_excel(writer, sheet_name='판정1_매연', index=False)
        grp18_2.to_excel(writer, sheet_name='판정1_CO', index=False)
        grp18_3.to_excel(writer, sheet_name='판정2_HC', index=False)
        grp18_4.to_excel(writer, sheet_name='판정3_NOx', index=False)
else:
    pass

### [❌] 4.3.19 STD_BD43_19_OVHUL_BRKWN

### [🔺] 4.3.20 STD_BD43_20_PRPT_OVHUL_YRIDNW

In [875]:
srt8_jt1n.shape, srt8_jt2n.shape, srt8_jt3n.shape, srt8_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [876]:
srt20_jt1n = srt8_jt1n.copy()
srt20_jt2n = srt8_jt2n.copy()
srt20_jt3n = srt8_jt3n.copy()
srt20_jt4n = srt8_jt4n.copy()
srt20_jt1n.shape, srt20_jt2n.shape, srt20_jt3n.shape, srt20_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [877]:
check_col20 = '연식'
check_col20 in srt20_jt1n.columns

True

In [878]:
srt20_jt1n[check_col20].isnull().sum(), srt20_jt2n[check_col20].isnull().sum(), srt20_jt3n[check_col20].isnull().sum(), srt20_jt4n[check_col20].isnull().sum()

(0, 0, 0, 0)

In [922]:
def func20_grp(df):
    grp = df.groupby([check_col20, '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
    grp = grp.rename(columns={'차량번호':'정비건수'})
    grp = grp.sort_values(['정비내용', '정비부품명', check_col20, '정비건수'], ascending=[False, False, True, False])
    # grp.loc[grp.shape[0], [check_col20, '정비건수']] = ['합계', grp['정비건수'].sum()]
    return grp

In [923]:
for i, one in enumerate([srt20_jt1n, srt20_jt2n, srt20_jt3n, srt20_jt4n]):
    globals()[f'grp20_{i+1}'] = func20_grp(one)

In [924]:
grp20_1.head()

,연식,정비내용,정비부품명,정비건수
10,1988.00,흡기계통,에어크리너,1
16,1989.00,흡기계통,에어크리너,2
29,1990.00,흡기계통,에어크리너,8
41,1991.00,흡기계통,에어크리너,8
58,1992.00,흡기계통,에어크리너,16


In [925]:
grp20_1.tail()

,연식,정비내용,정비부품명,정비건수
582,2009.00,NaN,NaN,7
617,2010.00,NaN,NaN,10
652,2011.00,NaN,NaN,4
687,2012.00,NaN,NaN,1
756,2014.00,NaN,NaN,7


In [883]:
# grp20_1 = srt8_jt1n.groupby([check_col20, '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp20_1 = grp20_1.rename(columns={'차량번호':'정비건수'})
# grp20_1.loc[grp20_1.shape[0], [check_col20, '정비건수']] = ['합계', grp20_1['정비건수'].sum()]
# grp20_1 = grp20_1.sort_values([check_col20, '정비건수'], ascending=[True, False])
# display(grp20_1.tail())
# print()
# grp20_2 = srt8_jt2n.groupby([check_col20, '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp20_2 = grp20_2.rename(columns={'차량번호':'정비건수'})
# grp20_2.loc[grp20_2.shape[0], [check_col20, '정비건수']] = ['합계', grp20_2['정비건수'].sum()]
# grp20_2 = grp20_2.sort_values([check_col20, '정비건수'], ascending=[True, False])
# display(grp20_2.tail())
# print()
# grp20_3 = srt8_jt3n.groupby([check_col20, '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp20_3 = grp20_3.rename(columns={'차량번호':'정비건수'})
# grp20_3.loc[grp20_3.shape[0], [check_col20, '정비건수']] = ['합계', grp20_3['정비건수'].sum()]
# grp20_3 = grp20_3.sort_values([check_col20, '정비건수'], ascending=[True, False])
# display(grp20_3.tail())
# print()
# grp20_4 = srt8_jt4n.groupby([check_col20, '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp20_4 = grp20_4.rename(columns={'차량번호':'정비건수'})
# grp20_4.loc[grp20_4.shape[0], [check_col20, '정비건수']] = ['합계', grp20_4['정비건수'].sum()]
# grp20_4 = grp20_4.sort_values([check_col20, '정비건수'], ascending=[True, False])
# display(grp20_4.tail())
# print()

In [926]:
grp20_1['불합격내역'] = '매연'
grp20_2['불합격내역'] = 'CO'
grp20_3['불합격내역'] = 'HC'
grp20_4['불합격내역'] = 'NOx'

In [927]:
grp20 = pd.concat([grp20_1, grp20_2, grp20_3, grp20_4], ignore_index=True)
grp20.tail()

,연식,정비내용,정비부품명,정비건수,불합격내역
2305,2000.00,NaN,NaN,4,NOx
2306,2001.00,NaN,NaN,1,NOx
2307,2002.00,NaN,NaN,5,NOx
2308,2003.00,NaN,NaN,1,NOx
2309,2004.00,NaN,NaN,1,NOx


In [928]:
grp20.columns

Index(['연식', '정비내용', '정비부품명', '정비건수', '불합격내역'], dtype='object')

##### [출력] STD_BD43_20_PRPT_OVHUL_YRIDNW 

In [884]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104633'

In [ ]:
# STD_BD43_20_PRPT_OVHUL_YRIDNW = grp20[[
#     '연식', 
#     '정비내용',
#     '정비부품명',
#     '정비건수',
#     '불합격내역',
# ]]
# grp20_dict = {
#     '연식':'YRIDNW', 
#     '정비내용':'OVHUL_CN',
#     '정비부품명':'OVHUL_CMPNT_NM',
#     '정비건수':'OVHUL_NOCS',
#     '불합격내역':'FAIL_BRKWN',
# }
# STD_BD43_20_PRPT_OVHUL_YRIDNW = STD_BD43_20_PRPT_OVHUL_YRIDNW.rename(columns=grp20_dict)
# STD_BD43_20_PRPT_OVHUL_YRIDNW['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD43_20_PRPT_OVHUL_YRIDNW.columns)

# if export_opt == 'y':
#     STD_BD43_20_PRPT_OVHUL_YRIDNW.to_csv(os.path.join(df4_fold, f'STD_BD43_20_PRPT_OVHUL_YRIDNW_{today_date_dtl}.csv'), index=False)
#     print('STD_BD43_20_PRPT_OVHUL_YRIDNW 출력 완료.')
# else:
#     pass

In [885]:
if export_opt == 'y':
    with pd.ExcelWriter(os.path.join(df4_fold, f'STD_BD43_20_PRPT_OVHUL_YRIDNW_{today_date_dtl}.xlsx')) as writer:
        grp20_1.to_excel(writer, sheet_name='판정1_매연', index=False)
        grp20_2.to_excel(writer, sheet_name='판정1_CO', index=False)
        grp20_3.to_excel(writer, sheet_name='판정2_HC', index=False)
        grp20_4.to_excel(writer, sheet_name='판정3_NOx', index=False)
else:
    pass

### [🔺] 4.3.21 STD_BD43_21_PRPT_OVHUL_DRVNG

In [886]:
srt8_jt1n.shape, srt8_jt2n.shape, srt8_jt3n.shape, srt8_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [887]:
check_col21 = '주행거리'
check_col21 in srt8_jt1n.columns

True

In [888]:
srt8_jt1n[check_col21].isnull().sum(), srt8_jt2n[check_col21].isnull().sum(), srt8_jt3n[check_col21].isnull().sum(), srt8_jt4n[check_col21].isnull().sum()

(0, 0, 0, 0)

In [889]:
# 주행거리 범위 : -9999 ~ 3447611
print(srt8_jt1n[check_col21].describe())
print()
print(srt8_jt2n[check_col21].describe())
print()
print(srt8_jt3n[check_col21].describe())
print()
print(srt8_jt4n[check_col21].describe())
print()

count    22468.00
mean    220427.29
std     112613.32
min      -9999.00
25%     154825.00
50%     204247.00
75%     269067.00
max     993791.00
Name: 주행거리, dtype: float64

count     1336.00
mean    196139.64
std      91135.40
min          1.00
25%     128817.50
50%     186742.00
75%     248980.75
max     658976.00
Name: 주행거리, dtype: float64

count     1354.00
mean    213014.21
std      97079.81
min      -9999.00
25%     143188.00
50%     203295.00
75%     277210.00
max     658976.00
Name: 주행거리, dtype: float64

count     2208.00
mean    220270.98
std     102752.37
min      -9999.00
25%     152598.75
50%     205566.00
75%     274482.00
max     666518.00
Name: 주행거리, dtype: float64



#### 전처리
- 주행거리 음수 -> 0으로 수정
- 주행거리 구간으로 지정
    - 주행거리 범위 : -9999 ~ 3447611
    - 구간(1년 주행거리 기준) : 1만3천 단위로 구분( 국가통계포털 일일평균 주행거리 참고(2022 기준) )

In [890]:
srt21_jt1n = srt8_jt1n.copy()
srt21_jt2n = srt8_jt2n.copy()
srt21_jt3n = srt8_jt3n.copy()
srt21_jt4n = srt8_jt4n.copy()
srt21_jt1n.shape, srt21_jt2n.shape, srt21_jt3n.shape, srt21_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [891]:
# print(srt21_jt1n.loc[srt21_jt1n[check_col21] < 0].shape)
# print()
# srt21_jt1n.loc[srt21_jt1n[check_col21] < 0, check_col21] = 0
# print(srt21_jt1n[check_col21].describe())
# print()
# num21_1 = math.ceil(srt21_jt1n[check_col21].max() / 13000)
# print(num21_1)
# print()
# div = 13000
# for i in range(num21_1):
#     srt21_jt1n.loc[(srt21_jt1n[check_col21] >= i*div) & (srt21_jt1n[check_col21] < (i+1)*div), check_col21 + '_구분'] = f'{i*div:,}km이상 {(i+1)*div:,}km미만'
#     srt21_jt1n.loc[(srt21_jt1n[check_col21] >= i*div) & (srt21_jt1n[check_col21] < (i+1)*div), check_col21 + '_구분번호'] = i

# print(srt21_jt1n[check_col21 + '_구분'].unique())
# print(srt21_jt1n[check_col21 + '_구분번호'].unique())
# len(srt21_jt1n[check_col21 + '_구분번호'].unique())

In [892]:
def func21(df):
    div = 13000
    col = '주행거리'
    df.loc[df[col] < 0, col] = 0
    num = math.ceil(df[col].max() / div)
    for i in range(num):
        df.loc[(df[col] >= i*div) & (df[col] < (i+1)*div), [col + '_구분번호', col + '_구분']] = [i, f'{i*div:,}km이상 {(i+1)*div:,}km미만']
    return df

In [893]:
for one in [srt21_jt1n, srt21_jt2n, srt21_jt3n, srt21_jt4n]:
    one = func21(one)

#### 분석

In [929]:
def func21_grp(df):
    grp = df.groupby(['주행거리_구분번호', '주행거리_구분', '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
    grp = grp.rename(columns={'차량번호':'정비건수'})
    grp = grp.sort_values(['정비내용', '정비부품명', '주행거리_구분번호'], ascending=[False, False, True])
    # grp.loc[grp.shape[0], ['주행거리_구분', '정비건수']] = ['합계', grp['정비건수'].sum()]
    grp = grp.drop('주행거리_구분번호', axis=1)
    return grp

In [930]:
for i, one in enumerate([srt21_jt1n, srt21_jt2n, srt21_jt3n, srt21_jt4n]):
    globals()[f'grp21_{i+1}'] = func21_grp(one)

In [931]:
display(grp21_1.tail())
display(grp21_2.tail())
display(grp21_3.tail())
display(grp21_4.tail())

,주행거리_구분,정비내용,정비부품명,정비건수
1039,"403,000km이상 416,000km미만",NaN,NaN,2
1059,"416,000km이상 429,000km미만",NaN,NaN,1
1110,"468,000km이상 481,000km미만",NaN,NaN,1
1133,"494,000km이상 507,000km미만",NaN,NaN,1
1322,"806,000km이상 819,000km미만",NaN,NaN,1


,주행거리_구분,정비내용,정비부품명,정비건수
273,"208,000km이상 221,000km미만",NaN,NaN,1
341,"234,000km이상 247,000km미만",NaN,NaN,1
355,"247,000km이상 260,000km미만",NaN,NaN,1
372,"260,000km이상 273,000km미만",NaN,NaN,2
424,"299,000km이상 312,000km미만",NaN,NaN,2


,주행거리_구분,정비내용,정비부품명,정비건수
250,"195,000km이상 208,000km미만",NaN,NaN,2
323,"234,000km이상 247,000km미만",NaN,NaN,1
339,"247,000km이상 260,000km미만",NaN,NaN,1
355,"260,000km이상 273,000km미만",NaN,NaN,1
438,"299,000km이상 312,000km미만",NaN,NaN,2


,주행거리_구분,정비내용,정비부품명,정비건수
339,"221,000km이상 234,000km미만",NaN,NaN,1
403,"247,000km이상 260,000km미만",NaN,NaN,1
422,"260,000km이상 273,000km미만",NaN,NaN,2
457,"273,000km이상 286,000km미만",NaN,NaN,1
603,"403,000km이상 416,000km미만",NaN,NaN,3


In [897]:
# grp21_1 = srt21_jt1n.groupby(['주행거리_구분번호', '주행거리_구분', '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp21_1 = grp21_1.rename(columns={'차량번호':'정비건수'})
# grp21_1.loc[grp21_1.shape[0], ['주행거리_구분번호', '정비건수']] = ['합계', grp21_1['정비건수'].sum()]
# grp21_1 = grp21_1.drop('주행거리_구분번호', axis=1)
# display(grp21_1.tail())
# print()

# grp21_2 = srt21_jt2n.groupby(['주행거리_구분번호', '주행거리_구분', '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp21_2 = grp21_2.rename(columns={'차량번호':'정비건수'})
# grp21_2.loc[grp21_2.shape[0], ['주행거리_구분번호', '정비건수']] = ['합계', grp21_2['정비건수'].sum()]
# grp21_2 = grp21_2.drop('주행거리_구분번호', axis=1)
# display(grp21_2.tail())
# print()

# grp21_3 = srt21_jt3n.groupby(['주행거리_구분번호', '주행거리_구분', '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp21_3 = grp21_3.rename(columns={'차량번호':'정비건수'})
# grp21_3.loc[grp21_3.shape[0], ['주행거리_구분번호', '정비건수']] = ['합계', grp21_3['정비건수'].sum()]
# grp21_3 = grp21_3.drop('주행거리_구분번호', axis=1)
# display(grp21_3.tail())
# print()

# grp21_4 = srt21_jt4n.groupby(['주행거리_구분번호', '주행거리_구분', '정비내용', '정비부품명'], dropna=False)['차량번호'].count().reset_index()
# grp21_4 = grp21_4.rename(columns={'차량번호':'정비건수'})
# grp21_4.loc[grp21_4.shape[0], ['주행거리_구분번호', '정비건수']] = ['합계', grp21_4['정비건수'].sum()]
# grp21_4 = grp21_4.drop('주행거리_구분번호', axis=1)
# display(grp21_4.tail())
# print()

In [932]:
grp21_1['불합격내역'] = '매연'
grp21_2['불합격내역'] = 'CO'
grp21_3['불합격내역'] = 'HC'
grp21_4['불합격내역'] = 'NOx'

In [933]:
grp21 = pd.concat([grp21_1, grp21_2, grp21_3, grp21_4], ignore_index=True)
grp21.tail()

,주행거리_구분,정비내용,정비부품명,정비건수,불합격내역
3175,"221,000km이상 234,000km미만",NaN,NaN,1,NOx
3176,"247,000km이상 260,000km미만",NaN,NaN,1,NOx
3177,"260,000km이상 273,000km미만",NaN,NaN,2,NOx
3178,"273,000km이상 286,000km미만",NaN,NaN,1,NOx
3179,"403,000km이상 416,000km미만",NaN,NaN,3,NOx


In [934]:
grp21.columns

Index(['주행거리_구분', '정비내용', '정비부품명', '정비건수', '불합격내역'], dtype='object')

##### [출력] STD_BD43_21_PRPT_OVHUL_DRVNG 

In [898]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20240119_AM_104634'

In [ ]:
# STD_BD43_21_PRPT_OVHUL_DRVNG = grp21[[
#     '주행거리_구분', 
#     '정비내용',
#     '정비부품명',
#     '정비건수',
#     '불합격내역',
# ]]
# grp21_dict = {
#     '주행거리_구분':'DRVNG_DSTNC_SE',
#     '정비내용':'OVHUL_CN',
#     '정비부품명':'OVHUL_CMPNT_NM',
#     '정비건수':'OVHUL_NOCS',
#     '불합격내역':'FAIL_BRKWN',
# }
# STD_BD43_21_PRPT_OVHUL_DRVNG = STD_BD43_21_PRPT_OVHUL_DRVNG.rename(columns=grp21_dict)
# STD_BD43_21_PRPT_OVHUL_DRVNG['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD43_21_PRPT_OVHUL_DRVNG.columns)

# if export_opt == 'y':
#     STD_BD43_21_PRPT_OVHUL_DRVNG.to_csv(os.path.join(df4_fold, f'STD_BD43_21_PRPT_OVHUL_DRVNG_{today_date_dtl}.csv'), index=False)
#     print('STD_BD43_21_PRPT_OVHUL_DRVNG 출력 완료.')
# else:
#     pass

In [899]:
if export_opt == 'y':
    with pd.ExcelWriter(os.path.join(df4_fold, f'STD_BD43_21_PRPT_OVHUL_DRVNG_{today_date_dtl}.xlsx')) as writer:
        grp21_1.to_excel(writer, sheet_name='판정1_매연', index=False)
        grp21_2.to_excel(writer, sheet_name='판정1_CO', index=False)
        grp21_3.to_excel(writer, sheet_name='판정2_HC', index=False)
        grp21_4.to_excel(writer, sheet_name='판정3_NOx', index=False)
else:
    pass

### [❌] 4.3.22 STD_BD43_22_PRPT_OVHUL_MNFCTR

In [900]:
# 제작사명 결측값이 너무 많음

In [901]:
srt22_jt1n = srt8_jt1n.copy()
srt22_jt2n = srt8_jt2n.copy()
srt22_jt3n = srt8_jt3n.copy()
srt22_jt4n = srt8_jt4n.copy()
srt22_jt1n.shape, srt22_jt2n.shape, srt22_jt3n.shape, srt22_jt4n.shape

((22468, 70), (1336, 70), (1354, 70), (2208, 70))

In [902]:
check_col22 = '제작사명'
check_col22 in srt22_jt1n.columns

True

In [903]:
srt22_jt1n[check_col22].isnull().sum(), srt22_jt2n[check_col22].isnull().sum(), srt22_jt3n[check_col22].isnull().sum(), srt22_jt4n[check_col22].isnull().sum()

(22141, 1324, 1351, 2195)

### [❌] 4.3.23 STD_BD43_23_PRPT_OVHUL_ENLF_CYCL

# 과제4 코드 end